# BIP v10.16.8 - Bond Invariant Principle

**Extracting moral knowledge from 5,000 years of human ethical reasoning**

This notebook implements a complete pipeline for:
1. Loading multi-lingual ancient and modern ethical texts
2. Extracting moral bonds (agent, patient, obligation type)
3. Training cross-cultural moral embeddings
4. Analyzing ethical patterns across traditions

**Bond Extraction Training Data (NEW in v10.14.4):**
- [ETHICS](https://github.com/hendrycks/ethics): 130K scenarios across 5 categories
- [Scruples](https://github.com/allenai/scruples): 32K real-life anecdotes with ethical judgments
- [EthicsSuite](https://github.com/llm-ethics/ethicssuite): 20K complex contextualized moral situations

**Corpus Coverage:**

*Ancient & Classical:*
- Hebrew (Biblical, Mishnaic, Talmudic) - Sefaria (88 texts)
- Aramaic (Talmud Bavli) - Sefaria
- Classical Chinese (Confucian, Daoist, Legalist, Buddhist) - ctext.org, CBETA
- Arabic (Quranic) - Tanzil
- Sanskrit (Dharmashastra, Upanishads, Itihasa) - GitHub
- Pali (Theravada Canon) - SuttaCentral
- Greek & Latin (Stoic, Platonic, Aristotelian) - Perseus Digital Library

*Western Philosophy & Religion:*
- English: Kant, Mill, Spinoza, Aristotle, Plato, Epictetus, Marcus Aurelius (Gutenberg)
- Bible KJV: Complete (80 books incl. Apocrypha)
- Luther's Catechisms (Small & Large)
- French: Montaigne, Voltaire, Rousseau (Gutenberg)
- Spanish: Cervantes Don Quixote (Gutenberg)
- Italian: Machiavelli, Dante (Gutenberg)

*Modern Ethics:*
- Dear Abby advice columns (68K letters)
- hendrycks/ethics dataset (134K scenarios)
- Folklore & Native American traditions (Ashliman Folktexts)

In [ ]:
# @title 1. Configuration & Setup { display-mode: "form" }

# @markdown ---
# @markdown ### Version
BIP_VERSION = "10.16.5"  # @param {type:"string"}
# @markdown Central version number - change to update all references
# @markdown ## Data Source Configuration

DATA_MODE = "Update missing"  # @param ["Refresh all", "Update missing", "Cache only"]
# @markdown - **Refresh all**: Re-download everything from source (slow, ~2hrs)
# @markdown - **Update missing**: Use cache, download only what's missing (recommended)
# @markdown - **Cache only**: Use only cached data, fail if missing

DRIVE_FOLDER = f"BIP_v{BIP_VERSION}"  # @param {type:"string"}
# @markdown **Folder name for persistent storage** (edit above to change)

# Derive flags from DATA_MODE
USE_DRIVE_DATA = True  # Always use Drive for caching
REFRESH_DATA_FROM_SOURCE = DATA_MODE == "Refresh all"
CACHE_ONLY = DATA_MODE == "Cache only"
# @markdown ---
# @markdown ## Model Backbone
BACKBONE = "LaBSE"  # @param ["MiniLM", "LaBSE", "XLM-R-base", "XLM-R-large"]
# @markdown - **MiniLM**: Fast, 118M params, good baseline
# @markdown - **LaBSE**: Best cross-lingual alignment, 471M params (recommended)
# @markdown - **XLM-R-base**: Strong multilingual, 270M params
# @markdown - **XLM-R-large**: Strongest representations, 550M params

# @markdown ---
# @markdown ## Output Options
CREATE_DOWNLOAD_ZIP = False  # @param {type:"boolean"}
# @markdown - **CREATE_DOWNLOAD_ZIP**: Create and download a zip file of results (optional)
# @markdown - Results are always persisted to Google Drive regardless of this setting

SKIP_TRAINING = False  # @param {type:"boolean"}  # v10.16.6: MUST train to apply confusion loss
# @markdown - **SKIP_TRAINING**: Skip Cell 7 training, load models from Drive instead
# @markdown - Use this to run evaluation (Cell 8+) on previously trained models

# @markdown ---
# @markdown ## Training Hyperparameters
FREEZE_ENCODER = False  # @param {type:"boolean"} # v10.16.2: Unfreeze for invariance
# @markdown - **FREEZE_ENCODER**: Only train probe head (recommended for stability)

USE_AMP = False  # @param {type:"boolean"} # DISABLED: gradient reversal causes NaN in float16
# @markdown - **USE_AMP**: Use Automatic Mixed Precision (float16). Disable if you get NaN errors.
# @markdown - Unfrozen: Fine-tune entire encoder (471M params, risk of catastrophic forgetting)

LEARNING_RATE = 1e-5  # @param {type:"number"}
# @markdown - **Frozen encoder**: 1e-4 to 1e-3 works well
# @markdown - **Unfrozen encoder**: Use 1e-5 to 5e-6 (lower = more stable)

WARMUP_RATIO = 0.1  # @param {type:"number"}
# @markdown - Fraction of training for learning rate warmup (0.0 to 0.2)

GRADIENT_CLIP = 1.0  # @param {type:"number"}
# @markdown - Max gradient norm (prevents exploding gradients, 0 = disabled)

NUM_EPOCHS = 15  # @param {type:"integer"} # v10.16.2: More epochs for fine-tuning
# @markdown - Number of training epochs per split

EARLY_STOPPING_PATIENCE = 5  # @param {type:"integer"}
# @markdown - Stop if no improvement for N epochs (0 = disabled)

ADV_WARMUP_EPOCHS = 2  # @param {type:"integer"}

# @markdown ---
# @markdown ### v10.15.1.3: Per-Split Parameter Tuning
PER_SPLIT_TUNING = True  # @param {type:"boolean"}
SPLIT_ADV_LAMBDA = {
    # v10.16.4: Increased all values for stronger adversarial training
    # Previous values were too weak (0.35 = only 23% of max strength)
    "mixed_baseline": 1.5,      # Was 0.35 - now full strength
    "ancient_to_modern": 1.2,   # Was 0.30
    "stoic_to_confucian": 1.5,  # Was 0.50
    "hebrew_to_arabic": 1.5,    # Was 0.40
    "chinese_to_greek": 1.5,    # Was 0.40
}
# @markdown ---
# @markdown ### v10.15.1.4: Encoder Fine-Tuning (KEY CHANGE)
# @markdown Previous versions froze the encoder. Now we can fine-tune it.
UNFREEZE_ENCODER = True  # @param {type:"boolean"} # v10.16.2: Enable fine-tuning
# @markdown - True: Fine-tune LaBSE to learn language-invariant moral structure

UNFREEZE_AFTER_EPOCHS = 3  # @param {type:"integer"} # v10.16.2: Earlier unfreeze
# @markdown - Epochs before unfreezing (if UNFREEZE_ENCODER=True)

UNFREEZE_LAYERS = 4  # @param {type:"integer"} # v10.16.2: More layers
# @markdown - Only unfreeze top N transformer layers (0=all)

ENCODER_LR_SCALE = 0.1  # @param {type:"number"}
# @markdown - Learning rate multiplier for encoder (0.1 = 10x smaller)
# @markdown - False: Probe-only mode (test pretrained representations)

ENCODER_LR = 1e-6  # @param {type:"number"} # v10.16.2: Slightly higher for fine-tuning
# @markdown Learning rate for encoder (1000x lower than head LR to prevent NaN)

HEAD_LR = 1e-3  # @param {type:"number"}
# @markdown Learning rate for classification/adversarial heads

UNFREEZE_AFTER_EPOCHS = 2  # @param {type:"integer"}
# @markdown Epochs to train heads before unfreezing encoder (warmup)

GRADIENT_ACCUMULATION_STEPS = 4  # @param {type:"integer"}
# @markdown Accumulate gradients to simulate larger batch (memory efficiency)

# @markdown ---
# @markdown ### v10.15.1.4: Stronger Adversarial Heads
ADV_HIDDEN_DIM = 1024  # @param {type:"integer"}
# @markdown Hidden dimension for adversarial classifier (was 256)

ADV_NUM_LAYERS = 4  # @param {type:"integer"}

# @markdown **v10.16.7: Multi-Head Adversarial Training**
NUM_ADV_HEADS = 4  # @param {type:"integer"}
# @markdown - Number of independent adversarial heads (prevents adversarial hiding)
# @markdown - Each head has different architecture (varying width/depth)
# @markdown - Encoder must fool ALL heads simultaneously
# @markdown Number of layers in adversarial head (was 2)

ADV_DROPOUT = 0.4  # @param {type:"number"}
# @markdown Dropout in adversarial heads for regularization

# @markdown - Epochs to ramp adversarial strength (longer = more stable)

ADV_MAX_LAMBDA = 1.5  # @param {type:"number"} # v10.16.2: Stronger adversarial # REDUCED from 1.0 for stability
# @markdown - Max adversarial weight (0.7 recommended for strong disentanglement)

# @markdown ### v10.16.5: Confusion Loss (KEY FIX)
# @markdown Forces embeddings where NO classifier can predict language/period
USE_CONFUSION_LOSS = True  # @param {type:"boolean"}
CONFUSION_WEIGHT = 2.0  # @param {type:"number"}  # v10.16.6: Increased from 0.5 for stronger invariance

# @markdown **v10.16.8: Variational Information Bottleneck (VIB)**
USE_VIB = True  # @param {type:"boolean"}
# @markdown - Forces stochastic, compressed representation
# @markdown - Encoder can't hide spurious info in deterministic features
VIB_BETA = 0.001  # @param {type:"number"}
# @markdown - Weight for KL divergence loss (start small: 0.0001-0.01)
# @markdown - Higher = more compression, less language info, but may hurt bond prediction
VIB_ANNEAL_EPOCHS = 5  # @param {type:"integer"}
# @markdown - Epochs to linearly increase VIB_BETA from 0 to full value
# @markdown - Weight for entropy maximization (forces uniform predictions)
# @markdown - This prevents adversarial heads from "learning to fail"

# @markdown ### v10.15.1: Surface Invariance Training
CONTRASTIVE_WEIGHT = 0.5  # @param {type:"number"} # v10.16.1: Increased
# @markdown - Weight for contrastive loss (same moral, different surface)

CONTRASTIVE_TEMPERATURE = 0.07  # @param {type:"number"}
# @markdown - InfoNCE temperature (lower = harder negatives)

SURFACE_AUGMENT = True  # @param {type:"boolean"}
# @markdown - Create surface-perturbed training pairs

AUGMENT_SIMILARITY_WEIGHT = 0.2  # @param {type:"number"}
# @markdown - Weight for augmented pair similarity loss

# @markdown ---
# @markdown ### v10.16.2: Encoder Fine-Tuning Strategy (KEY CHANGE)
# @markdown The frozen encoder preserves language info (99.6% lang acc).
# @markdown Unfreezing allows the model to learn language-invariant representations.

USE_GRADIENT_CHECKPOINTING = True  # @param {type:"boolean"}
# @markdown - Save memory during encoder fine-tuning (slower but fits in VRAM)

ENCODER_WARMUP_EPOCHS = 3  # @param {type:"integer"}
# @markdown - Epochs to warm up encoder LR after unfreezing

MIN_LANG_ACC_TARGET = 0.20  # @param {type:"number"}
# @markdown - Target language accuracy (0.125 = random for 8 languages)

# @markdown ---
# @markdown ### v10.16.1: Structural Contrastive Training (NEW)
USE_STRUCTURAL_CONTRASTIVE = True  # @param {type:"boolean"}
# @markdown - Enable structural perturbation contrastive loss

STRUCTURAL_CONTRASTIVE_WEIGHT = 0.4  # @param {type:"number"}
# @markdown - Weight for structural contrastive loss (push apart)

STRUCTURAL_CONTRASTIVE_MARGIN = 0.8  # @param {type:"number"}
# @markdown - Minimum distance for structural perturbations

# @markdown ---
# @markdown ### v10.16.1: Triplet Loss (NEW)
USE_TRIPLET_LOSS = True  # @param {type:"boolean"}
# @markdown - Enable triplet loss (anchor, surface+, structural-)

TRIPLET_MARGIN = 0.5  # @param {type:"number"}
# @markdown - Triplet loss margin

TRIPLET_WEIGHT = 0.3  # @param {type:"number"}
# @markdown - Weight for triplet loss

# @markdown ---
# @markdown ### v10.16.1: Ratio Regularization (NEW)
USE_RATIO_LOSS = True  # @param {type:"boolean"}
# @markdown - Encourage structural distance > surface distance

TARGET_RATIO = 2.0  # @param {type:"number"}
# @markdown - Target ratio: structural/surface > this value

RATIO_LOSS_WEIGHT = 0.2  # @param {type:"number"}
# @markdown - Weight for ratio loss

Z_DIM = 64  # @param {type:"integer"} # v10.16.1: Reduced for stronger abstraction
# @markdown - Bond embedding dimension (smaller = more abstraction)

# Backbone configurations
BACKBONE_CONFIGS = {
    "MiniLM": {
        "model_name": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        "hidden_size": 384,
        "recommended_batch": {
            "L4/A100": 4096,
            "T4": 512,
            "2xT4": 1024,
            "SMALL": 128,
            "MINIMAL/CPU": 64,
        },
    },
    "LaBSE": {
        "model_name": "sentence-transformers/LaBSE",
        "hidden_size": 768,
        "recommended_batch": {
            "L4/A100": 4096,  # Increased: only using 2.1/22.5GB at 256
            "T4": 512,
            "2xT4": 1024,
            "SMALL": 128,
            "MINIMAL/CPU": 64,
        },
    },
    "XLM-R-base": {
        "model_name": "xlm-roberta-base",
        "hidden_size": 768,
        "recommended_batch": {
            "L4/A100": 2048,  # Increased for better GPU utilization
            "T4": 256,
            "2xT4": 512,
            "SMALL": 128,
            "MINIMAL/CPU": 64,
        },
    },
    "XLM-R-large": {
        "model_name": "xlm-roberta-large",
        "hidden_size": 1024,
        "recommended_batch": {
            "L4/A100": 256,
            "T4": 64,
            "2xT4": 128,
            "SMALL": 32,
            "MINIMAL/CPU": 16,
        },
    },
}

BACKBONE_CONFIG = BACKBONE_CONFIGS[BACKBONE]
MODEL_NAME = BACKBONE_CONFIG["model_name"]
BACKBONE_HIDDEN = BACKBONE_CONFIG["hidden_size"]


# @markdown ---
# @markdown ## Run Setup

import os
import sys
import time

EXPERIMENT_START = time.time()

print("=" * 60)
print("BIP v10.9 - ENVIRONMENT DETECTION")
print("=" * 60)

# ===== ENVIRONMENT DETECTION =====
# Detect which cloud platform we're running on

ENV_NAME = "UNKNOWN"
ENV_GPU_QUOTA = "Unknown"
PERSISTENT_STORAGE = None
DATA_DIR = "/content"  # Default


def detect_environment():
    """Detect cloud environment and return (name, gpu_quota, storage_path, data_dir)"""

    # 1. Google Colab
    try:
        import google.colab

        return ("COLAB", "Free: T4 ~12h/day, Pro: L4/A100", "/content/drive/MyDrive", "/content")
    except ImportError:
        pass

    # 2. Kaggle Kernels
    if os.path.exists("/kaggle"):
        # Kaggle has /kaggle/input for datasets, /kaggle/working for output
        return ("KAGGLE", "Free: 2xT4 30h/week, TPU 30h/week", "/kaggle/working", "/kaggle/working")

    # 3. Lightning.ai Studios
    if os.environ.get("LIGHTNING_CLOUDSPACE_HOST") or os.path.exists("/teamspace"):
        # Lightning.ai has /teamspace/studios for persistent storage
        return (
            "LIGHTNING_AI",
            "Free: 22h/month GPU, Pro: A10G/H100",
            "/teamspace/studios",
            "/teamspace/studios",
        )

    # 4. Paperspace Gradient
    if os.environ.get("PAPERSPACE_NOTEBOOK_REPO_ID") or os.path.exists("/notebooks"):
        return ("PAPERSPACE", "Free: M4000 6h, Pro: A100/H100", "/storage", "/notebooks")

    # 5. Saturn Cloud
    if os.environ.get("SATURN_RESOURCE_ID") or "saturn" in os.environ.get("HOSTNAME", "").lower():
        return (
            "SATURN_CLOUD",
            "Free: T4 10h/month, Pro: A10G/A100",
            "/home/jovyan/workspace",
            "/home/jovyan",
        )

    # 6. HuggingFace Spaces
    if os.environ.get("SPACE_ID") or os.environ.get("HF_SPACE_ID"):
        return (
            "HUGGINGFACE_SPACES",
            "Free: CPU only, ZeroGPU: A10G/A100 quota",
            "/data",
            "/home/user/app",
        )

    # 7. AWS SageMaker Studio Lab
    if os.path.exists("/home/studio-lab-user"):
        return (
            "SAGEMAKER_STUDIO_LAB",
            "Free: T4 4h/session, 24h max/day",
            "/home/studio-lab-user",
            "/home/studio-lab-user",
        )

    # 8. Deepnote
    if os.environ.get("DEEPNOTE_PROJECT_ID"):
        return ("DEEPNOTE", "Free: CPU, Pro: T4/A10G", "/work", "/work")

    # 9. Local/Unknown
    return ("LOCAL", "Depends on local hardware", os.getcwd(), os.getcwd())


ENV_NAME, ENV_GPU_QUOTA, PERSISTENT_STORAGE, DATA_DIR = detect_environment()

print(f"\nEnvironment: {ENV_NAME}")
print(f"GPU Quota:   {ENV_GPU_QUOTA}")
print(f"Storage:     {PERSISTENT_STORAGE}")
print(f"Data Dir:    {DATA_DIR}")

# Environment-specific setup
ENV_TIPS = {
    "COLAB": [
        "Tip: Use GPU runtime (Runtime -> Change runtime type -> T4 GPU)",
        "Tip: Colab Pro gives L4 GPU access (~2x faster than T4)",
    ],
    "KAGGLE": [
        "Tip: Enable GPU (Settings -> Accelerator -> GPU T4 x2)",
        "Tip: 30h/week GPU quota resets every Saturday",
        "Tip: Upload data as a Kaggle Dataset for persistence",
    ],
    "LIGHTNING_AI": [
        "Tip: Select GPU studio (A10G recommended for this workload)",
        "Tip: /teamspace/studios persists across sessions",
    ],
    "PAPERSPACE": [
        "Tip: Use /storage for persistent data across runs",
        "Tip: Free tier has 6h/month GPU limit",
    ],
    "SATURN_CLOUD": [
        "Tip: Start a T4 instance from the Resources tab",
        "Tip: 10h/month free GPU quota",
    ],
    "HUGGINGFACE_SPACES": [
        "Tip: ZeroGPU provides A10G/A100 access with quota system",
        "Tip: Use Gradio/Streamlit for interactive demos",
    ],
    "SAGEMAKER_STUDIO_LAB": [
        "Tip: Request GPU runtime from the launcher",
        "Tip: Sessions timeout after 4h, max 24h/day",
    ],
    "LOCAL": ["Tip: Running locally - ensure CUDA is installed for GPU support"],
}

print("\n" + "-" * 60)
print("ENVIRONMENT TIPS:")
for tip in ENV_TIPS.get(ENV_NAME, ["No specific tips for this environment"]):
    print(f"  {tip}")
print("-" * 60)

# ===== INSTALL DEPENDENCIES =====
import subprocess

print("\nInstalling dependencies...")
for pkg in [
    "transformers",
    "sentence-transformers",
    "pandas",
    "tqdm",
    "scikit-learn",
    "pyyaml",
    "psutil",
    "datasets",
]:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

import psutil
import torch

print("\n" + "=" * 60)
print("GPU DETECTION & RESOURCE ALLOCATION")
print("=" * 60)

# Detect hardware
if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    VRAM_GB = torch.cuda.get_device_properties(0).total_memory / 1e9
    GPU_COUNT = torch.cuda.device_count()
else:
    GPU_NAME = "CPU"
    VRAM_GB = 0
    GPU_COUNT = 0

RAM_GB = psutil.virtual_memory().total / 1e9

print("\nDetected Hardware:")
print(f"  GPU:  {GPU_NAME}" + (f" (x{GPU_COUNT})" if GPU_COUNT > 1 else ""))
print(
    f"  VRAM: {VRAM_GB:.1f} GB"
    + (f" (total: {VRAM_GB * GPU_COUNT:.1f} GB)" if GPU_COUNT > 1 else "")
)
print(f"  RAM:  {RAM_GB:.1f} GB")

# Set optimal parameters based on hardware
if VRAM_GB >= 22:  # L4 (24GB) or A100
    GPU_TIER = "L4/A100"
elif VRAM_GB >= 14:  # T4 (16GB)
    GPU_TIER = "T4"
elif VRAM_GB >= 10:
    GPU_TIER = "SMALL"
else:
    GPU_TIER = "MINIMAL/CPU"

# Kaggle with 2xT4 can use larger batch
if ENV_NAME == "KAGGLE" and GPU_COUNT >= 2:
    GPU_TIER = "2xT4"
    print("  ** Kaggle 2xT4 detected **")

# Get backbone-specific batch size
BATCH_SIZE = BACKBONE_CONFIG["recommended_batch"].get(GPU_TIER, 64)
# Eval can use larger batch (no gradients)
EVAL_BATCH_SIZE = min(BATCH_SIZE * 4, 512) if VRAM_GB >= 20 else BATCH_SIZE * 2
print(f"  Backbone: {BACKBONE} -> batch size {BATCH_SIZE}")

MAX_PER_LANG = 50000  # Language sample limit
CPU_CORES = os.cpu_count() or 2
NUM_WORKERS = min(4, CPU_CORES - 1) if RAM_GB >= 24 and VRAM_GB >= 14 else 0
MAX_TEST_SAMPLES = 20000
# Use LEARNING_RATE from UI, or scale with batch size
if LEARNING_RATE and LEARNING_RATE != 1e-5:  # 1e-5 is the default
    LR = LEARNING_RATE
else:
    LR = 2e-5 * (BATCH_SIZE / 256)  # Linear scaling with batch size

print("\n" + "-" * 60)
print("OPTIMAL SETTINGS:")
print("-" * 60)
print(f"  Environment:     {ENV_NAME}")
print(f"  GPU Tier:        {GPU_TIER}")
print(f"  Backbone:        {BACKBONE}")
print(f"  Batch size:      {BATCH_SIZE}")
print(f"  Eval batch:     {EVAL_BATCH_SIZE}")
print(f"  Max per lang:    {MAX_PER_LANG:,}")
print(f"  DataLoader workers: {NUM_WORKERS}")
print(f"  Learning rate:   {LR:.2e}")

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# USE_AMP is set by form field above
USE_AMP = USE_AMP and torch.cuda.is_available()  # Only enable if GPU available
scaler = torch.amp.GradScaler("cuda") if USE_AMP else None

# ===== PERSISTENT STORAGE SETUP =====
print("\n" + "=" * 60)
print("PERSISTENT STORAGE SETUP")
print("=" * 60)

SAVE_DIR = None
DRIVE_HAS_DATA = False
DRIVE_FILES = set()  # Use set for O(1) lookup

if ENV_NAME == "COLAB":
    # Google Colab - mount Drive
    try:
        from google.colab import drive

        DRIVE_MOUNT_PATH = "/content/drive"

        if os.path.exists(f"{DRIVE_MOUNT_PATH}/MyDrive"):
            print("Google Drive already mounted")
        else:
            try:
                drive.mount(DRIVE_MOUNT_PATH, force_remount=False)
                print("Google Drive mounted successfully")
            except Exception as e:
                print(f"Drive mount issue: {e}")
                try:
                    drive.mount(DRIVE_MOUNT_PATH, force_remount=True)
                    print("Google Drive mounted (force remount)")
                except Exception as e2:
                    print(f"WARNING: Could not mount Drive: {e2}")
                    print("Falling back to local storage")
                    PERSISTENT_STORAGE = DATA_DIR

        SAVE_DIR = f"{DRIVE_MOUNT_PATH}/MyDrive/{DRIVE_FOLDER}"
    except Exception as e:
        print(f"Colab Drive setup failed: {e}")
        SAVE_DIR = f"{DATA_DIR}/{DRIVE_FOLDER}"

elif ENV_NAME == "KAGGLE":
    # Kaggle - use working directory
    SAVE_DIR = f"{PERSISTENT_STORAGE}/{DRIVE_FOLDER}"
    print(f"Using Kaggle working directory: {SAVE_DIR}")
    print("Note: Data persists until kernel is reset")
    # Check for uploaded datasets
    if os.path.exists("/kaggle/input"):
        datasets = os.listdir("/kaggle/input")
        if datasets:
            print(f"Available datasets: {datasets[:5]}")

elif ENV_NAME == "LIGHTNING_AI":
    SAVE_DIR = f"{PERSISTENT_STORAGE}/{DRIVE_FOLDER}"
    print(f"Using Lightning.ai studio storage: {SAVE_DIR}")

elif ENV_NAME == "PAPERSPACE":
    SAVE_DIR = f"{PERSISTENT_STORAGE}/{DRIVE_FOLDER}"
    print(f"Using Paperspace /storage: {SAVE_DIR}")

elif ENV_NAME == "HUGGINGFACE_SPACES":
    # HF Spaces has limited persistent storage
    SAVE_DIR = f"{PERSISTENT_STORAGE}/{DRIVE_FOLDER}"
    print(f"Using HuggingFace Spaces storage: {SAVE_DIR}")
    print("Warning: HF Spaces storage is limited")

else:
    SAVE_DIR = f"{PERSISTENT_STORAGE}/{DRIVE_FOLDER}"
    print(f"Using local storage: {SAVE_DIR}")

# Check if folder exists BEFORE creating it
folder_existed = os.path.exists(SAVE_DIR)
os.makedirs(SAVE_DIR, exist_ok=True)

# Check what's available in storage - use BOTH listdir AND direct exists checks
# (Google Drive can have sync issues where listdir misses files)
if os.path.exists(SAVE_DIR):
    DRIVE_FILES = set(os.listdir(SAVE_DIR))  # O(1) membership test

    # Direct existence checks for key files (bypasses listdir caching issues)
    key_files = ["passages.jsonl", "bonds.jsonl", "dear_abby.csv", "all_splits.json"]
    for kf in key_files:
        kf_path = os.path.join(SAVE_DIR, kf)
        if os.path.exists(kf_path) and kf not in DRIVE_FILES:
            print(f"  [Drive sync fix] Found {kf} via os.path.exists() but not listdir()")
            DRIVE_FILES.add(kf)

    DRIVE_HAS_DATA = "passages.jsonl" in DRIVE_FILES and "bonds.jsonl" in DRIVE_FILES

print("\n" + "-" * 60)
print("STORAGE STATUS:")
print("-" * 60)
print(f"  Folder: {SAVE_DIR}")
print(f"  Folder existed: {folder_existed}")
print(f"  Files found: {len(DRIVE_FILES)}")

# If folder was empty/new, show what folders exist in parent to help debug
if not DRIVE_FILES and ENV_NAME == "COLAB":
    parent = os.path.dirname(SAVE_DIR)  # e.g., /content/drive/MyDrive
    if os.path.exists(parent):
        siblings = [d for d in os.listdir(parent) if "bip" in d.lower() or "BIP" in d]
        if siblings:
            print(f"  ** Similar folders in {parent}: {siblings}")
        else:
            print(f"  ** No BIP folders found in {parent}")
if DRIVE_FILES:
    for f in sorted(DRIVE_FILES)[:10]:  # sorted() converts to list for slicing
        print(f"    - {f}")
    if len(DRIVE_FILES) > 10:
        print(f"    ... and {len(DRIVE_FILES) - 10} more")
print(f"  Pre-processed data available: {DRIVE_HAS_DATA}")

# Decide data loading strategy
LOAD_FROM_DRIVE = USE_DRIVE_DATA and DRIVE_HAS_DATA and not REFRESH_DATA_FROM_SOURCE

print("\n" + "=" * 60)
print(f"DATA LOADING STRATEGY: {DATA_MODE}")
print("-" * 60)
if DATA_MODE == "Refresh all":
    print("  -> Will re-download ALL data from online sources")
    print("     (This takes ~2 hours, use 'Update missing' to save time)")
elif DATA_MODE == "Cache only":
    if LOAD_FROM_DRIVE:
        print("  -> Using cached data only (no downloads)")
    else:
        print("  -> ERROR: Cache-only mode but no cached data found!")
        print("     Change DATA_MODE to 'Update missing'")
else:  # Update missing (default)
    if LOAD_FROM_DRIVE:
        print("  -> Using cached processed data from Drive")
        print("     (v10.9 corpora will be added if missing)")
    else:
        print("  -> Will download missing data, use cached where available")
        print(
            f"     Sefaria: {'cached' if os.path.exists(f'{SAVE_DIR}/Sefaria-Export-json.tar.gz') else 'will download'}"
        )
print("=" * 60)

# Create local directories
for d in ["data/processed", "data/splits", "data/raw", "models/checkpoints", "results"]:
    os.makedirs(d, exist_ok=True)

print("\n" + "=" * 60)
print("SETUP COMPLETE")
print("=" * 60)
print(f"  Environment: {ENV_NAME}")
print(f"  GPU:         {GPU_NAME} ({GPU_TIER})")
print(f"  Storage:     {SAVE_DIR}")
print("  Ready to run: Cell 2 (Imports)")

BIP v10.9 - ENVIRONMENT DETECTION

Environment: COLAB
GPU Quota:   Free: T4 ~12h/day, Pro: L4/A100
Storage:     /content/drive/MyDrive
Data Dir:    /content

------------------------------------------------------------
ENVIRONMENT TIPS:
  Tip: Use GPU runtime (Runtime -> Change runtime type -> T4 GPU)
  Tip: Colab Pro gives L4 GPU access (~2x faster than T4)
------------------------------------------------------------

Installing dependencies...

GPU DETECTION & RESOURCE ALLOCATION

Detected Hardware:
  GPU:  NVIDIA L4
  VRAM: 23.8 GB
  RAM:  56.9 GB
  Backbone: LaBSE -> batch size 4096

------------------------------------------------------------
OPTIMAL SETTINGS:
------------------------------------------------------------
  Environment:     COLAB
  GPU Tier:        L4/A100
  Backbone:        LaBSE
  Batch size:      4096
  Eval batch:     512
  Max per lang:    50,000
  DataLoader workers: 4
  Learning rate:   3.20e-04

PERSISTENT STORAGE SETUP
Mounted at /content/drive
Google Drive

In [ ]:
# @title 2. Load Corpora (v10.12 - Self-Contained) { display-mode: "form" }
# @markdown Downloads from verified external sources - fully self-contained, no external imports
# @markdown
# @markdown **Sources (9 categories):**
# @markdown - Sanskrit: Itihasa (93K shlokas)
# @markdown - Pali: SuttaCentral API (Full Canon)
# @markdown - Arabic: Tanzil.net (Quran)
# @markdown - Hebrew/Aramaic: Sefaria GitHub

INCLUDE_RESPONSA = False  # @param {type:"boolean"}
# @markdown - **INCLUDE_RESPONSA**: Include Responsa texts (requires 30-50 min git clone)
# @markdown - Set to True only if you need the full Responsa collection
# @markdown - Chinese: ctext.org API
# @markdown - Greek/Latin: Perseus Digital Library
# @markdown - Romance: Don Quijote, Montaigne, Voltaire, Rousseau, Machiavelli, Dante
# @markdown - Folklore: Ashliman Folktexts (incl. Native American)
# @markdown - English: Gutenberg philosophy, Dear Abby (68K), hendrycks/ethics (134K)

import csv
import json
import os
import subprocess
import threading
import time
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

import requests

# Global session for all HTTP requests (shared across loaders)
ctext_session = requests.Session()
ctext_session.headers.update({
    "User-Agent": "Mozilla/5.0 (compatible; BIP-Corpus-Loader/1.0; +https://github.com/)",
    "Accept": "application/json, text/plain, */*",
})

print("=" * 60)
print("LOADING CORPORA (v10.12 - Self-Contained)")
print("=" * 60)

# ============================================================================
# CONFIGURATION
# ============================================================================

DATA_DIR = Path("data/raw/v10.12")
CACHE_DIR = DATA_DIR / "cache"
DATA_DIR.mkdir(parents=True, exist_ok=True)
CACHE_DIR.mkdir(parents=True, exist_ok=True)

# v10.15.1: Immediate Drive caching helper
def cache_to_drive(cache_file: Path, quiet: bool = False) -> None:
    """Immediately copy cache file to Drive for persistence."""
    try:
        if '_save_dir' in globals() and _save_dir and os.path.exists(_save_dir):
            drive_cache = Path(_save_dir) / 'corpus_cache'
            drive_cache.mkdir(exist_ok=True)
            dest = drive_cache / cache_file.name
            if not dest.exists():
                import shutil
                shutil.copy(cache_file, dest)
                if not quiet:
                    print(f"      -> Cached {cache_file.name} to Drive")
    except Exception:
        pass  # Drive cache is optional

# Get settings from Cell 1
try:
    _cache_only = CACHE_ONLY
except NameError:
    _cache_only = False

try:
    _save_dir = SAVE_DIR
except NameError:
    _save_dir = "data/processed"

# Memory limits per language (L4 GPU safe)
MAX_PASSAGES_PER_LANG = {
    "sanskrit": 15000,
    "pali": 10000,
    "arabic": 10000,
    "classical_chinese": 10000,
    "hebrew": 15000,
    "aramaic": 10000,
    "english": 50000,  # Increased for folklore + ethics
    "greek": 10000,
    "latin": 10000,
    "spanish": 5000,
    "french": 5000,
    "italian": 5000,
    "default": 5000,
}

MIN_PASSAGES = 500  # For 6-sigma confidence

# ============================================================================
# RESTORE CACHE FROM DRIVE (if available)
# ============================================================================
# In hybrid mode, check if Drive has cached corpus files and restore them
# This avoids re-downloading on every Colab restart

if _save_dir and os.path.exists(_save_dir):
    drive_cache = Path(_save_dir) / "corpus_cache"
    if drive_cache.exists():
        import shutil

        restored = 0
        for cache_file in drive_cache.glob("*.json"):
            local_cache = CACHE_DIR / cache_file.name
            if not local_cache.exists():
                shutil.copy(cache_file, local_cache)
                restored += 1
        if restored:
            print(f"Restored {restored} cache files from Drive")
    # Note: sefaria.json IS cached to Drive after first successful load
    # Git clone is faster than Drive copy for many small files

# ============================================================================
# RATE LIMITING
# ============================================================================


class RateLimiter:
    def __init__(self, calls_per_minute: int = 20):
        self.min_interval = 60.0 / calls_per_minute
        self.last_call = 0.0
        self.lock = threading.Lock()

    def wait(self):
        with self.lock:
            elapsed = time.time() - self.last_call
            if elapsed < self.min_interval:
                time.sleep(self.min_interval - elapsed)
            self.last_call = time.time()


GITHUB_LIMITER = RateLimiter(calls_per_minute=60)
SUTTACENTRAL_LIMITER = RateLimiter(calls_per_minute=120)
CTEXT_LIMITER = RateLimiter(calls_per_minute=30)

# ============================================================================
# SANSKRIT - Itihasa from GitHub (VERIFIED)
# https://github.com/rahular/itihasa - 93K shlokas
# ============================================================================


def load_itihasa_github() -> list[dict]:
    """Load Itihasa Sanskrit shlokas from GitHub."""
    passages = []
    cache_file = CACHE_DIR / "itihasa.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            if passages and "time_periods" in passages[0]:
                print(f"  Itihasa: {len(passages):,} passages (cached)")
                return passages
            print("  Itihasa cache missing time_periods - rebuilding...")
            passages = []

    print("  Downloading Itihasa from GitHub...")
    data_path = DATA_DIR / "itihasa"
    data_path.mkdir(parents=True, exist_ok=True)

    files = [
        ("train.sn", "https://raw.githubusercontent.com/rahular/itihasa/main/data/train.sn"),
        ("dev.sn", "https://raw.githubusercontent.com/rahular/itihasa/main/data/dev.sn"),
        ("test.sn", "https://raw.githubusercontent.com/rahular/itihasa/main/data/test.sn"),
    ]

    for name, url in files:
        local_file = data_path / name
        if not local_file.exists():
            try:
                GITHUB_LIMITER.wait()
                resp = ctext_session.get(url, timeout=120)
                if resp.status_code == 200:
                    with open(local_file, "w", encoding="utf-8") as f:
                        f.write(resp.text)
                    print(f"    Downloaded {name}: {len(resp.text) // 1024}KB")
            except Exception as e:
                print(f"    Failed {name}: {e}")

    # Parse .sn files
    for sn_file in data_path.glob("*.sn"):
        with open(sn_file, encoding="utf-8") as f:
            for i, line in enumerate(f, 1):
                text = line.strip()
                if text and len(text) > 10:
                    passages.append(
                        {
                            "id": f"itihasa_{sn_file.stem}_{i}",
                            "text": text,
                            "language": "sanskrit",
                            "source": f"Itihasa/{sn_file.stem}",
                            "time_periods": ["DHARMA", "ANCIENT", "INDIC"],
                        }
                    )

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

    print(f"  Itihasa: {len(passages):,} passages")
    return passages


# ============================================================================
# PALI - SuttaCentral API (VERIFIED)
# https://suttacentral.net/api/bilarasuttas/{id}/pli
# ============================================================================


def load_pali_suttacentral() -> list[dict]:
    """Load Pali texts from SuttaCentral API."""
    passages = []
    cache_file = CACHE_DIR / "suttacentral.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            if passages and "time_periods" in passages[0]:
                print(f"  SuttaCentral: {len(passages):,} passages (cached)")
                return passages
            print("  SuttaCentral cache missing time_periods - rebuilding...")
            passages = []

    print("  Fetching from SuttaCentral API...")

    # Expanded sutta list
    sutta_ids = []
    # Majjhima Nikaya (152 suttas)
    sutta_ids.extend([f"mn{i}" for i in range(1, 153)])
    # Digha Nikaya (34 suttas)
    sutta_ids.extend([f"dn{i}" for i in range(1, 35)])
    # Samyutta Nikaya (key vaggas)
    for v in [1, 3, 6, 12, 22, 35, 45, 56]:
        sutta_ids.extend([f"sn{v}.{i}" for i in range(1, 20)])
    # Anguttara Nikaya
    for n in [1, 2, 3, 4, 5, 6, 7, 8, 10]:
        sutta_ids.extend([f"an{n}.{i}" for i in range(1, 50)])
    # Dhammapada
    sutta_ids.extend([f"dhp{i}" for i in range(1, 27)])

    def fetch_sutta(sid):
        results = []
        try:
            SUTTACENTRAL_LIMITER.wait()
            url = f"https://suttacentral.net/api/bilarasuttas/{sid}/pli"
            resp = ctext_session.get(url, timeout=30)
            if resp.status_code == 200:
                data = resp.json()
                if isinstance(data, dict):
                    segments = data.get("root_text", {})
                    if isinstance(segments, dict):
                        for seg_id, text in segments.items():
                            if text and len(text) > 20:
                                results.append(
                                    {
                                        "id": f"pali_{sid}_{seg_id}",
                                        "text": text.strip(),
                                        "language": "pali",
                                        "source": sid,
                                        "time_periods": ["PALI", "ANCIENT", "INDIC", "BUDDHIST"],
                                    }
                                )
        except Exception:
            pass
        return results

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(fetch_sutta, sid) for sid in sutta_ids[:300]]
        for done, future in enumerate(as_completed(futures), 1):
            passages.extend(future.result())
            if done % 50 == 0:
                print(f"    Fetched {done}/{min(300, len(sutta_ids))} suttas...")

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

    print(f"  SuttaCentral: {len(passages):,} passages")
    return passages


# ============================================================================
# ARABIC - Tanzil.net (VERIFIED)
# https://tanzil.net/download/
# ============================================================================


def load_quran_tanzil() -> list[dict]:
    """Load Quran from Tanzil.net."""
    passages = []
    cache_file = CACHE_DIR / "tanzil.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            if passages and "time_periods" in passages[0]:
                print(f"  Tanzil Quran: {len(passages):,} passages (cached)")
                return passages
            print("  Tanzil cache missing time_periods - rebuilding...")
            passages = []

    print("  Downloading Quran from Tanzil.net...")
    try:
        url = "https://tanzil.net/pub/download/index.php?quranType=uthmani&outType=txt-2&agree=true"
        resp = ctext_session.get(url, timeout=60)
        if resp.status_code == 200:
            for line in resp.text.strip().split("\n"):
                if "|" in line:
                    parts = line.split("|")
                    if len(parts) >= 3:
                        surah, ayah, text = parts[0], parts[1], parts[2].strip()
                        if len(text) > 10:
                            passages.append(
                                {
                                    "id": f"quran_{surah}_{ayah}",
                                    "text": text,
                                    "language": "arabic",
                                    "source": f"Quran {surah}:{ayah}",
                                    "time_periods": ["QURANIC", "MEDIEVAL", "SEMITIC"],
                                }
                            )
            print(f"    Downloaded {len(passages)} verses")
    except Exception as e:
        print(f"    Failed: {e}")

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

    print(f"  Tanzil Quran: {len(passages):,} passages")
    return passages


# ============================================================================
# HEBREW/ARAMAIC - Sefaria GitHub (VERIFIED)
# https://github.com/Sefaria/Sefaria-Export
# ============================================================================


def load_sefaria_github() -> list[dict]:
    """Load Hebrew/Aramaic from Sefaria GitHub."""
    passages = []
    cache_file = CACHE_DIR / "sefaria.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            # v10.16.1: Validate cache has required fields
            if passages and "time_periods" in passages[0]:
                print(f"  Sefaria: {len(passages):,} passages (cached)")
                return passages
            else:
                print(f"  Sefaria cache missing time_periods - rebuilding...")
                passages = []  # Invalidate cache

    base_path = DATA_DIR / "Sefaria-Export"
    json_path = base_path / "json"

    # Key texts to download (path, language, period)
    # Pattern: path -> Hebrew/merged.json or Hebrew/Merged.json
    key_texts = [
        # =====================================================================
        # TANAKH - Complete Hebrew Bible (~39 books, ~20MB)
        # =====================================================================
        # Torah (Pentateuch) - 5 books
        ("Tanakh/Torah/Genesis", "hebrew", "BIBLICAL"),
        ("Tanakh/Torah/Exodus", "hebrew", "BIBLICAL"),
        ("Tanakh/Torah/Leviticus", "hebrew", "BIBLICAL"),
        ("Tanakh/Torah/Numbers", "hebrew", "BIBLICAL"),
        ("Tanakh/Torah/Deuteronomy", "hebrew", "BIBLICAL"),
        # Former Prophets - 6 books
        ("Tanakh/Prophets/Joshua", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Judges", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/I Samuel", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/II Samuel", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/I Kings", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/II Kings", "hebrew", "BIBLICAL"),
        # Latter Prophets - Major - 3 books
        ("Tanakh/Prophets/Isaiah", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Jeremiah", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Ezekiel", "hebrew", "BIBLICAL"),
        # Latter Prophets - Minor (Trei Asar) - 12 books
        ("Tanakh/Prophets/Hosea", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Joel", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Amos", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Obadiah", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Jonah", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Micah", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Nahum", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Habakkuk", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Zephaniah", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Haggai", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Zechariah", "hebrew", "BIBLICAL"),
        ("Tanakh/Prophets/Malachi", "hebrew", "BIBLICAL"),
        # Writings (Ketuvim) - 13 books
        ("Tanakh/Writings/Psalms", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Proverbs", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Job", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Song of Songs", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Ruth", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Lamentations", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Ecclesiastes", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Esther", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Daniel", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Ezra", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/Nehemiah", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/I Chronicles", "hebrew", "BIBLICAL"),
        ("Tanakh/Writings/II Chronicles", "hebrew", "BIBLICAL"),
        # =====================================================================
        # MISHNAH - Complete 6 Orders (~63 tractates, ~15MB)
        # =====================================================================
        # Seder Zeraim (Seeds) - Agricultural ethics
        ("Mishnah/Seder Zeraim/Mishnah Berakhot", "hebrew", "TANNAITIC"),
        ("Mishnah/Seder Zeraim/Mishnah Peah", "hebrew", "TANNAITIC"),  # Corners for poor
        ("Mishnah/Seder Zeraim/Mishnah Maasrot", "hebrew", "TANNAITIC"),  # Tithes
        # Seder Moed (Festivals) - Sabbath ethics
        ("Mishnah/Seder Moed/Mishnah Shabbat", "hebrew", "TANNAITIC"),
        ("Mishnah/Seder Moed/Mishnah Yoma", "hebrew", "TANNAITIC"),  # Day of Atonement
        ("Mishnah/Seder Moed/Mishnah Taanit", "hebrew", "TANNAITIC"),  # Fasts
        # Seder Nashim (Women) - Family/gender ethics
        ("Mishnah/Seder Nashim/Mishnah Yevamot", "hebrew", "TANNAITIC"),  # Levirate marriage
        ("Mishnah/Seder Nashim/Mishnah Ketubot", "hebrew", "TANNAITIC"),  # Marriage contracts
        ("Mishnah/Seder Nashim/Mishnah Nedarim", "hebrew", "TANNAITIC"),  # Vows
        ("Mishnah/Seder Nashim/Mishnah Nazir", "hebrew", "TANNAITIC"),  # Nazirite vows
        ("Mishnah/Seder Nashim/Mishnah Sotah", "hebrew", "TANNAITIC"),  # Suspected adulteress
        ("Mishnah/Seder Nashim/Mishnah Gittin", "hebrew", "TANNAITIC"),  # Divorce
        ("Mishnah/Seder Nashim/Mishnah Kiddushin", "hebrew", "TANNAITIC"),  # Betrothal
        # Seder Nezikin (Damages) - Civil/criminal ethics (CORE)
        # Note: Sefaria uses "Mishnah X" prefix for tractate folders
        ("Mishnah/Seder Nezikin/Mishnah Bava Kamma", "hebrew", "TANNAITIC"),  # First Gate - damages
        (
            "Mishnah/Seder Nezikin/Mishnah Bava Metzia",
            "hebrew",
            "TANNAITIC",
        ),  # Middle Gate - property
        ("Mishnah/Seder Nezikin/Mishnah Bava Batra", "hebrew", "TANNAITIC"),  # Last Gate - sales
        ("Mishnah/Seder Nezikin/Mishnah Sanhedrin", "hebrew", "TANNAITIC"),  # Courts/capital
        ("Mishnah/Seder Nezikin/Mishnah Makkot", "hebrew", "TANNAITIC"),  # Lashes
        ("Mishnah/Seder Nezikin/Mishnah Shevuot", "hebrew", "TANNAITIC"),  # Oaths
        ("Mishnah/Seder Nezikin/Mishnah Eduyot", "hebrew", "TANNAITIC"),  # Testimonies
        ("Mishnah/Seder Nezikin/Mishnah Avodah Zarah", "hebrew", "TANNAITIC"),  # Idolatry
        (
            "Mishnah/Seder Nezikin/Pirkei Avot",
            "hebrew",
            "TANNAITIC",
        ),  # Ethics of Fathers (no prefix)
        ("Mishnah/Seder Nezikin/Mishnah Horayot", "hebrew", "TANNAITIC"),  # Rulings
        # Seder Kodashim (Holy Things) - Temple/sacred
        ("Mishnah/Seder Kodashim/Mishnah Zevachim", "hebrew", "TANNAITIC"),
        ("Mishnah/Seder Kodashim/Mishnah Menachot", "hebrew", "TANNAITIC"),
        # Seder Tohorot (Purities) - Purity ethics
        ("Mishnah/Seder Tohorot/Mishnah Niddah", "hebrew", "TANNAITIC"),  # Menstrual purity
        # =====================================================================
        # TALMUD BAVLI - Key tractates (~20MB)
        # =====================================================================
        # Foundational
        ("Talmud/Bavli/Seder Zeraim/Berakhot", "aramaic", "AMORAIC"),
        # Ethics tractates (Seder Nezikin)
        ("Talmud/Bavli/Seder Nezikin/Bava Kamma", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nezikin/Bava Metzia", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nezikin/Bava Batra", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nezikin/Sanhedrin", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nezikin/Makkot", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nezikin/Shevuot", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nezikin/Avodah Zarah", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nezikin/Horayot", "aramaic", "AMORAIC"),
        # Family ethics (Seder Nashim)
        ("Talmud/Bavli/Seder Nashim/Yevamot", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nashim/Ketubot", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nashim/Kiddushin", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nashim/Gittin", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Nashim/Sotah", "aramaic", "AMORAIC"),
        # Sabbath (Seder Moed)
        ("Talmud/Bavli/Seder Moed/Shabbat", "aramaic", "AMORAIC"),
        ("Talmud/Bavli/Seder Moed/Yoma", "aramaic", "AMORAIC"),
        # =====================================================================
        # RESPONSA - Ethical Q&A (only if INCLUDE_RESPONSA=True)
        # =====================================================================
        ("Responsa/Geonim", "hebrew", "GEONIC"),  # 600-1000 CE
        ("Responsa/Rishonim", "hebrew", "RISHONIM"),  # 1000-1500 CE
        ("Responsa/Acharonim", "hebrew", "ACHARONIM"),  # 1500-1800 CE
        ("Responsa/Modern", "hebrew", "MODERN_RESPONSA"),  # 1800-present
        ("Responsa/Teshuvot Maharsham Volume I", "hebrew", "ACHARONIM"),
        ("Responsa/Teshuvot Maharsham Volume II", "hebrew", "ACHARONIM"),
        ("Responsa/Teshuvot Maharsham Volume III", "hebrew", "ACHARONIM"),
    ]

    # Download strategy depends on INCLUDE_RESPONSA setting
    # - False: Staged download only (fast, ~2 min, core texts)
    # - True: Full clone only (slow, 30-50 min, includes Responsa)

    if INCLUDE_RESPONSA:
        # Skip staged download - we need full clone for Responsa anyway
        print("  INCLUDE_RESPONSA=True: Will do full clone for Responsa...")
        need_staged = False
    else:
        need_staged = not json_path.exists()

    if need_staged:
        print("  Downloading Sefaria texts (staged download)...")
        json_path.mkdir(parents=True, exist_ok=True)

        base_url = "https://raw.githubusercontent.com/Sefaria/Sefaria-Export/master/json"

        def download_sefaria_text(text_info):
            """Download merged.json for a Sefaria text, handling various structures."""
            text_path, lang, period = text_info
            url_path = text_path.replace(" ", "%20")

            # Different structures for different text types
            if "Responsa" in text_path:
                # Responsa have nested structure - try to get index or first collection
                # For now, skip in staged mode - these need full clone
                return text_path, False, 0, []

            # Standard texts: try Hebrew/merged.json first
            patterns = [
                ("Hebrew/merged.json", "Hebrew"),
                ("Aramaic/merged.json", "Aramaic"),  # For Talmud
                ("merged.json", ""),  # Direct merged.json
            ]

            for pattern, subdir in patterns:
                try:
                    url = f"{base_url}/{url_path}/{pattern}"
                    GITHUB_LIMITER.wait()
                    resp = ctext_session.get(url, timeout=60)
                    if resp.status_code == 200 and len(resp.text) > 100:
                        local_dir = json_path / text_path
                        if subdir:
                            local_dir = local_dir / subdir
                        local_dir.mkdir(parents=True, exist_ok=True)
                        local_file = local_dir / "merged.json"
                        with open(local_file, "w", encoding="utf-8") as f:
                            f.write(resp.text)
                        return text_path, True, len(resp.text), []
                except Exception:
                    continue

            return text_path, False, 0, []

        # Download in parallel
        downloaded = 0
        total_size = 0
        responsa_skipped = []
        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(download_sefaria_text, t) for t in key_texts]
            for future in as_completed(futures):
                text_path, success, size, _ = future.result()
                if "Responsa" in text_path:
                    responsa_skipped.append(text_path)
                elif success:
                    downloaded += 1
                    total_size += size
                    print(
                        f"    Downloaded {downloaded}: {text_path.split('/')[-1][:30]} ({size // 1024}KB)"
                    )

        non_responsa_texts = [t for t in key_texts if "Responsa" not in t[0]]
        responsa_texts = [t for t in key_texts if "Responsa" in t[0]]
        print(
            f"    Staged: {downloaded}/{len(non_responsa_texts)} core texts, {total_size // 1024}KB"
        )
        if not INCLUDE_RESPONSA:
            # Show what we got from staged download
            if responsa_texts:
                print("    Responsa skipped (set INCLUDE_RESPONSA=True to include)")
            print(f"    Using staged download results ({downloaded} texts)")
            need_clone = False  # Don't clone if INCLUDE_RESPONSA is False
        else:
            # INCLUDE_RESPONSA is True - we need to clone
            print(f"    Responsa ({len(responsa_texts)} collections) require full clone")
            need_clone = True

        if need_clone:
            print("    Starting full clone (this takes 30-50 min)...")
            # Clone to a temp location, then move
            import shutil

            clone_path = DATA_DIR / "Sefaria-Clone-Temp"
            if clone_path.exists():
                shutil.rmtree(clone_path)

            print("  Cloning Sefaria-Export from GitHub (~2GB, 30-50 min)...")
            try:
                import re

                proc = subprocess.Popen(
                    [
                        "git",
                        "clone",
                        "--depth",
                        "1",
                        "--progress",
                        "https://github.com/Sefaria/Sefaria-Export.git",
                        str(clone_path),
                    ],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,  # Git progress goes to stderr
                    text=True,
                    bufsize=1,
                )
                last_progress_time = time.time()
                last_pct = -1
                current_phase = ""
                stall_timeout = 120  # Kill if no progress for 2 minutes

                while proc.poll() is None:
                    # Check for stall (no progress for stall_timeout seconds)
                    if time.time() - last_progress_time > stall_timeout:
                        proc.kill()
                        print(f"\n    Stalled (no progress for {stall_timeout}s)")
                        raise subprocess.TimeoutExpired("git clone", stall_timeout)

                    # Read stderr for progress (git writes progress there)
                    try:
                        line = proc.stderr.readline()
                        if line:
                            line = line.strip()
                            last_progress_time = time.time()  # Got output = progress

                            # Detect phase changes
                            if "Receiving objects" in line and current_phase != "receiving":
                                current_phase = "receiving"
                                print("    Receiving objects: ", end="", flush=True)
                                last_pct = -1
                            elif "Resolving deltas" in line and current_phase != "resolving":
                                current_phase = "resolving"
                                print("\n    Resolving deltas:  ", end="", flush=True)
                                last_pct = -1
                            elif "Updating files" in line and current_phase != "updating":
                                current_phase = "updating"
                                print("\n    Updating files:    ", end="", flush=True)
                                last_pct = -1

                            # Extract and print percentage (every 10%)
                            if "%" in line:
                                match = re.search(r"(\d+)%", line)
                                if match:
                                    pct = int(match.group(1))
                                    # Print at 0, 10, 20, ... 100
                                    if pct // 10 > last_pct // 10 or pct == 100:
                                        print(f"{pct}% ", end="", flush=True)
                                        last_pct = pct
                    except Exception:
                        pass
                    time.sleep(0.05)

                print()  # Newline after progress
                # Drain any remaining output
                _, stderr = proc.communicate(timeout=5)
                if proc.returncode == 0:
                    print("    Clone successful!")
                    # Move cloned json to base_path
                    cloned_json = clone_path / "json"
                    if cloned_json.exists():
                        import shutil

                        if json_path.exists():
                            shutil.rmtree(json_path)
                        shutil.move(str(cloned_json), str(json_path))
                        shutil.rmtree(clone_path)  # Clean up
                else:
                    print(f"    Clone failed (code {proc.returncode})")
                    if stderr:
                        print(f"    {stderr[:200]}")
            except subprocess.TimeoutExpired:
                print("    Using staged results (Responsa unavailable)")
            except Exception as e:
                print(f"\n    Clone failed: {e} - using staged results")
        else:
            print("    Staged download sufficient, skipping full clone")

    def extract_text(obj, depth=0):
        if depth > 5:
            return []
        texts = []
        if isinstance(obj, str) and len(obj) > 20:
            texts.append(obj)
        elif isinstance(obj, list):
            for item in obj:
                texts.extend(extract_text(item, depth + 1))
        elif isinstance(obj, dict):
            for key in ["he", "text", "content"]:
                if key in obj:
                    texts.extend(extract_text(obj[key], depth + 1))
        return texts

    for text_path, lang, period in key_texts:
        full_path = json_path / text_path
        if not full_path.exists():
            json_file = json_path / f"{text_path}.json"
            if json_file.exists():
                full_path = json_file
            else:
                continue

        try:
            files_to_parse = []
            if full_path.is_file():
                files_to_parse = [full_path]
            elif full_path.is_dir():
                # Responsa have many nested files - allow more
                max_files = 500 if "Responsa" in text_path else 100
                files_to_parse = list(full_path.rglob("*.json"))[:max_files]

            text_count = 0
            for jf in files_to_parse:
                with open(jf, encoding="utf-8") as f:
                    data = json.load(f)
                texts = extract_text(data)
                # More texts per file for Responsa (rich ethical Q&A)
                max_per_file = 500 if "Responsa" in text_path else 200
                for text in texts[:max_per_file]:
                    if len(passages) >= MAX_PASSAGES_PER_LANG.get(lang, 5000):
                        break
                    passages.append(
                        {
                            "id": f"sefaria_{len(passages)}",
                            "text": text.strip(),
                            "language": lang,
                            "source": text_path.split("/")[-1],
                            "time_periods": [period],
                        }
                    )
                    text_count += 1
            if text_count > 0 and "Responsa" in text_path:
                print(f"    {text_path.split('/')[-1]}: {text_count} responsa")
        except Exception:
            continue

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

        # Immediately cache to Drive (clone takes 30-50 min - don't lose this!)
        try:
            if "_save_dir" in dir() and _save_dir and os.path.exists(_save_dir):
                import shutil

                drive_cache = Path(_save_dir) / "corpus_cache"
                drive_cache.mkdir(exist_ok=True)
                drive_sefaria = drive_cache / "sefaria.json"
                if not drive_sefaria.exists():
                    shutil.copy(cache_file, drive_sefaria)
                    print("    -> Cached sefaria.json to Drive for future runs")
        except Exception:
            pass  # Drive cache is optional

    print(f"  Sefaria: {len(passages):,} passages")
    return passages


# ============================================================================
# CHINESE - ctext.org API (VERIFIED)
# https://api.ctext.org/gettext?urn=ctp:analects/xue-er
# ============================================================================


def load_chinese_ctext() -> list[dict]:
    """Load Chinese classics from ctext.org API."""
    passages = []
    cache_file = CACHE_DIR / "ctext.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            # v10.16.1: Validate cache has time_periods
            if passages and "time_periods" in passages[0]:
                # Show breakdown by period
                by_period = {}
                for p in passages:
                    period = p.get("time_periods", ["UNKNOWN"])[0]
                    by_period[period] = by_period.get(period, 0) + 1
                print(f"  ctext.org: {len(passages):,} passages (cached)")
                for period, count in sorted(by_period.items()):
                    print(f"    {period}: {count}")
                return passages
            print("  ctext cache missing time_periods - rebuilding...")
            passages = []

    # v10.15.1: Add headers to avoid API blocking
    ctext_headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "application/json",
        "Accept-Language": "en-US,en;q=0.9",
    }
    ctext_session = requests.Session()
    ctext_session.headers.update(ctext_headers)

    print("  Fetching from ctext.org API...")

    # v10.14.3: Diagnostic check
    try:
        _test_url = "https://api.ctext.org/gettext?urn=ctp:analects/xue-er"
        _test_resp = ctext_session.get(_test_url, timeout=10)
        print(f"    [DIAG] API test: {_test_resp.status_code}")
        if _test_resp.status_code == 200:
            _test_data = _test_resp.json()
            print(f"    [DIAG] Response keys: {list(_test_data.keys())}")
            if "fulltext" in _test_data:
                print(f"    [DIAG] fulltext count: {len(_test_data['fulltext'])}")
        else:
            print(f"    [DIAG] API returned: {_test_resp.text[:200]}")
    except Exception as e:
        print(f"    [DIAG] API test failed: {type(e).__name__}: {e}")

    # ctext.org requires chapter-level URNs
    texts = [
        # Working texts (no auth required):
        (
            "ctp:analects",
            "Analects",
            "CONFUCIAN",
            [
                "xue-er",
                "wei-zheng",
                "ba-yi",
                "li-ren",
                "gong-ye-chang",
                "yong-ye",
                "shu-er",
                "tai-bo",
                "zi-han",
                "xiang-dang",
                "xian-jin",
                "yan-yuan",
                "zi-lu",
                "xian-wen",
                "wei-ling-gong",
                "ji-shi",
                "yang-huo",
                "wei-zi",
                "zi-zhang",
                "yao-yue",
            ],
        ),
        (
            "ctp:mengzi",
            "Mencius",
            "CONFUCIAN",
            [
                "liang-hui-wang-i",
                "liang-hui-wang-ii",
                "gong-sun-chou-i",
                "gong-sun-chou-ii",
                "teng-wen-gong-i",
                "teng-wen-gong-ii",
                "li-lou-i",
                "li-lou-ii",
                "wan-zhang-i",
                "wan-zhang-ii",
                "gaozi-i",
                "gaozi-ii",
                "jin-xin-i",
                "jin-xin-ii",
            ],
        ),
        ("ctp:dao-de-jing", "Daodejing", "DAOIST", []),  # Book-level fetch
        # NOTE: Zhuangzi, Xunzi, Han Feizi, Mozi require special handling or auth
    ]

    errors_by_text = {}

    for text_id, name, period, chapters in texts:
        count = 0
        errors = []
        # If no chapters specified, fetch book-level
        if not chapters:
            try:
                CTEXT_LIMITER.wait()
                url = f"https://api.ctext.org/gettext?urn={text_id}"
                resp = ctext_session.get(url, timeout=30)
                if resp.status_code == 200:
                    data = resp.json()
                    if isinstance(data, dict) and "fulltext" in data:
                        for text in data["fulltext"]:
                            if text and len(text) > 10:
                                passages.append(
                                    {
                                        "id": f"ctext_{len(passages)}",
                                        "text": text,
                                        "language": "classical_chinese",
                                        "source": name,
                                        "time_periods": [period],
                                    }
                                )
                                count += 1
            except Exception as e:
                errors.append(f"book-level: {type(e).__name__}")

        for chapter in chapters:
            try:
                CTEXT_LIMITER.wait()
                urn = f"{text_id}/{chapter}"
                url = f"https://api.ctext.org/gettext?urn={urn}"
                resp = ctext_session.get(url, timeout=30)

                if resp.status_code != 200:
                    errors.append(f"{chapter}: HTTP {resp.status_code}")
                    continue

                data = resp.json()

                # Check for API error response
                if isinstance(data, dict) and "error" in data:
                    errors.append(f"{chapter}: {data['error']}")
                    continue

                # API returns {"fulltext": [...], "title": "..."}
                if isinstance(data, dict) and "fulltext" in data:
                    for text in data["fulltext"]:
                        if text and len(text) > 10:
                            passages.append(
                                {
                                    "id": f"ctext_{len(passages)}",
                                    "text": text,
                                    "language": "classical_chinese",
                                    "source": f"{name}/{chapter}",
                                    "time_periods": [period],
                                }
                            )
                            count += 1
                # Fallback: list format
                elif isinstance(data, list):
                    for item in data:
                        text = item.get("text", "") if isinstance(item, dict) else str(item)
                        if text and len(text) > 10:
                            passages.append(
                                {
                                    "id": f"ctext_{len(passages)}",
                                    "text": text,
                                    "language": "classical_chinese",
                                    "source": f"{name}/{chapter}",
                                    "time_periods": [period],
                                }
                            )
                            count += 1
                else:
                    errors.append(f"{chapter}: unexpected response format")

            except requests.exceptions.Timeout:
                errors.append(f"{chapter}: timeout")
            except requests.exceptions.RequestException as e:
                errors.append(f"{chapter}: {type(e).__name__}")
            except json.JSONDecodeError:
                errors.append(f"{chapter}: invalid JSON")
            except Exception as e:
                errors.append(f"{chapter}: {type(e).__name__}: {e}")

        # Always print status for each text
        if count > 0:
            print(f"    {name} ({period}): {count} passages")
        else:
            print(f"    {name} ({period}): 0 passages [FAILED]")

        if errors:
            errors_by_text[name] = errors

    # Print error summary
    if errors_by_text:
        print("\n  ctext.org API errors:")
        for name, errs in errors_by_text.items():
            print(f"    {name}: {len(errs)} failed chapters")
            for err in errs[:3]:  # Show first 3 errors
                print(f"      - {err}")
            if len(errs) > 3:
                print(f"      - ... and {len(errs) - 3} more")
    # === MOZI: Requires nested subsection navigation ===
    print("    Fetching Mozi (nested subsections)...")
    mozi_count = 0
    try:
        CTEXT_LIMITER.wait()
        resp = ctext_session.get("https://api.ctext.org/gettext?urn=ctp:mozi", timeout=30)
        if resp.status_code == 200:
            data = resp.json()
            if "subsections" in data:
                for book_urn in data["subsections"][:10]:
                    CTEXT_LIMITER.wait()
                    book_resp = ctext_session.get(
                        f"https://api.ctext.org/gettext?urn={book_urn}", timeout=30
                    )
                    if book_resp.status_code == 200:
                        book_data = book_resp.json()
                        if "subsections" in book_data:
                            for chapter_urn in book_data["subsections"][:5]:
                                CTEXT_LIMITER.wait()
                                ch_resp = ctext_session.get(
                                    f"https://api.ctext.org/gettext?urn={chapter_urn}", timeout=30
                                )
                                if ch_resp.status_code == 200:
                                    ch_data = ch_resp.json()
                                    if "fulltext" in ch_data:
                                        for text in ch_data["fulltext"]:
                                            if text and len(text) > 10:
                                                passages.append(
                                                    {
                                                        "id": f"ctext_{len(passages)}",
                                                        "text": text,
                                                        "language": "classical_chinese",
                                                        "source": chapter_urn.replace("ctp:", ""),
                                                        "time_periods": ["MOHIST"],
                                                    }
                                                )
                                                mozi_count += 1
        if mozi_count > 0:
            print(f"    Mozi (MOHIST): {mozi_count} passages")
        else:
            print("    Mozi (MOHIST): 0 passages [FAILED]")
    except Exception as e:
        print(f"    Mozi: ERROR - {type(e).__name__}: {e}")

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

    print(f"  ctext.org: {len(passages):,} passages total")
    return passages


# ============================================================================
# ============================================================================
# CHINESE BUDDHIST - CBETA (Chinese Buddhist Electronic Text Association)
# Key sutras for Buddhist moral philosophy
# ============================================================================


def load_chinese_buddhist() -> list[dict]:
    """Load Chinese Buddhist texts from CBETA via CLTK GitHub mirrors.

    Sources key sutras representing Buddhist ethics/philosophy:
    - Diamond Sutra (金剛經) - Prajnaparamita
    - Heart Sutra (心經) - Core emptiness teaching
    - Platform Sutra (六祖壇經) - Chan/Zen ethics
    - Sutra of 42 Sections (四十二章經) - Basic moral teachings
    - Vimalakirti Sutra (維摩詰經) - Lay Buddhist ethics
    - Lotus Sutra (妙法蓮華經) - Devotional Buddhism
    """
    passages = []
    cache_file = CACHE_DIR / "cbeta_buddhist.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            if passages and "time_periods" in passages[0]:
                print(f"  CBETA Buddhist: {len(passages):,} passages (cached)")
                return passages
            print("  CBETA cache missing time_periods - rebuilding...")
            passages = []

    print("  Fetching Chinese Buddhist texts from CBETA/CLTK...")

    # CLTK uses format: cbeta__taisho-tripitaka-electronic-version-no-XXXX__chinese.json
    base_url = "https://raw.githubusercontent.com/cltk/chinese_text_cbeta_02/master/cltk_json"
    texts = [
        ("0235", "Diamond Sutra"),
        ("0251", "Heart Sutra"),
        ("0262", "Lotus Sutra"),
        ("0475", "Vimalakirti Sutra"),
        ("0784", "42 Sections Sutra"),
        ("2008", "Platform Sutra"),
    ]

    for text_num, name in texts:
        try:
            url = (
                f"{base_url}/cbeta__taisho-tripitaka-electronic-version-no-{text_num}__chinese.json"
            )
            resp = ctext_session.get(url, timeout=60)
            if resp.status_code != 200:
                print(f"    {name}: HTTP {resp.status_code}")
                continue

            data = resp.json()
            count = 0

            # CLTK format: {"text": {"0": "line", "1": "line", ...}}
            if isinstance(data, dict) and "text" in data:
                text_dict = data["text"]
                if isinstance(text_dict, dict):
                    for key in sorted(text_dict.keys(), key=lambda x: int(x) if x.isdigit() else 0):
                        text = text_dict[key]
                        if isinstance(text, str) and len(text) > 10:
                            # Skip metadata lines
                            if (
                                text.startswith("No.")
                                or text.startswith("[")
                                or text.startswith("【")
                            ):
                                continue
                            if "CBETA" in text or "Taisho" in text:
                                continue
                            passages.append(
                                {
                                    "id": f"cbeta_T{text_num}_{count}",
                                    "text": text.strip(),
                                    "language": "classical_chinese",
                                    "source": f"CBETA/{name}",
                                    "time_periods": ["BUDDHIST"],
                                }
                            )
                            count += 1

            if count > 0:
                print(f"    {name}: {count} passages")
            else:
                print(f"    {name}: 0 (format issue)")

        except requests.exceptions.Timeout:
            print(f"    {name}: timeout")
        except Exception as e:
            print(f"    {name}: {type(e).__name__}: {e}")

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

    print(f"  CBETA Buddhist: {len(passages):,} passages total")
    return passages


# GREEK/LATIN - Perseus Digital Library (VERIFIED)
# https://github.com/PerseusDL/canonical-greekLit
# https://github.com/PerseusDL/canonical-latinLit
# ============================================================================


def load_perseus_github() -> list[dict]:
    """Load Greek and Latin philosophy from Perseus Digital Library."""
    passages = []
    cache_file = CACHE_DIR / "perseus.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            if passages and "time_periods" in passages[0]:
                print(f"  Perseus: {len(passages):,} passages (cached)")
                return passages
            print("  Perseus cache missing time_periods - rebuilding...")
            passages = []

    print("  Fetching from Perseus GitHub...")

    # Key philosophical texts
    # Format: (author_id, work_id, name, period)
    # URL pattern: /data/{author_id}/{work_id}/{author_id}.{work_id}.perseus-{lang}2.xml
    greek_texts = [
        ("tlg0059", "tlg030", "Plato Republic", "CLASSICAL_GREEK"),
        ("tlg0059", "tlg031", "Plato Laws", "CLASSICAL_GREEK"),
        ("tlg0059", "tlg017", "Plato Apology", "CLASSICAL_GREEK"),
        ("tlg0059", "tlg004", "Plato Symposium", "CLASSICAL_GREEK"),
        ("tlg0059", "tlg003", "Plato Phaedo", "CLASSICAL_GREEK"),
        ("tlg0086", "tlg010", "Aristotle Nicomachean Ethics", "CLASSICAL_GREEK"),
        ("tlg0086", "tlg028", "Aristotle Politics", "CLASSICAL_GREEK"),
        ("tlg0086", "tlg035", "Aristotle Rhetoric", "CLASSICAL_GREEK"),
        ("tlg0555", "tlg001", "Epictetus Discourses", "HELLENISTIC"),
        ("tlg0555", "tlg002", "Epictetus Enchiridion", "HELLENISTIC"),
        ("tlg0562", "tlg001", "Marcus Aurelius Meditations", "HELLENISTIC"),
    ]

    latin_texts = [
        ("phi0474", "phi038", "Cicero De Officiis", "CLASSICAL_LATIN"),
        ("phi0474", "phi044", "Cicero Tusculan Disputations", "CLASSICAL_LATIN"),
        ("phi0474", "phi019", "Cicero De Finibus", "CLASSICAL_LATIN"),
        ("phi0690", "phi003", "Seneca Epistles", "CLASSICAL_LATIN"),
        ("phi0690", "phi001", "Seneca De Beneficiis", "CLASSICAL_LATIN"),
        ("phi0959", "phi001", "Lucretius De Rerum Natura", "CLASSICAL_LATIN"),
    ]

    def fetch_perseus_text(author_id, work_id, name, period, lang_code):
        """Fetch text from Perseus using correct URL pattern."""
        results = []
        repo = "greekLit" if lang_code == "grc" else "latinLit"
        text_id = f"{author_id}.{work_id}"

        # Try multiple filename patterns
        patterns = [
            f"{text_id}.perseus-{lang_code}2.xml",  # Most common: tlg0059.tlg030.perseus-grc2.xml
            f"{text_id}.perseus-{lang_code}1.xml",
            f"{text_id}.{lang_code}1.xml",
        ]

        for pattern in patterns:
            try:
                url = f"https://raw.githubusercontent.com/PerseusDL/canonical-{repo}/master/data/{author_id}/{work_id}/{pattern}"
                GITHUB_LIMITER.wait()
                resp = ctext_session.get(url, timeout=60)
                if resp.status_code == 200:
                    import re

                    # Extract text between tags, remove markup
                    text_content = re.sub(r"<[^>]+>", " ", resp.text)
                    text_content = re.sub(r"\s+", " ", text_content).strip()

                    # Split into chunks of ~500 chars
                    chunks = []
                    words = text_content.split()
                    current = []
                    current_len = 0
                    for word in words:
                        current.append(word)
                        current_len += len(word) + 1
                        if current_len > 400:
                            chunks.append(" ".join(current))
                            current = []
                            current_len = 0
                    if current:
                        chunks.append(" ".join(current))

                    lang = "greek" if lang_code == "grc" else "latin"
                    for i, chunk in enumerate(chunks[:500]):  # Limit per text
                        if len(chunk) > 50:
                            results.append(
                                {
                                    "id": f"perseus_{text_id}_{i}",
                                    "text": chunk,
                                    "language": lang,
                                    "source": name,
                                    "time_periods": [period],
                                }
                            )
                    return results  # Success, stop trying patterns
            except Exception:
                continue
        return results

    # Fetch Greek texts
    for author_id, work_id, name, period in greek_texts:
        result = fetch_perseus_text(author_id, work_id, name, period, "grc")
        passages.extend(result)
        if result:
            print(f"    {name}: {len(result)} passages")

    # Fetch Latin texts
    for author_id, work_id, name, period in latin_texts:
        result = fetch_perseus_text(author_id, work_id, name, period, "lat")
        passages.extend(result)
        if result:
            print(f"    {name}: {len(result)} passages")

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

    print(f"  Perseus: {len(passages):,} passages")
    return passages


# ============================================================================
# WESTERN PHILOSOPHY - Project Gutenberg (direct download by ID)
# Like R's gutenbergr::gutenberg_download() - just give it a list of IDs
# ============================================================================


def load_gutenberg_philosophy(target_passages: int = 5000) -> list[dict]:
    """Load Western philosophy classics from Project Gutenberg by ID.

    Uses gutenberg_download(id) like R's gutenbergr package - just give it IDs.
    JIT loading: fetches texts one at a time, caches individually.

    Args:
        target_passages: Stop fetching after reaching this many passages.
                        Set to 0 for unlimited (fetch all texts).
    """
    passages = []
    cache_dir = CACHE_DIR / "gutenberg_texts"
    cache_dir.mkdir(parents=True, exist_ok=True)

    # Check for combined cache first (legacy)
    legacy_cache = CACHE_DIR / "gutenberg.json"
    if legacy_cache.exists():
        with open(legacy_cache, encoding="utf-8") as f:
            passages = json.load(f)
            print(f"  Gutenberg: {len(passages):,} passages (cached)")
            return passages

    print("  Fetching from GITenberg mirrors (JIT)...")

    # Western Philosophy texts by Gutenberg ID
    # Format: (gutenberg_id, title, period)
    texts = [
        # Kant
        (5683, "Kant Critique of Practical Reason", "MODERN_ETHICS"),
        (5684, "Kant Metaphysical Elements of Ethics", "MODERN_ETHICS"),
        (4280, "Kant Critique of Pure Reason", "MODERN_ETHICS"),
        # Mill
        (11224, "Mill Utilitarianism", "MODERN_ETHICS"),
        (34901, "Mill On Liberty", "MODERN_ETHICS"),
        # Spinoza
        (3800, "Spinoza Ethics", "MODERN_ETHICS"),
        # Aristotle
        (8438, "Aristotle Nicomachean Ethics", "CLASSICAL_GREEK"),
        # Plato
        (1497, "Plato Republic", "CLASSICAL_GREEK"),
        (1656, "Plato Apology", "CLASSICAL_GREEK"),
        # Stoics
        (10661, "Epictetus Discourses", "HELLENISTIC"),
        (2680, "Marcus Aurelius Meditations", "HELLENISTIC"),
        # New Testament & Apocrypha (KJV) - Christian ethics
        (10, "Bible KJV Complete", "BIBLICAL_CHRISTIAN"),  # Complete Bible (80 books)
        (
            124,
            "Apocrypha Deuterocanonical",
            "APOCRYPHA",
        ),  # Tobit, Judith, Wisdom, Sirach, Maccabees
        # Catechisms - Christian doctrine/ethics
        (1670, "Luther Small Catechism", "REFORMATION"),
        (1722, "Luther Large Catechism", "REFORMATION"),
        # American practical ethics
    ]

    headers = {"User-Agent": "Mozilla/5.0 (compatible; BIP-Corpus/1.0)"}

    def gutenberg_download(gutenberg_id: int) -> str | None:
        """Download text from Project Gutenberg by ID. Like R's gutenbergr::gutenberg_download()."""
        # Primary: direct UTF-8 URL (works from most locations)
        urls = [
            f"https://www.gutenberg.org/ebooks/{gutenberg_id}.txt.utf-8",
            f"https://www.gutenberg.org/cache/epub/{gutenberg_id}/pg{gutenberg_id}.txt",
        ]
        for url in urls:
            try:
                r = ctext_session.get(url, headers=headers, timeout=60)
                if r.status_code == 200:
                    return r.text
            except Exception:
                continue
        return None

    def extract_passages(content: str, guten_id: int, name: str, period: str) -> list[dict]:
        """Extract paragraphs from Gutenberg text content."""
        results = []
        # Normalize line endings (Gutenberg uses \r\n)
        content = content.replace("\r\n", "\n").replace("\r", "\n")

        # Skip Gutenberg header/footer
        start_marker = "*** START OF"
        end_marker = "*** END OF"
        start_idx = content.find(start_marker)
        end_idx = content.find(end_marker)
        if start_idx > 0:
            content = content[start_idx:]
            newline_idx = content.find("\n")
            if newline_idx > 0:
                content = content[newline_idx + 1 :]
        if end_idx > 0 and start_idx > 0:
            content = content[: end_idx - start_idx - 100]
        elif end_idx > 0:
            content = content[:end_idx]

        # Split into paragraphs
        paras = content.split("\n\n")
        count = 0
        for para in paras:
            para = para.strip().replace("\n", " ")
            para = " ".join(para.split())
            if len(para) > 100 and len(para) < 2000:
                results.append(
                    {
                        "id": f"gutenberg_{guten_id}_{count}",
                        "text": para,
                        "language": "english",
                        "source": name,
                        "time_periods": [period],
                    }
                )
                count += 1
        return results

    # JIT loading: fetch texts one at a time, stop when we have enough
    for guten_id, name, period in texts:
        # Check individual cache first
        text_cache = cache_dir / f"{guten_id}.json"
        if text_cache.exists():
            with open(text_cache, encoding="utf-8") as f:
                text_passages = json.load(f)
                passages.extend(text_passages)
                print(f"    {name}: {len(text_passages):,} passages (cached)")
        else:
            # Download by ID (like R's gutenbergr::gutenberg_download)
            time.sleep(0.3)  # Rate limit
            content = gutenberg_download(guten_id)

            if content:
                text_passages = extract_passages(content, guten_id, name, period)
                if text_passages:
                    # Cache this text individually
                    with open(text_cache, "w", encoding="utf-8") as f:
                        json.dump(text_passages, f, ensure_ascii=False)
                    passages.extend(text_passages)
                    print(f"    {name}: {len(text_passages):,} passages")
                else:
                    print(f"    {name}: no passages extracted")
            else:
                print(f"    {name}: download failed (ID {guten_id})")

        # JIT early stop: if we have enough, stop fetching
        if target_passages > 0 and len(passages) >= target_passages:
            print(f"    (reached {target_passages:,} target, stopping)")
            break

    print(f"  Gutenberg: {len(passages):,} passages")
    return passages


# ============================================================================
# NATIVE AMERICAN & WORLD FOLKLORE - HuggingFace (VERIFIED)
# Source: merve/folk-mythology-tales (246K stories from Ashliman Folktexts)
# ============================================================================


def load_folk_mythology() -> list[dict]:
    """Load folk tales and mythology including Native American from HuggingFace."""
    passages = []
    cache_file = CACHE_DIR / "folk_mythology.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            print(f"  Folk/Mythology: {len(passages):,} passages (cached)")
            return passages

    print("  Loading folk-mythology-tales from HuggingFace...")
    try:
        from datasets import load_dataset

        ds = load_dataset("merve/folk-mythology-tales", split="train")

        for i, item in enumerate(ds):
            text = item.get("text", "")
            if text and len(text) > 50:
                passages.append(
                    {
                        "id": f"folk_{i}",
                        "text": text.strip()[:2000],  # Limit length
                        "language": "english",
                        "source": "Ashliman Folktexts",
                        "time_periods": ["FOLKLORE", "TRADITIONAL"],
                    }
                )
                if len(passages) >= 50000:  # Limit total
                    break
        print(f"    Loaded {len(passages):,} folk tales")
    except ImportError:
        print("    datasets library not available, skipping")
    except Exception as e:
        print(f"    Failed: {e}")

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

    print(f"  Folk/Mythology: {len(passages):,} passages")
    return passages


# ============================================================================
# ROMANCE LANGUAGE PHILOSOPHY - Project Gutenberg (direct download by ID)
# Spanish: Don Quixote, La Celestina, Lazarillo de Tormes
# French: Montaigne, Voltaire, Rousseau, Pascal
# Italian: Machiavelli, Dante, Boccaccio | Portuguese: Camões
# ============================================================================


def load_romance_philosophy(target_passages: int = 10000) -> list[dict]:
    """Load Romance language philosophy from Project Gutenberg by ID.

    Uses gutenberg_download(id) like R's gutenbergr package.
    JIT loading: fetches texts one at a time, caches individually.

    Args:
        target_passages: Stop fetching after reaching this many passages.
                        Set to 0 for unlimited (fetch all texts).
    """
    passages = []
    cache_dir = CACHE_DIR / "romance_texts"
    cache_dir.mkdir(parents=True, exist_ok=True)

    # Check for legacy combined cache
    legacy_cache = CACHE_DIR / "romance_philosophy.json"
    if legacy_cache.exists():
        with open(legacy_cache, encoding="utf-8") as f:
            passages = json.load(f)
            print(f"  Romance Philosophy: {len(passages):,} passages (cached)")
            return passages

    print("  Fetching Romance philosophy (JIT)...")

    # Romance language texts by Gutenberg ID
    # Format: (gutenberg_id, title, language, period)
    texts = [
        # Spanish
        (996, "Don Quixote", "spanish", "SPANISH_GOLDEN_AGE"),
        (1619, "La Celestina", "spanish", "SPANISH_GOLDEN_AGE"),
        (320, "Lazarillo de Tormes", "spanish", "SPANISH_GOLDEN_AGE"),
        # French
        (19942, "Candide (Voltaire)", "french", "FRENCH_ENLIGHTENMENT"),
        (46333, "Social Contract (Rousseau)", "french", "FRENCH_ENLIGHTENMENT"),
        (3600, "Essais de Montaigne", "french", "FRENCH_RENAISSANCE"),
        (18269, "Pensées (Pascal)", "french", "FRENCH_ENLIGHTENMENT"),
        # Italian
        (1232, "The Prince (Machiavelli)", "italian", "ITALIAN_RENAISSANCE"),
        (1004, "Divine Comedy (Dante)", "italian", "MEDIEVAL_ITALIAN"),
        (3726, "Decameron Vol I (Boccaccio)", "italian", "MEDIEVAL_ITALIAN"),
        (13102, "Decameron Vol II (Boccaccio)", "italian", "MEDIEVAL_ITALIAN"),
        # Portuguese
        (3333, "Os Lusíadas (Camões)", "portuguese", "PORTUGUESE_RENAISSANCE"),
    ]

    headers = {"User-Agent": "Mozilla/5.0 (compatible; BIP-Corpus/1.0)"}

    def gutenberg_download(gutenberg_id: int) -> str | None:
        """Download text from Project Gutenberg by ID."""
        urls = [
            f"https://www.gutenberg.org/ebooks/{gutenberg_id}.txt.utf-8",
            f"https://www.gutenberg.org/cache/epub/{gutenberg_id}/pg{gutenberg_id}.txt",
        ]
        for url in urls:
            try:
                r = ctext_session.get(url, headers=headers, timeout=60)
                if r.status_code == 200:
                    return r.text
            except Exception:
                continue
        return None

    def extract_passages(
        content: str, guten_id: int, name: str, lang: str, period: str
    ) -> list[dict]:
        """Extract paragraphs from Gutenberg text content."""
        results = []
        # Normalize line endings (Gutenberg uses \r\n)
        content = content.replace("\r\n", "\n").replace("\r", "\n")

        # Skip Gutenberg header/footer
        start_marker = "*** START OF"
        end_marker = "*** END OF"
        start_idx = content.find(start_marker)
        end_idx = content.find(end_marker)
        if start_idx > 0:
            content = content[start_idx:]
            nl = content.find("\n")
            if nl > 0:
                content = content[nl + 1 :]
        if end_idx > 0 and start_idx > 0:
            content = content[: end_idx - start_idx - 100]
        elif end_idx > 0:
            content = content[:end_idx]

        # Split into paragraphs
        paras = content.split("\n\n")
        count = 0
        for para in paras:
            para = para.strip().replace("\n", " ")
            para = " ".join(para.split())
            if len(para) > 100 and len(para) < 2000:
                results.append(
                    {
                        "id": f"romance_{guten_id}_{count}",
                        "text": para,
                        "language": lang,
                        "source": name,
                        "time_periods": [period],
                    }
                )
                count += 1
        return results

    # JIT loading: fetch texts one at a time
    for guten_id, name, lang, period in texts:
        text_cache = cache_dir / f"{guten_id}.json"
        if text_cache.exists():
            with open(text_cache, encoding="utf-8") as f:
                text_passages = json.load(f)
                passages.extend(text_passages)
                print(f"    {name}: {len(text_passages):,} passages (cached)")
        else:
            # Download by ID (like R's gutenbergr::gutenberg_download)
            time.sleep(0.3)  # Rate limit
            content = gutenberg_download(guten_id)

            if content:
                text_passages = extract_passages(content, guten_id, name, lang, period)
                if text_passages:
                    with open(text_cache, "w", encoding="utf-8") as f:
                        json.dump(text_passages, f, ensure_ascii=False)
                    passages.extend(text_passages)
                    print(f"    {name}: {len(text_passages):,} passages")
                else:
                    print(f"    {name}: no passages extracted")
            else:
                print(f"    {name}: download failed (ID {guten_id})")

        # JIT early stop
        if target_passages > 0 and len(passages) >= target_passages:
            print(f"    (reached {target_passages:,} target, stopping)")
            break

    print(f"  Romance Philosophy: {len(passages):,} passages")
    return passages


# ============================================================================
# ENGLISH ETHICS - Dear Abby (68K letters)
# Source: https://github.com/Mac-STAT/data (VERIFIED)
# ============================================================================

DEAR_ABBY_URL = "https://raw.githubusercontent.com/Mac-STAT/data/main/dear_abby.csv"


def load_dear_abby() -> list[dict]:
    """Load Dear Abby advice columns (68K letters) from Mac-STAT GitHub."""
    passages = []
    cache_file = CACHE_DIR / "dear_abby.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            print(f"  Dear Abby: {len(passages):,} passages (cached)")
            return passages

    # Check local file first
    local_paths = [
        Path("data/raw/dear_abby.csv"),
        DATA_DIR / "dear_abby.csv",
    ]

    csv_file = None
    for p in local_paths:
        if p.exists():
            csv_file = p
            print(f"  Found local: {csv_file}")
            break

    # Download if not local
    if not csv_file:
        print("  Downloading Dear Abby from GitHub (17.9MB)...")
        csv_file = DATA_DIR / "dear_abby.csv"
        try:
            resp = requests.get(DEAR_ABBY_URL, timeout=120)
            if resp.status_code == 200:
                csv_file.parent.mkdir(parents=True, exist_ok=True)
                with open(csv_file, "w", encoding="utf-8") as f:
                    f.write(resp.text)
                print(f"    Downloaded to {csv_file}")
            else:
                print(f"    Failed: HTTP {resp.status_code}")
                return passages
        except Exception as e:
            print(f"    Download failed: {e}")
            return passages

    # Parse CSV using pandas for better handling of multi-line fields
    print("  Parsing Dear Abby CSV...")
    try:
        import pandas as pd

        df = pd.read_csv(csv_file, encoding="utf-8", on_bad_lines="skip")
        print(f"    CSV has {len(df):,} rows, columns: {list(df.columns)}")

        skipped_empty = 0
        skipped_short = 0
        for _, row in df.iterrows():
            # Primary column is "question_only"
            text = str(row.get("question_only", ""))
            if not text or text == "nan" or pd.isna(row.get("question_only")):
                skipped_empty += 1
                continue
            if len(text) <= 50:
                skipped_short += 1
                continue
            year = row.get("year", "")
            passages.append(
                {
                    "id": f"abby_{row.get('letterId', len(passages))}",
                    "text": text.strip(),
                    "language": "english",
                    "source": f"Dear Abby {year}",
                    "time_periods": ["DEAR_ABBY", "MODERN", "ENGLISH_ETHICS"],
                }
            )
        print(
            f"    Loaded {len(passages):,} letters (skipped: {skipped_empty} empty, {skipped_short} short)"
        )
    except ImportError:
        # Fallback to csv module if pandas not available
        print("    pandas not available, using csv module")
        with open(csv_file, encoding="utf-8", newline="") as f:
            reader = csv.DictReader(f)
            for row in reader:
                text = row.get("question_only", "")
                if text and len(text) > 50:
                    passages.append(
                        {
                            "id": f"abby_{row.get('letterId', len(passages))}",
                            "text": text.strip(),
                            "language": "english",
                            "source": f"Dear Abby {row.get('year', '')}",
                            "time_periods": ["DEAR_ABBY", "MODERN", "ENGLISH_ETHICS"],
                        }
                    )
        print(f"    Loaded {len(passages):,} letters")
    except Exception as e:
        print(f"    Failed to parse CSV: {e}")

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

    print(f"  Dear Abby: {len(passages):,} passages")
    return passages


# ============================================================================
# ENGLISH ETHICS - hendrycks/ethics (134K examples) - SUPPLEMENTAL
# ============================================================================


def load_hendrycks_ethics() -> list[dict]:
    """Load ethics scenarios from hendrycks/ethics dataset (supplemental).

    Downloads directly from Berkeley (HuggingFace loader is deprecated).
    """
    passages = []
    cache_file = CACHE_DIR / "hendrycks_ethics.json"

    if cache_file.exists():
        with open(cache_file, encoding="utf-8") as f:
            passages = json.load(f)
            print(f"  hendrycks/ethics: {len(passages):,} passages (cached)")
            return passages

    print("  Downloading hendrycks/ethics from Berkeley...")

    import io
    import tarfile

    TAR_URL = "https://people.eecs.berkeley.edu/~hendrycks/ethics.tar"

    try:
        resp = requests.get(TAR_URL, timeout=120)
        resp.raise_for_status()
    except Exception as e:
        print(f"    Download failed: {e}")
        return passages

    # Extract CSVs from tar
    subsets = {
        "commonsense": ("cm_train.csv", "input"),
        "deontology": ("deontology_train.csv", "scenario"),
        "justice": ("justice_train.csv", "scenario"),
        "utilitarianism": ("util_train.csv", "baseline"),  # Note: different name
        "virtue": ("virtue_train.csv", "scenario"),
    }

    try:
        tar_bytes = io.BytesIO(resp.content)
        with tarfile.open(fileobj=tar_bytes, mode="r:") as tar:
            for subset, (filename, text_col) in subsets.items():
                # Find the file in the archive (may be in subdirectory)
                csv_member = None
                for member in tar.getmembers():
                    if member.name.endswith(filename):
                        csv_member = member
                        break

                if not csv_member:
                    print(f"    {subset}: file not found ({filename})")
                    continue

                # Extract and parse CSV
                csv_file = tar.extractfile(csv_member)
                if csv_file is None:
                    continue

                csv_content = csv_file.read().decode("utf-8")
                reader = csv.DictReader(io.StringIO(csv_content))

                count = 0
                for row in reader:
                    text = row.get(text_col, "")
                    if text and len(text) > 30:
                        passages.append(
                            {
                                "id": f"ethics_{subset}_{count}",
                                "text": text.strip(),
                                "language": "english",
                                "source": f"hendrycks/ethics/{subset}",
                                "time_periods": ["MODERN_ETHICS", "MODERN", "ENGLISH_ETHICS"],
                            }
                        )
                        count += 1

                print(f"    {subset}: {count} passages")

    except Exception as e:
        print(f"    Tar extraction failed: {e}")

    if passages:
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(passages, f, ensure_ascii=False)
        cache_to_drive(cache_file, quiet=True)

    print(f"  hendrycks/ethics: {len(passages):,} passages")
    return passages


# ============================================================================
# MAIN LOADER
# ============================================================================

print("\n" + "-" * 60)
print("Fetching from verified external sources...")
print(f"Cache only mode: {_cache_only}")
print("-" * 60)

by_language = defaultdict(list)

# Load all sources
print("\n[SANSKRIT]")
_t0 = time.time()
by_language["sanskrit"].extend(load_itihasa_github())
print(f"  Elapsed: {time.time() - _t0:.1f}s")

print("\n[PALI]")
_t0 = time.time()
by_language["pali"].extend(load_pali_suttacentral())
print(f"  Elapsed: {time.time() - _t0:.1f}s")

print("\n[ARABIC]")
_t0 = time.time()
by_language["arabic"].extend(load_quran_tanzil())
print(f"  Elapsed: {time.time() - _t0:.1f}s")

print("\n[HEBREW/ARAMAIC]")
_t0 = time.time()
sefaria = load_sefaria_github()
print(f"  Elapsed: {time.time() - _t0:.1f}s")
for p in sefaria:
    by_language[p["language"]].append(p)

print("\n[CHINESE]")
_t0 = time.time()
by_language["classical_chinese"].extend(load_chinese_ctext())
print(f"  Elapsed: {time.time() - _t0:.1f}s")
print("\n[CHINESE BUDDHIST]")
_t0 = time.time()
by_language["classical_chinese"].extend(load_chinese_buddhist())
print(f"  Elapsed: {time.time() - _t0:.1f}s")  # CBETA Buddhist

print("\n[GREEK/LATIN]")
_t0 = time.time()
perseus = load_perseus_github()
print(f"  Elapsed: {time.time() - _t0:.1f}s")
for p in perseus:
    by_language[p["language"]].append(p)

print("\n[WESTERN PHILOSOPHY]")
_t0 = time.time()
by_language["english"].extend(load_gutenberg_philosophy())
print(f"  Elapsed: {time.time() - _t0:.1f}s")

print("\n[ROMANCE LANGUAGES]")
_t0 = time.time()
romance = load_romance_philosophy()
print(f"  Elapsed: {time.time() - _t0:.1f}s")
for p in romance:
    by_language[p["language"]].append(p)

print("\n[ENGLISH ETHICS]")
_t0 = time.time()
by_language["english"].extend(load_dear_abby())
by_language["english"].extend(load_hendrycks_ethics())
print(f"  Elapsed: {time.time() - _t0:.1f}s")  # Supplemental

print("\n[FOLKLORE/NATIVE AMERICAN]")
_t0 = time.time()
by_language["english"].extend(load_folk_mythology())
print(f"  Elapsed: {time.time() - _t0:.1f}s")

# ============================================================================
# APPLY MEMORY LIMITS
# ============================================================================

print("\n" + "-" * 60)
print("Applying memory limits...")
for lang in list(by_language.keys()):
    max_count = MAX_PASSAGES_PER_LANG.get(lang, MAX_PASSAGES_PER_LANG["default"])
    if len(by_language[lang]) > max_count:
        original = len(by_language[lang])
        by_language[lang] = by_language[lang][:max_count]
        print(f"  {lang}: {original:,} -> {max_count:,}")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 60)
print("CORPUS SUMMARY")
print("=" * 60)

total = 0
for lang, passages in sorted(by_language.items(), key=lambda x: -len(x[1])):
    count = len(passages)
    total += count
    status = "OK" if count >= MIN_PASSAGES else f"NEED {MIN_PASSAGES - count} MORE"
    print(f"  {lang:20s}: {count:6,} passages  [{status}]")

print("-" * 60)
print(f"  {'TOTAL':20s}: {total:6,} passages")
print("=" * 60)

# ============================================================================
# SAVE TO JSONL FOR LATER CELLS
# ============================================================================

print("\nConverting to training format...")

all_passages = []
for _lang, passages in by_language.items():
    for p in passages:
        all_passages.append(
            {
                "id": p["id"],
                "text": p["text"],
                "language": p["language"],
                "source": p["source"],
                "time_periods": p.get("time_periods", [p.get("time_period", "UNKNOWN")]),
            }
        )

os.makedirs("data/processed", exist_ok=True)
with open("data/processed/passages.jsonl", "w", encoding="utf-8") as f:
    for p in all_passages:
        f.write(json.dumps(p, ensure_ascii=False) + "\n")
print("Saved to data/processed/passages.jsonl")

# Cache to Drive if available
if _save_dir and os.path.exists(os.path.dirname(_save_dir)):
    os.makedirs(_save_dir, exist_ok=True)
    import shutil

    shutil.copy("data/processed/passages.jsonl", f"{_save_dir}/passages.jsonl")
    print(f"Cached to {_save_dir}/passages.jsonl")

    # Also cache the corpus cache files to Drive for faster restarts
    drive_cache = Path(_save_dir) / "corpus_cache"
    drive_cache.mkdir(exist_ok=True)
    cached_count = 0
    for cache_file in CACHE_DIR.glob("*.json"):
        dest = drive_cache / cache_file.name
        if not dest.exists() or dest.stat().st_size != cache_file.stat().st_size:
            shutil.copy(cache_file, dest)
            cached_count += 1
    if cached_count:
        print(f"  Cached {cached_count} corpus files to Drive")

    # Note: sefaria.json (processed output) IS cached to Drive
    # Raw Sefaria-Export not cached (50K files), but processed cache restores instantly

print("\n" + "=" * 60)
print("CORPUS LOADING COMPLETE")
print("=" * 60)

LOADING CORPORA (v10.12 - Self-Contained)
Restored 10 cache files from Drive

------------------------------------------------------------
Fetching from verified external sources...
Cache only mode: False
------------------------------------------------------------

[SANSKRIT]
  Itihasa: 93,009 passages (cached)
  Elapsed: 0.4s

[PALI]
  SuttaCentral: 42,513 passages (cached)
  Elapsed: 0.2s

[ARABIC]
  Tanzil Quran: 6,235 passages (cached)
  Elapsed: 0.0s

[HEBREW/ARAMAIC]
  Sefaria: 10,000 passages (cached)
  Elapsed: 0.0s

[CHINESE]
  ctext.org: 1,647 passages (cached)
    CONFUCIAN: 1141
    DAOIST: 81
    MOHIST: 425
  Elapsed: 0.0s

[CHINESE BUDDHIST]
  CBETA Buddhist: 3,277 passages (cached)
  Elapsed: 0.0s

[GREEK/LATIN]
  Perseus: 4,290 passages (cached)
  Elapsed: 0.0s

[WESTERN PHILOSOPHY]
  Fetching from GITenberg mirrors (JIT)...
    Kant Critique of Practical Reason: 182 passages
    Kant Metaphysical Elements of Ethics: 91 passages
    Kant Critique of Pure Reason: 1,036

In [ ]:
# @title 3. Load Ethics Datasets for Bond Extraction { display-mode: "form" }
# @markdown Load ETHICS, Scruples, and EthicsSuite datasets for bond extraction training
# @markdown These provide modern English moral reasoning examples with labeled judgments

# @markdown ---
# @markdown ### Dataset Selection
LOAD_ETHICS_DATASET = True  # @param {type:"boolean"}
# @markdown hendrycks/ethics: Justice, deontology, virtue, utilitarianism, commonsense

LOAD_SCRUPLES_DATASET = True  # @param {type:"boolean"}
# @markdown allenai/scruples: 32K real-life anecdotes with ethical judgments

LOAD_ETHICSUITE_DATASET = True  # @param {type:"boolean"}
# @markdown LLM-Ethics/EthicsSuite: 20K complex contextualized moral situations

# @markdown ---
# @markdown ### Size Limits (0 = unlimited)
MAX_ETHICS_ITEMS = 50000  # @param {type:"integer"}
MAX_SCRUPLES_ITEMS = 30000  # @param {type:"integer"}
MAX_ETHICSUITE_ITEMS = 20000  # @param {type:"integer"}

# @markdown ---
# @markdown ### Output Options
EXPORT_BIP_FORMAT = True  # @param {type:"boolean"}
# @markdown Export as BIP passages for integration with Cell 2 corpus

CREATE_TRAIN_TEST_SPLIT = True  # @param {type:"boolean"}
TEST_SPLIT_RATIO = 0.2  # @param {type:"number"}

import json
import os
import re
from collections import defaultdict
from dataclasses import asdict, dataclass
from pathlib import Path

print("=" * 60)
print("BOND EXTRACTION TRAINING DATA (v10.14)")
print("=" * 60)


# =============================================================================
# BOND SCHEMA
# =============================================================================


@dataclass
class BondAnnotation:
    """A moral bond extracted from text."""

    text: str
    agent: str | None
    patient: str | None
    bond_type: str
    hohfeld_state: str
    context: str
    confidence: float
    source_dataset: str
    source_category: str
    raw_label: str


BOND_TYPES = [
    "OBLIGATION",
    "PROHIBITION",
    "PERMISSION",
    "CLAIM",
    "POWER",
    "IMMUNITY",
    "VIRTUE",
    "VICE",
    "SUPEREROGATORY",
]

HOHFELD_STATES = [
    "DUTY",
    "CLAIM",
    "LIBERTY",
    "NO_CLAIM",
    "POWER",
    "LIABILITY",
    "IMMUNITY",
    "DISABILITY",
]


# =============================================================================
# ETHICS DATASET LOADER
# =============================================================================


class EthicsLoader:
    """Load hendrycks/ethics dataset."""

    CATEGORY_TO_BOND = {
        "deontology": ("OBLIGATION", "DUTY"),
        "justice": ("CLAIM", "CLAIM"),
        "virtue": ("VIRTUE", "DUTY"),
        "utilitarianism": ("PERMISSION", "LIBERTY"),
        "commonsense": ("OBLIGATION", "DUTY"),
    }

    def load(self, max_items: int = 0) -> list[BondAnnotation]:
        try:
            from datasets import load_dataset
        except ImportError:
            print("  Installing datasets library...")
            os.system("pip install datasets -q")
            from datasets import load_dataset

        annotations = []
        categories = ["commonsense", "deontology", "justice", "utilitarianism", "virtue"]

        for category in categories:
            if max_items > 0 and len(annotations) >= max_items:
                break

            print(f"  Loading ETHICS/{category}...")
            try:
                dataset = load_dataset("hendrycks/ethics", category)

                for split in ["train", "test"]:
                    if split not in dataset:
                        continue
                    for item in dataset[split]:
                        if max_items > 0 and len(annotations) >= max_items:
                            break

                        text = item.get("input") or item.get("scenario") or item.get("text", "")
                        if not text or len(text) < 10:
                            continue

                        label = item.get("label", 0)
                        bond_type, hohfeld = self.CATEGORY_TO_BOND.get(
                            category, ("OBLIGATION", "DUTY")
                        )

                        # Extract agent/patient
                        agent, patient = self._extract_roles(text)

                        if label == 1:
                            context = "descriptive"
                            if bond_type == "OBLIGATION":
                                bond_type = "PROHIBITION"
                        else:
                            context = "prescriptive"

                        annotations.append(
                            BondAnnotation(
                                text=text[:500],
                                agent=agent,
                                patient=patient,
                                bond_type=bond_type,
                                hohfeld_state=hohfeld,
                                context=context,
                                confidence=0.8,
                                source_dataset="ethics",
                                source_category=category,
                                raw_label=str(label),
                            )
                        )

            except Exception as e:
                print(f"    Warning: {e}")

        return annotations

    def _extract_roles(self, text: str) -> tuple[str | None, str | None]:
        agent = patient = None

        if re.match(r"^I\s+(should|must|ought)", text, re.I):
            agent = "speaker"
        elif re.match(r"^You\s+(should|must|ought)", text, re.I):
            agent = "addressee"
        else:
            match = re.match(r"^([A-Z][a-z]+)\s+(should|must|ought)", text)
            if match:
                agent = match.group(1).lower()

        patient_match = re.search(r"(help|protect|harm|hurt)\s+(\w+)", text, re.I)
        if patient_match:
            p = patient_match.group(2).lower()
            if p not in ["the", "a", "an", "my", "your"]:
                patient = p

        return agent, patient


# =============================================================================
# SCRUPLES DATASET LOADER
# =============================================================================


class ScruplesLoader:
    """Load allenai/scruples dataset."""

    LABEL_TO_BOND = {
        "AUTHOR_WRONG": ("PROHIBITION", "DUTY", "author"),
        "OTHER_WRONG": ("PROHIBITION", "DUTY", "other"),
        "EVERYBODY_WRONG": ("PROHIBITION", "DUTY", "both"),
        "NOBODY_WRONG": ("PERMISSION", "LIBERTY", None),
        "INFO": ("OBLIGATION", "DUTY", None),
    }

    ANECDOTES_URL = "https://storage.googleapis.com/ai2-mosaic-public/projects/scruples/v1.0/data/anecdotes.tar.gz"
    DILEMMAS_URL = "https://storage.googleapis.com/ai2-mosaic-public/projects/scruples/v1.0/data/dilemmas.tar.gz"

    def load(self, max_items: int = 0) -> list[BondAnnotation]:
        import tarfile

        import requests

        cache_dir = Path("data/ethics_cache")
        cache_dir.mkdir(parents=True, exist_ok=True)
        annotations = []

        # Load anecdotes from Google Cloud
        print("  Loading Scruples/anecdotes...")
        anecdotes_cache = cache_dir / "scruples_anecdotes.tar.gz"
        try:
            if not anecdotes_cache.exists():
                print("    Downloading from Google Cloud...")
                resp = requests.get(self.ANECDOTES_URL, timeout=120)
                resp.raise_for_status()
                with open(anecdotes_cache, "wb") as f:
                    f.write(resp.content)
                print(f"    Downloaded {len(resp.content) // 1024}KB")

            with tarfile.open(anecdotes_cache, "r:gz") as tar:
                for member in tar.getmembers():
                    if not member.name.endswith(".jsonl"):
                        continue
                    f = tar.extractfile(member)
                    if f is None:
                        continue
                    for line in f:
                        if max_items > 0 and len(annotations) >= max_items:
                            break
                        try:
                            item = json.loads(line.decode("utf-8"))
                        except:
                            continue
                        title = item.get("title", "")
                        text = item.get("text", "")
                        full_text = f"{title} {text}".strip()
                        if len(full_text) < 20:
                            continue
                        label = item.get("binarized_label", item.get("label", 0))
                        bond_info = self.LABEL_TO_BOND.get(label, ("OBLIGATION", "DUTY", None))
                        if len(bond_info) == 3:
                            bond_type, hohfeld, violator = bond_info
                        else:
                            bond_type, hohfeld = bond_info[:2]
                            violator = None
                        agent = patient = None
                        if violator == "author":
                            agent, patient = "author", "other"
                        elif violator == "other":
                            agent, patient = "other", "author"
                        annotations.append(
                            BondAnnotation(
                                text=full_text[:500],
                                agent=agent,
                                patient=patient,
                                bond_type=bond_type,
                                hohfeld_state=hohfeld,
                                context="descriptive",
                                confidence=0.7,
                                source_dataset="scruples",
                                source_category="anecdotes",
                                raw_label=str(label),
                            )
                        )
            print(f"    Loaded {len(annotations)} anecdotes")
        except Exception as e:
            print(f"    Warning: Dataset 'allenai/scruples' - {e}")

        # Load dilemmas
        print("  Loading Scruples/dilemmas...")
        dilemmas_cache = cache_dir / "scruples_dilemmas.tar.gz"
        dilemma_start = len(annotations)
        try:
            if not dilemmas_cache.exists():
                resp = requests.get(self.DILEMMAS_URL, timeout=120)
                resp.raise_for_status()
                with open(dilemmas_cache, "wb") as f:
                    f.write(resp.content)

            with tarfile.open(dilemmas_cache, "r:gz") as tar:
                for member in tar.getmembers():
                    if not member.name.endswith(".jsonl"):
                        continue
                    f = tar.extractfile(member)
                    if f is None:
                        continue
                    for line in f:
                        try:
                            item = json.loads(line.decode("utf-8"))
                        except:
                            continue
                        action1 = item.get("action1", "")
                        action2 = item.get("action2", "")
                        text = f"Choice A: {action1} Choice B: {action2}"
                        if len(text) < 20:
                            continue
                        annotations.append(
                            BondAnnotation(
                                text=text[:500],
                                agent="actor",
                                patient="affected",
                                bond_type="OBLIGATION",
                                hohfeld_state="DUTY",
                                context="hypothetical",
                                confidence=0.6,
                                source_dataset="scruples",
                                source_category="dilemmas",
                                raw_label=str(item.get("label", 0)),
                            )
                        )
            print(f"    Loaded {len(annotations) - dilemma_start} dilemmas")
        except Exception as e:
            print(f"    Warning: Dataset 'allenai/scruples' - {e}")

        return annotations

    def _load_hf_legacy(self, max_items: int = 0) -> list[BondAnnotation]:
        """Legacy HuggingFace loader (no longer works)."""
        try:
            from datasets import load_dataset

            dataset = load_dataset("allenai/scruples", "anecdotes")

            for split in ["train", "dev", "test"]:
                if split not in dataset:
                    continue
                for item in dataset[split]:
                    if max_items > 0 and len(annotations) >= max_items:
                        break

                    title = item.get("title", "")
                    text = item.get("text", "")
                    full_text = f"{title}\n{text}" if title else text

                    if len(full_text) < 20:
                        continue

                    label = item.get("binarized_label") or item.get("label", "INFO")
                    if isinstance(label, int):
                        label = "AUTHOR_WRONG" if label == 1 else "NOBODY_WRONG"

                    bond_type, hohfeld, violator = self.LABEL_TO_BOND.get(
                        label, ("OBLIGATION", "DUTY", None)
                    )

                    agent = patient = None
                    if violator == "author":
                        agent, patient = "author", "other"
                    elif violator == "other":
                        agent, patient = "other", "author"
                    elif violator == "both":
                        agent = patient = "both"

                    annotations.append(
                        BondAnnotation(
                            text=full_text[:500],
                            agent=agent,
                            patient=patient,
                            bond_type=bond_type,
                            hohfeld_state=hohfeld,
                            context="descriptive",
                            confidence=0.7,
                            source_dataset="scruples",
                            source_category="anecdotes",
                            raw_label=label,
                        )
                    )

        except Exception as e:
            print(f"    Warning: {e}")

        # Load dilemmas
        print("  Loading Scruples/dilemmas...")
        try:
            dataset = load_dataset("allenai/scruples", "dilemmas")
            dilemma_limit = max_items // 3 if max_items > 0 else 0

            count = 0
            for split in ["train", "dev", "test"]:
                if split not in dataset:
                    continue
                for item in dataset[split]:
                    if dilemma_limit > 0 and count >= dilemma_limit:
                        break

                    action1 = item.get("action1", "")
                    action2 = item.get("action2", "")
                    text = f"Choice A: {action1}\nChoice B: {action2}"

                    if len(text) < 20:
                        continue

                    annotations.append(
                        BondAnnotation(
                            text=text[:500],
                            agent="actor",
                            patient="affected",
                            bond_type="OBLIGATION",
                            hohfeld_state="DUTY",
                            context="hypothetical",
                            confidence=0.6,
                            source_dataset="scruples",
                            source_category="dilemmas",
                            raw_label=str(item.get("label", 0)),
                        )
                    )
                    count += 1

        except Exception as e:
            print(f"    Warning: {e}")

        return annotations


# =============================================================================
# ETHICSUITE LOADER
# =============================================================================


class EthicsSuiteLoader:
    """Load LLM-Ethics/EthicsSuite dataset."""

    def load(self, max_items: int = 0) -> list[BondAnnotation]:
        import urllib.request

        url = "https://raw.githubusercontent.com/LLM-Ethics/EthicsSuite/main/data.jsonl"
        cache_dir = Path("data/ethics_cache")
        cache_dir.mkdir(parents=True, exist_ok=True)
        cache_file = cache_dir / "ethicsuite.jsonl"

        annotations = []

        print("  Loading EthicsSuite...")
        try:
            if not cache_file.exists():
                print("    Downloading...")
                urllib.request.urlretrieve(url, cache_file)

            category_map = {
                "deontology": ("OBLIGATION", "DUTY"),
                "justice": ("CLAIM", "CLAIM"),
                "virtue": ("VIRTUE", "DUTY"),
                "utilitarianism": ("PERMISSION", "LIBERTY"),
                "commonsense": ("OBLIGATION", "DUTY"),
            }

            with open(cache_file, encoding="utf-8") as f:
                for line in f:
                    if max_items > 0 and len(annotations) >= max_items:
                        break

                    item = json.loads(line)
                    text = item.get("text", "")
                    if len(text) < 20:
                        continue

                    source = item.get("source", "unknown")
                    bond_type, hohfeld = category_map.get(source, ("OBLIGATION", "DUTY"))

                    annotations.append(
                        BondAnnotation(
                            text=text[:500],
                            agent=None,
                            patient=None,
                            bond_type=bond_type,
                            hohfeld_state=hohfeld,
                            context="hypothetical",
                            confidence=0.75,
                            source_dataset="ethicsuite",
                            source_category=source,
                            raw_label=item.get("original_text", "")[:100],
                        )
                    )

        except Exception as e:
            print(f"    Warning: {e}")

        return annotations


# =============================================================================
# MAIN LOADING LOGIC
# =============================================================================

all_bond_annotations = []

if LOAD_ETHICS_DATASET:
    print("\n[1] ETHICS Dataset (hendrycks/ethics)")
    loader = EthicsLoader()
    ethics_anns = loader.load(MAX_ETHICS_ITEMS)
    print(f"    Loaded: {len(ethics_anns):,} annotations")
    all_bond_annotations.extend(ethics_anns)

if LOAD_SCRUPLES_DATASET:
    print("\n[2] Scruples Dataset (allenai/scruples)")
    loader = ScruplesLoader()
    scruples_anns = loader.load(MAX_SCRUPLES_ITEMS)
    print(f"    Loaded: {len(scruples_anns):,} annotations")
    all_bond_annotations.extend(scruples_anns)

if LOAD_ETHICSUITE_DATASET:
    print("\n[3] EthicsSuite Dataset (LLM-Ethics/EthicsSuite)")
    loader = EthicsSuiteLoader()
    suite_anns = loader.load(MAX_ETHICSUITE_ITEMS)
    print(f"    Loaded: {len(suite_anns):,} annotations")
    all_bond_annotations.extend(suite_anns)


# =============================================================================
# STATISTICS
# =============================================================================

print("\n" + "=" * 60)
print("BOND EXTRACTION DATA STATISTICS")
print("=" * 60)

stats = {
    "by_dataset": defaultdict(int),
    "by_bond_type": defaultdict(int),
    "by_hohfeld": defaultdict(int),
    "by_context": defaultdict(int),
    "by_category": defaultdict(int),
    "has_agent": 0,
    "has_patient": 0,
}

for ann in all_bond_annotations:
    stats["by_dataset"][ann.source_dataset] += 1
    stats["by_bond_type"][ann.bond_type] += 1
    stats["by_hohfeld"][ann.hohfeld_state] += 1
    stats["by_context"][ann.context] += 1
    stats["by_category"][ann.source_category] += 1
    if ann.agent:
        stats["has_agent"] += 1
    if ann.patient:
        stats["has_patient"] += 1

print(f"\nTotal annotations: {len(all_bond_annotations):,}")

print("\nBy Dataset:")
for ds, count in sorted(stats["by_dataset"].items()):
    print(f"  {ds}: {count:,}")

print("\nBy Bond Type:")
for bt, count in sorted(stats["by_bond_type"].items(), key=lambda x: -x[1]):
    print(f"  {bt}: {count:,}")

print("\nBy Context:")
for ctx, count in sorted(stats["by_context"].items(), key=lambda x: -x[1]):
    print(f"  {ctx}: {count:,}")

print(
    f"\nAgent extracted: {stats['has_agent']:,} ({100 * stats['has_agent'] / max(1, len(all_bond_annotations)):.1f}%)"
)
print(
    f"Patient extracted: {stats['has_patient']:,} ({100 * stats['has_patient'] / max(1, len(all_bond_annotations)):.1f}%)"
)


# =============================================================================
# EXPORT
# =============================================================================

output_dir = Path("data/bond_training")
output_dir.mkdir(parents=True, exist_ok=True)

# Save all annotations
print("\n" + "=" * 60)
print("SAVING DATA")
print("=" * 60)

with open(output_dir / "bond_annotations.jsonl", "w", encoding="utf-8") as f:
    for ann in all_bond_annotations:
        f.write(json.dumps(asdict(ann), ensure_ascii=False) + "\n")
print(f"Saved: {output_dir / 'bond_annotations.jsonl'}")

# Train/test split
if CREATE_TRAIN_TEST_SPLIT:
    import random

    random.seed(42)
    shuffled = all_bond_annotations.copy()
    random.shuffle(shuffled)
    split_idx = int(len(shuffled) * (1 - TEST_SPLIT_RATIO))
    train_anns = shuffled[:split_idx]
    test_anns = shuffled[split_idx:]

    with open(output_dir / "bond_train.jsonl", "w", encoding="utf-8") as f:
        for ann in train_anns:
            f.write(json.dumps(asdict(ann), ensure_ascii=False) + "\n")

    with open(output_dir / "bond_test.jsonl", "w", encoding="utf-8") as f:
        for ann in test_anns:
            f.write(json.dumps(asdict(ann), ensure_ascii=False) + "\n")

    print(f"Train/Test split: {len(train_anns):,} / {len(test_anns):,}")

# Export in BIP format
if EXPORT_BIP_FORMAT:
    bip_passages = []
    for i, ann in enumerate(all_bond_annotations):
        passage = {
            "id": f"ethics_{ann.source_dataset}_{i}",
            "text": ann.text,
            "language": "english",
            "time_periods": ["MODERN_ETHICS"],
            "tags": ["modern", "english", "western", "ethics", ann.source_category],
            "bonds": [
                {
                    "agent": ann.agent or "unspecified",
                    "patient": ann.patient or "unspecified",
                    "bond_type": ann.bond_type,
                    "hohfeld_state": ann.hohfeld_state,
                    "context": ann.context,
                    "confidence": ann.confidence,
                }
            ],
            "source": ann.source_dataset,
            "category": ann.source_category,
        }
        bip_passages.append(passage)

    with open(output_dir / "ethics_corpus.jsonl", "w", encoding="utf-8") as f:
        for p in bip_passages:
            f.write(json.dumps(p, ensure_ascii=False) + "\n")

    print(f"BIP format: {output_dir / 'ethics_corpus.jsonl'}")

print("\n" + "=" * 60)
print("BOND EXTRACTION DATA READY")
print("=" * 60)
print("Use data/bond_training/bond_train.jsonl for training")
print("Use data/bond_training/ethics_corpus.jsonl for BIP integration")

BOND EXTRACTION TRAINING DATA (v10.14)

[1] ETHICS Dataset (hendrycks/ethics)
  Loading ETHICS/commonsense...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

ethics.py: 0.00B [00:00, ?B/s]

  Loading ETHICS/deontology...
  Loading ETHICS/justice...
  Loading ETHICS/utilitarianism...
  Loading ETHICS/virtue...
    Loaded: 0 annotations

[2] Scruples Dataset (allenai/scruples)
  Loading Scruples/anecdotes...
    Downloaded 24600KB
    Loaded 30000 anecdotes
  Loading Scruples/dilemmas...
    Loaded 68286 dilemmas
    Loaded: 98,286 annotations

[3] EthicsSuite Dataset (LLM-Ethics/EthicsSuite)
  Loading EthicsSuite...
    Downloading...
    Loaded: 19,804 annotations

BOND EXTRACTION DATA STATISTICS

Total annotations: 118,090

By Dataset:
  ethicsuite: 19,804
  scruples: 98,286

By Bond Type:
  OBLIGATION: 118,090

By Context:
  hypothetical: 88,090
  descriptive: 30,000

Agent extracted: 68,286 (57.8%)
Patient extracted: 68,286 (57.8%)

SAVING DATA
Saved: data/bond_training/bond_annotations.jsonl
Train/Test split: 94,472 / 23,618
BIP format: data/bond_training/ethics_corpus.jsonl

BOND EXTRACTION DATA READY
Use data/bond_training/bond_train.jsonl for training
Use data/bond

In [ ]:
# @title 4. Patterns + Normalization { display-mode: "form" }
# @markdown BIP v10.16: Enhanced NLP bond extraction with selectable methods
# @markdown - Level 1: Regex patterns (fast, all languages)
# @markdown - Level 2: Grammar-aware (Chinese, Arabic, Hebrew, Sanskrit)
# @markdown - Level 3: spaCy dependency parsing (English only)

# @markdown ---
# @markdown ### Extraction Method
EXTRACTION_LEVEL = "level2"  # @param ["level1", "level2", "level3"]
# @markdown - **level1**: Regex patterns only (fastest, baseline)
# @markdown - **level2**: Language-specific grammar analysis (recommended)
# @markdown - **level3**: spaCy NLP for English + level2 for others

USE_SPACY_FOR_ENGLISH = True  # @param {type:"boolean"}
# @markdown Enable spaCy dependency parsing for English texts (level3 only)

INSTALL_SPACY_IF_NEEDED = True  # @param {type:"boolean"}
# @markdown Auto-install spaCy and en_core_web_md model if not available

# @markdown ---
# @markdown ### Extraction Options
EXTRACT_AGENT_PATIENT = True  # @param {type:"boolean"}
# @markdown Extract agent/patient roles from text (level2/3 only)

DETECT_CAUSATIVES = True  # @param {type:"boolean"}
# @markdown Detect causative constructions

DETECT_PASSIVES = True  # @param {type:"boolean"}
# @markdown Detect passive voice constructions
import re
import unicodedata
from enum import Enum, auto

print("=" * 60)
print("TEXT NORMALIZATION & PATTERNS")
print("=" * 60)


# ===== TEXT NORMALIZATION =====
def normalize_hebrew(text):
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"[\u0591-\u05C7]", "", text)  # Remove nikud
    for final, regular in [
        ("\u05da", "\u05db"),
        ("\u05dd", "\u05de"),
        ("\u05df", "\u05e0"),
        ("\u05e3", "\u05e4"),
        ("\u05e5", "\u05e6"),
    ]:
        text = text.replace(final, regular)
    return text


def normalize_arabic(text):
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"[\u064B-\u065F]", "", text)  # Remove tashkeel
    text = text.replace("\u0640", "")  # Remove tatweel
    for v in ["\u0623", "\u0625", "\u0622", "\u0671"]:
        text = text.replace(v, "\u0627")
    text = text.replace("\u0629", "\u0647").replace("\u0649", "\u064a")
    return text


# NEW in v10.9: Sanskrit normalization
def normalize_sanskrit(text):
    """Normalize Sanskrit/Devanagari text."""
    text = unicodedata.normalize("NFC", text)
    # Remove vedic accents and other diacriticals
    text = re.sub(r"[\u0951-\u0954]", "", text)  # Vedic tone marks
    text = re.sub(r"[\u0900-\u0902]", "", text)  # Chandrabindu variants
    return text


# NEW in v10.9: Pali normalization
def normalize_pali(text):
    """Normalize Pali text (romanized or script)."""
    text = unicodedata.normalize("NFC", text)
    # Normalize romanized Pali diacritics
    text = text.lower()
    # Handle common Pali romanization variations
    text = text.replace("ṃ", "m").replace("ṅ", "n").replace("ñ", "n")
    text = text.replace("ṭ", "t").replace("ḍ", "d").replace("ṇ", "n")
    text = text.replace("ḷ", "l").replace("ā", "a").replace("ī", "i").replace("ū", "u")
    return text


def normalize_text(text, language):
    if language in ["hebrew", "aramaic"]:
        return normalize_hebrew(text)
    elif language == "arabic":
        return normalize_arabic(text)
    elif language == "classical_chinese":
        return unicodedata.normalize("NFKC", text)
    elif language == "sanskrit":
        return normalize_sanskrit(text)
    elif language == "pali":
        return normalize_pali(text)
    else:
        return unicodedata.normalize("NFKC", text.lower())


# ===== BOND AND HOHFELD TYPES =====
class BondType(Enum):
    HARM_PREVENTION = auto()
    RECIPROCITY = auto()
    AUTONOMY = auto()
    PROPERTY = auto()
    FAMILY = auto()
    AUTHORITY = auto()
    CARE = auto()
    FAIRNESS = auto()
    CONTRACT = auto()
    NONE = auto()


class HohfeldState(Enum):
    OBLIGATION = auto()
    RIGHT = auto()
    LIBERTY = auto()
    NO_RIGHT = auto()


# ===== COMPLETE BOND PATTERNS =====
ALL_BOND_PATTERNS = {
    "hebrew": {
        BondType.HARM_PREVENTION: [
            r"\u05d4\u05e8\u05d2",
            r"\u05e8\u05e6\u05d7",
            r"\u05e0\u05d6\u05e7",
            r"\u05d4\u05db\u05d4",
            r"\u05d4\u05e6\u05d9\u05dc",
            r"\u05e9\u05de\u05e8",
            r"\u05e4\u05e7\u05d5\u05d7.\u05e0\u05e4\u05e9",
        ],
        BondType.RECIPROCITY: [
            r"\u05d2\u05de\u05d5\u05dc",
            r"\u05d4\u05e9\u05d9\u05d1",
            r"\u05e4\u05e8\u05e2",
            r"\u05e0\u05ea\u05df.*\u05e7\u05d1\u05dc",
            r"\u05de\u05d3\u05d4.\u05db\u05e0\u05d2\u05d3",
        ],
        BondType.AUTONOMY: [
            r"\u05d1\u05d7\u05e8",
            r"\u05e8\u05e6\u05d5\u05df",
            r"\u05d7\u05e4\u05e9",
            r"\u05e2\u05e6\u05de",
        ],
        BondType.PROPERTY: [
            r"\u05e7\u05e0\u05d4",
            r"\u05de\u05db\u05e8",
            r"\u05d2\u05d6\u05dc",
            r"\u05d2\u05e0\u05d1",
            r"\u05de\u05de\u05d5\u05df",
            r"\u05e0\u05db\u05e1",
            r"\u05d9\u05e8\u05e9",
        ],
        BondType.FAMILY: [
            r"\u05d0\u05d1",
            r"\u05d0\u05dd",
            r"\u05d1\u05df",
            r"\u05db\u05d1\u05d3.*\u05d0\u05d1",
            r"\u05db\u05d1\u05d3.*\u05d0\u05dd",
            r"\u05de\u05e9\u05e4\u05d7\u05d4",
            r"\u05d0\u05d7",
            r"\u05d0\u05d7\u05d5\u05ea",
        ],
        BondType.AUTHORITY: [
            r"\u05de\u05dc\u05db",
            r"\u05e9\u05d5\u05e4\u05d8",
            r"\u05e6\u05d5\u05d4",
            r"\u05ea\u05d5\u05e8\u05d4",
            r"\u05de\u05e6\u05d5\u05d4",
            r"\u05d3\u05d9\u05df",
            r"\u05d7\u05e7",
        ],
        BondType.CARE: [
            r"\u05d7\u05e1\u05d3",
            r"\u05e8\u05d7\u05dd",
            r"\u05e2\u05d6\u05e8",
            r"\u05ea\u05de\u05db",
            r"\u05e6\u05d3\u05e7\u05d4",
        ],
        BondType.FAIRNESS: [
            r"\u05e6\u05d3\u05e7",
            r"\u05de\u05e9\u05e4\u05d8",
            r"\u05d9\u05e9\u05e8",
            r"\u05e9\u05d5\u05d4",
        ],
        BondType.CONTRACT: [
            r"\u05d1\u05e8\u05d9\u05ea",
            r"\u05e0\u05d3\u05e8",
            r"\u05e9\u05d1\u05d5\u05e2",
            r"\u05d4\u05ea\u05d7\u05d9\u05d1",
            r"\u05e2\u05e8\u05d1",
        ],
    },
    "aramaic": {
        BondType.HARM_PREVENTION: [
            r"\u05e7\u05d8\u05dc",
            r"\u05e0\u05d6\u05e7",
            r"\u05d7\u05d1\u05dc",
            r"\u05e9\u05d6\u05d9\u05d1",
            r"\u05e4\u05e6\u05d9",
        ],
        BondType.RECIPROCITY: [r"\u05e4\u05e8\u05e2", r"\u05e9\u05dc\u05dd", r"\u05d0\u05d2\u05e8"],
        BondType.AUTONOMY: [r"\u05e6\u05d1\u05d9", r"\u05e8\u05e2\u05d5"],
        BondType.PROPERTY: [
            r"\u05d6\u05d1\u05df",
            r"\u05e7\u05e0\u05d4",
            r"\u05d2\u05d6\u05dc",
            r"\u05de\u05de\u05d5\u05e0\u05d0",
            r"\u05e0\u05db\u05e1\u05d9",
        ],
        BondType.FAMILY: [
            r"\u05d0\u05d1\u05d0",
            r"\u05d0\u05de\u05d0",
            r"\u05d1\u05e8\u05d0",
            r"\u05d1\u05e8\u05ea\u05d0",
            r"\u05d9\u05e7\u05e8",
            r"\u05d0\u05d7\u05d0",
        ],
        BondType.AUTHORITY: [
            r"\u05de\u05dc\u05db\u05d0",
            r"\u05d3\u05d9\u05e0\u05d0",
            r"\u05d3\u05d9\u05d9\u05e0\u05d0",
            r"\u05e4\u05e7\u05d5\u05d3\u05d0",
            r"\u05d0\u05d5\u05e8\u05d9\u05ea",
        ],
        BondType.CARE: [r"\u05d7\u05e1\u05d3", r"\u05e8\u05d7\u05dd", r"\u05e1\u05e2\u05d3"],
        BondType.FAIRNESS: [
            r"\u05d3\u05d9\u05e0\u05d0",
            r"\u05e7\u05e9\u05d5\u05d8",
            r"\u05ea\u05e8\u05d9\u05e6",
        ],
        BondType.CONTRACT: [
            r"\u05e7\u05d9\u05de\u05d0",
            r"\u05e9\u05d1\u05d5\u05e2\u05d4",
            r"\u05e0\u05d3\u05e8\u05d0",
            r"\u05e2\u05e8\u05d1\u05d0",
        ],
    },
    "classical_chinese": {
        BondType.HARM_PREVENTION: [
            r"\u6bba",
            r"\u5bb3",
            r"\u50b7",
            r"\u6551",
            r"\u8b77",
            r"\u885b",
            r"\u66b4",
        ],
        BondType.RECIPROCITY: [r"\u5831", r"\u9084", r"\u511f", r"\u8ced", r"\u7b54"],
        BondType.AUTONOMY: [r"\u81ea", r"\u7531", r"\u4efb", r"\u610f", r"\u5fd7"],
        BondType.PROPERTY: [
            r"\u8ca1",
            r"\u7269",
            r"\u7522",
            r"\u76dc",
            r"\u7aca",
            r"\u8ce3",
            r"\u8cb7",
        ],
        BondType.FAMILY: [
            r"\u5b5d",
            r"\u7236",
            r"\u6bcd",
            r"\u89aa",
            r"\u5b50",
            r"\u5f1f",
            r"\u5144",
            r"\u5bb6",
        ],
        BondType.AUTHORITY: [
            r"\u541b",
            r"\u81e3",
            r"\u738b",
            r"\u547d",
            r"\u4ee4",
            r"\u6cd5",
            r"\u6cbb",
        ],
        BondType.CARE: [r"\u4ec1", r"\u611b", r"\u6148", r"\u60e0", r"\u6069", r"\u6190"],
        BondType.FAIRNESS: [r"\u7fa9", r"\u6b63", r"\u516c", r"\u5e73", r"\u5747"],
        BondType.CONTRACT: [r"\u7d04", r"\u76df", r"\u8a93", r"\u8afe", r"\u4fe1"],
    },
    "arabic": {
        BondType.HARM_PREVENTION: [
            r"\u0642\u062a\u0644",
            r"\u0636\u0631\u0631",
            r"\u0627\u0630[\u064a\u0649]",
            r"\u0638\u0644\u0645",
            r"\u0627\u0646\u0642\u0630",
            r"\u062d\u0641\u0638",
            r"\u0627\u0645\u0627\u0646",
        ],
        BondType.RECIPROCITY: [
            r"\u062c\u0632\u0627",
            r"\u0631\u062f",
            r"\u0642\u0635\u0627\u0635",
            r"\u0645\u062b\u0644",
            r"\u0639\u0648\u0636",
        ],
        BondType.AUTONOMY: [
            r"\u062d\u0631",
            r"\u0627\u0631\u0627\u062f\u0629",
            r"\u0627\u062e\u062a\u064a\u0627\u0631",
            r"\u0645\u0634\u064a\u0626",
        ],
        BondType.PROPERTY: [
            r"\u0645\u0627\u0644",
            r"\u0645\u0644\u0643",
            r"\u0633\u0631\u0642",
            r"\u0628\u064a\u0639",
            r"\u0634\u0631\u0627",
            r"\u0645\u064a\u0631\u0627\u062b",
            r"\u063a\u0635\u0628",
        ],
        BondType.FAMILY: [
            r"\u0648\u0627\u0644\u062f",
            r"\u0627\u0628\u0648",
            r"\u0627\u0645",
            r"\u0627\u0628\u0646",
            r"\u0628\u0646\u062a",
            r"\u0627\u0647\u0644",
            r"\u0642\u0631\u0628[\u064a\u0649]",
            r"\u0631\u062d\u0645",
        ],
        BondType.AUTHORITY: [
            r"\u0637\u0627\u0639",
            r"\u0627\u0645\u0631",
            r"\u062d\u0643\u0645",
            r"\u0633\u0644\u0637\u0627\u0646",
            r"\u062e\u0644\u064a\u0641",
            r"\u0627\u0645\u0627\u0645",
            r"\u0634\u0631\u064a\u0639",
        ],
        BondType.CARE: [
            r"\u0631\u062d\u0645",
            r"\u0627\u062d\u0633\u0627\u0646",
            r"\u0639\u0637\u0641",
            r"\u0635\u062f\u0642",
            r"\u0632\u0643\u0627",
        ],
        BondType.FAIRNESS: [
            r"\u0639\u062f\u0644",
            r"\u0642\u0633\u0637",
            r"\u062d\u0642",
            r"\u0627\u0646\u0635\u0627\u0641",
            r"\u0633\u0648[\u064a\u0649]",
        ],
        BondType.CONTRACT: [
            r"\u0639\u0647\u062f",
            r"\u0639\u0642\u062f",
            r"\u0646\u0630\u0631",
            r"\u064a\u0645\u064a\u0646",
            r"\u0648\u0641\u0627",
            r"\u0627\u0645\u0627\u0646",
        ],
    },
    "english": {
        BondType.HARM_PREVENTION: [
            r"\bkill",
            r"\bmurder",
            r"\bharm",
            r"\bhurt",
            r"\bsave",
            r"\bprotect",
            r"\bviolence",
        ],
        BondType.RECIPROCITY: [
            r"\breturn",
            r"\brepay",
            r"\bexchange",
            r"\bgive.*back",
            r"\breciproc",
        ],
        BondType.AUTONOMY: [
            r"\bfree",
            r"\bchoice",
            r"\bchoose",
            r"\bconsent",
            r"\bautonomy",
            r"\bright to",
        ],
        BondType.PROPERTY: [
            r"\bsteal",
            r"\btheft",
            r"\bown",
            r"\bproperty",
            r"\bbelong",
            r"\binherit",
        ],
        BondType.FAMILY: [
            r"\bfather",
            r"\bmother",
            r"\bparent",
            r"\bchild",
            r"\bfamily",
            r"\bhonor.*parent",
        ],
        BondType.AUTHORITY: [
            r"\bobey",
            r"\bcommand",
            r"\bauthority",
            r"\blaw",
            r"\brule",
            r"\bgovern",
        ],
        BondType.CARE: [r"\bcare", r"\bhelp", r"\bkind", r"\bcompassion", r"\bcharity", r"\bmercy"],
        BondType.FAIRNESS: [r"\bfair", r"\bjust", r"\bequal", r"\bequity", r"\bright\b"],
        BondType.CONTRACT: [
            r"\bpromise",
            r"\bcontract",
            r"\bagreem",
            r"\bvow",
            r"\boath",
            r"\bcommit",
        ],
    },
    "sanskrit": {
        BondType.HARM_PREVENTION: [r"हिंसा", r"अहिंसा", r"वध", r"रक्षा", r"त्राण"],
        BondType.RECIPROCITY: [r"प्रतिदान", r"प्रत्युपकार", r"दान", r"ऋण"],
        BondType.AUTONOMY: [r"स्वतंत्र", r"मोक्ष", r"स्वेच्छा"],
        BondType.PROPERTY: [r"धन", r"स्व", r"चोर", r"दाय"],
        BondType.FAMILY: [r"पितृ", r"मातृ", r"पुत्र", r"कुल", r"गृह"],
        BondType.AUTHORITY: [r"राज", r"धर्म", r"विधि", r"नियम", r"शास्त्र"],
        BondType.CARE: [r"करुणा", r"दया", r"प्रेम", r"मैत्री", r"सेवा"],
        BondType.FAIRNESS: [r"न्याय", r"समता", r"धर्म", r"ऋत"],
        BondType.CONTRACT: [r"प्रतिज्ञा", r"संविद", r"वचन", r"शपथ"],
    },
    "pali": {
        BondType.HARM_PREVENTION: [r"himsa", r"ahimsa", r"panatipata", r"rakkhati"],
        BondType.RECIPROCITY: [r"dana", r"patidana", r"ina"],
        BondType.AUTONOMY: [r"vimutti", r"nibbana", r"attadhipa"],
        BondType.PROPERTY: [r"dhana", r"theyya", r"adinnadana"],
        BondType.FAMILY: [r"mata", r"pita", r"putta", r"kula"],
        BondType.AUTHORITY: [r"raja", r"dhamma", r"vinaya", r"sikkhapada"],
        BondType.CARE: [r"karuna", r"metta", r"mudita", r"upekkha"],
        BondType.FAIRNESS: [r"samma", r"dhamma", r"sacca"],
        BondType.CONTRACT: [r"patijna", r"vacana", r"sacca"],
    },
}

# ===== COMPLETE HOHFELD PATTERNS =====
ALL_HOHFELD_PATTERNS = {
    "hebrew": {
        HohfeldState.OBLIGATION: [
            r"\u05d7\u05d9\u05d9\u05d1",
            r"\u05e6\u05e8\u05d9\u05db",
            r"\u05de\u05d5\u05db\u05e8\u05d7",
            r"\u05de\u05e6\u05d5\u05d5\u05d4",
        ],
        HohfeldState.RIGHT: [
            r"\u05d6\u05db\u05d5\u05ea",
            r"\u05e8\u05e9\u05d0\u05d9",
            r"\u05d6\u05db\u05d0\u05d9",
            r"\u05de\u05d2\u05d9\u05e2",
        ],
        HohfeldState.LIBERTY: [
            r"\u05de\u05d5\u05ea\u05e8",
            r"\u05e8\u05e9\u05d5\u05ea",
            r"\u05e4\u05d8\u05d5\u05e8",
            r"\u05d9\u05db\u05d5\u05dc",
        ],
        HohfeldState.NO_RIGHT: [
            r"\u05d0\u05e1\u05d5\u05e8",
            r"\u05d0\u05d9\u05e0\u05d5 \u05e8\u05e9\u05d0\u05d9",
            r"\u05d0\u05d9\u05df.*\u05d6\u05db\u05d5\u05ea",
        ],
    },
    "aramaic": {
        HohfeldState.OBLIGATION: [
            r"\u05d7\u05d9\u05d9\u05d1",
            r"\u05de\u05d7\u05d5\u05d9\u05d1",
            r"\u05d1\u05e2\u05d9",
        ],
        HohfeldState.RIGHT: [
            r"\u05d6\u05db\u05d5\u05ea",
            r"\u05e8\u05e9\u05d0\u05d9",
            r"\u05d6\u05db\u05d9",
        ],
        HohfeldState.LIBERTY: [
            r"\u05e9\u05e8\u05d9",
            r"\u05de\u05d5\u05ea\u05e8",
            r"\u05e4\u05d8\u05d5\u05e8",
        ],
        HohfeldState.NO_RIGHT: [
            r"\u05d0\u05e1\u05d5\u05e8",
            r"\u05dc\u05d0.*\u05e8\u05e9\u05d0\u05d9",
        ],
    },
    "classical_chinese": {
        HohfeldState.OBLIGATION: [r"\u5fc5", r"\u9808", r"\u7576", r"\u61c9", r"\u5b9c"],
        HohfeldState.RIGHT: [r"\u53ef", r"\u5f97", r"\u6b0a", r"\u5b9c"],
        HohfeldState.LIBERTY: [r"\u8a31", r"\u4efb", r"\u807d", r"\u514d"],
        HohfeldState.NO_RIGHT: [r"\u4e0d\u53ef", r"\u52ff", r"\u7981", r"\u83ab", r"\u975e"],
    },
    "arabic": {
        HohfeldState.OBLIGATION: [
            r"\u064a\u062c\u0628",
            r"\u0648\u0627\u062c\u0628",
            r"\u0641\u0631\u0636",
            r"\u0644\u0627\u0632\u0645",
            r"\u0648\u062c\u0648\u0628",
        ],
        HohfeldState.RIGHT: [
            r"\u062d\u0642",
            r"\u064a\u062d\u0642",
            r"\u062c\u0627\u0626\u0632",
            r"\u064a\u062c\u0648\u0632",
        ],
        HohfeldState.LIBERTY: [
            r"\u0645\u0628\u0627\u062d",
            r"\u062d\u0644\u0627\u0644",
            r"\u062c\u0627\u0626\u0632",
            r"\u0627\u0628\u0627\u062d",
        ],
        HohfeldState.NO_RIGHT: [
            r"\u062d\u0631\u0627\u0645",
            r"\u0645\u062d\u0631\u0645",
            r"\u0645\u0645\u0646\u0648\u0639",
            r"\u0644\u0627 \u064a\u062c\u0648\u0632",
            r"\u0646\u0647[\u064a\u0649]",
        ],
    },
    "english": {
        HohfeldState.OBLIGATION: [r"\bmust\b", r"\bshall\b", r"\bobligat", r"\bduty", r"\brequir"],
        HohfeldState.RIGHT: [r"\bright\b", r"\bentitle", r"\bdeserve", r"\bclaim"],
        HohfeldState.LIBERTY: [r"\bmay\b", r"\bpermit", r"\ballow", r"\bfree to"],
        HohfeldState.NO_RIGHT: [r"\bforbid", r"\bprohibit", r"\bmust not", r"\bshall not"],
    },
    "sanskrit": {
        HohfeldState.OBLIGATION: [r"कर्तव्य", r"अवश्य", r"नियम", r"विधि"],
        HohfeldState.RIGHT: [r"अधिकार", r"स्वत्व"],
        HohfeldState.LIBERTY: [r"शक्य", r"अनुज्ञा", r"उचित"],
        HohfeldState.NO_RIGHT: [r"निषिद्ध", r"वर्जित", r"अकर्तव्य"],
    },
    "pali": {
        HohfeldState.OBLIGATION: [r"kicca", r"karaniiya", r"dhammo"],
        HohfeldState.RIGHT: [r"adhikaara", r"bhaaga"],
        HohfeldState.LIBERTY: [r"anujaanati", r"kappati"],
        HohfeldState.NO_RIGHT: [r"nisiddha", r"akaraniya", r"na kappati"],
    },
}


# ===== CONTEXT MARKERS FOR GRAMMAR-AWARE EXTRACTION =====
CONTEXT_MARKERS = {
    "hebrew": {
        "negation": [r"לא", r"אל", r"אין", r"בלי", r"אינ"],
        "obligation": [r"חייב", r"צריך", r"מוכרח", r"צווה"],
        "prohibition": [r"אסור", r"אל.*ת"],
        "permission": [r"מותר", r"רשאי", r"פטור"],
    },
    "aramaic": {
        "negation": [r"לא", r"לית", r"לאו"],
        "obligation": [r"חייב", r"בעי"],
        "prohibition": [r"אסור"],
        "permission": [r"שרי", r"מותר"],
    },
    "classical_chinese": {
        "negation": [r"不", r"非", r"無", r"未", r"毋"],
        "obligation": [r"必", r"當", r"須", r"應", r"宜"],
        "prohibition": [r"勿", r"禁", r"莫", r"不可"],
        "permission": [r"可", r"得", r"許"],
    },
    "arabic": {
        "negation": [r"لا", r"ما", r"ليس", r"لم", r"غير"],
        "obligation": [r"يجب", r"واجب", r"فرض", r"عليه"],
        "prohibition": [r"حرام", r"محرم", r"لا يجوز", r"نهى"],
        "permission": [r"حلال", r"مباح", r"جائز"],
    },
    "english": {
        "negation": [r"not", r"no", r"never", r"neither", r"n't"],
        "obligation": [r"must", r"shall", r"should", r"ought", r"required"],
        "prohibition": [r"forbid", r"prohibit", r"must not", r"shall not", r"don't"],
        "permission": [r"may", r"can", r"allowed", r"permit"],
    },
    "sanskrit": {
        "negation": [r"न", r"मा", r"अ"],
        "obligation": [r"कर्तव्य", r"अवश्य", r"विधि"],
        "prohibition": [r"निषिद्ध", r"वर्जित", r"मा"],
        "permission": [r"शक्य", r"अनुज्ञा"],
    },
    "pali": {
        "negation": [r"na", r"ma", r"a-"],
        "obligation": [r"kicca", r"karaniya"],
        "prohibition": [r"nisiddha", r"akaraniya"],
        "permission": [r"anujaanati", r"kappati"],
    },
}


def detect_context(text, language, match_pos, window=30):
    """Detect grammatical context around a pattern match."""
    markers = CONTEXT_MARKERS.get(language, {})
    if not markers:
        return "unknown", None

    start = max(0, match_pos - window)
    end = min(len(text), match_pos + window)
    window_text = text[start:end]

    for marker_type in ["prohibition", "obligation", "permission"]:
        for pattern in markers.get(marker_type, []):
            if re.search(pattern, window_text):
                return "prescriptive", marker_type

    for pattern in markers.get("negation", []):
        if re.search(pattern, window_text):
            return "descriptive", "negated"

    return "descriptive", None


# ===== NLP IMPROVEMENTS (v10.9 Phase 1) =====
NEGATION_CUES = {
    "english": ["not", "no", "never", "neither", "nor", "n't", "without", "lack", "none"],
    "classical_chinese": ["不", "非", "無", "莫", "勿", "未", "弗", "毋", "否"],
    "arabic": ["لا", "ما", "لم", "لن", "ليس", "غير", "بدون"],
    "hebrew": ["לא", "אל", "בלי", "אין", "מבלי"],
    "aramaic": ["לא", "לית", "לאו"],
    "sanskrit": ["न", "मा", "अ"],
    "pali": ["na", "ma", "a", "an"],
}

MODAL_CLASSIFICATION = {
    "english": {
        "obligation": ["must", "shall", "have to", "ought to", "need to", "required", "obligated"],
        "permission": ["may", "can", "allowed", "permitted", "free to", "entitled"],
        "prohibition": ["must not", "shall not", "cannot", "forbidden", "prohibited", "banned"],
        "supererogation": ["should", "ought", "would be good", "ideally", "preferably"],
    },
    "classical_chinese": {
        "obligation": ["必", "當", "宜", "須", "應", "要"],
        "permission": ["可", "得", "許", "容", "能"],
        "prohibition": ["不可", "不得", "勿", "莫", "禁", "不許", "不宜"],
        "supererogation": ["善", "美", "德", "宜"],
    },
    "arabic": {
        "obligation": ["يجب", "فرض", "واجب", "لازم", "فريضة"],
        "permission": ["يجوز", "مباح", "حلال", "جائز"],
        "prohibition": ["حرام", "محرم", "ممنوع", "لا يجوز", "محظور"],
        "supererogation": ["مستحب", "سنة", "مندوب", "نافلة"],
    },
    "hebrew": {
        "obligation": ["חייב", "מצווה", "צריך", "מוכרח", "חובה"],
        "permission": ["מותר", "רשאי", "יכול", "היתר"],
        "prohibition": ["אסור", "לא יעשה", "אל", "איסור"],
        "supererogation": ["ראוי", "טוב", "מידת חסידות", "לפנים משורת הדין"],
    },
    "sanskrit": {
        "obligation": ["कर्तव्य", "अवश्य", "नियम"],
        "permission": ["शक्य", "अनुज्ञा"],
        "prohibition": ["निषिद्ध", "वर्जित", "मा"],
    },
    "pali": {
        "obligation": ["kicca", "karaniya", "dhamma"],
        "permission": ["kappati", "anujanati"],
        "prohibition": ["akappiya", "akaraniya", "na kappati"],
    },
}


# =============================================================================
# LEVEL 2: LANGUAGE-SPECIFIC GRAMMAR-AWARE EXTRACTORS (v10.16)
# =============================================================================

from dataclasses import dataclass, field


@dataclass
class ExtractedAgent:
    """Agent of the moral action."""

    text: str
    position: str | None = None
    case_marking: str | None = None


@dataclass
class ExtractedPatient:
    """Patient/theme of the moral action."""

    text: str
    position: str | None = None
    case_marking: str | None = None


@dataclass
class MoralFeature:
    """A morally-relevant grammatical feature."""

    feature_type: str
    value: str
    source: str


@dataclass
class EnhancedBondResult:
    """Full bond extraction result."""

    bond_type: str | None = None
    hohfeld_state: str = "OBLIGATION"
    agent: ExtractedAgent | None = None
    patient: ExtractedPatient | None = None
    is_negated: bool = False
    modal: str | None = None
    context: str = "unknown"
    moral_features: list[MoralFeature] = field(default_factory=list)
    confidence: float = 0.5
    method: str = "unknown"

    def to_dict(self) -> dict:
        """Convert to dict compatible with existing code."""
        return {
            "bond_type": self.bond_type,
            "hohfeld_state": self.hohfeld_state,
            "agent": self.agent.text if self.agent else None,
            "patient": self.patient.text if self.patient else None,
            "negated": self.is_negated,
            "modal": self.modal,
            "confidence": self.confidence,
            "context": self.context,
            "method": self.method,
        }


class ClassicalChineseExtractor:
    """Classical Chinese extraction using position and particles."""

    PARTICLES = {
        "之": "GEN",
        "者": "AGENT_NOM",
        "所": "PATIENT_NOM",
        "於": "PREP_LOC",
        "以": "PREP_INST",
        "為": "COPULA",
        "被": "PASSIVE",
        "見": "PASSIVE",
        "使": "CAUSATIVE",
        "令": "CAUSATIVE",
    }

    MODALS = {
        "必": "OBLIGATION",
        "須": "OBLIGATION",
        "當": "OBLIGATION",
        "應": "OBLIGATION",
        "宜": "OBLIGATION",
        "可": "LIBERTY",
        "得": "LIBERTY",
        "許": "LIBERTY",
        "勿": "PROHIBITION",
        "莫": "PROHIBITION",
        "禁": "PROHIBITION",
        "不可": "PROHIBITION",
    }

    NEGATION = {"不", "非", "無", "未", "毋", "弗", "莫", "勿"}

    PREDICATES = {
        "殺": "HARM_PREVENTION",
        "害": "HARM_PREVENTION",
        "傷": "HARM_PREVENTION",
        "救": "CARE",
        "護": "CARE",
        "衛": "CARE",
        "愛": "CARE",
        "命": "AUTHORITY",
        "治": "AUTHORITY",
        "孝": "FAMILY",
        "悌": "FAMILY",
        "義": "FAIRNESS",
        "正": "FAIRNESS",
        "公": "FAIRNESS",
        "報": "RECIPROCITY",
        "還": "RECIPROCITY",
        "償": "RECIPROCITY",
        "仁": "CARE",
    }

    NOMINALS = {"君", "臣", "民", "人", "子", "父", "母", "者"}

    def extract(self, text: str) -> EnhancedBondResult:
        """Extract bond from Classical Chinese text."""
        result = EnhancedBondResult(method="chinese_positional")
        chars = list(text)

        # Find modal/deontic markers
        for modal, hohfeld in self.MODALS.items():
            if modal in text:
                result.modal = modal
                result.hohfeld_state = hohfeld
                result.context = "prescriptive"
                break

        # Check negation
        for neg in self.NEGATION:
            if neg in text:
                result.is_negated = True
                break

        # Check passive/causative markers
        is_passive = False
        for char in chars:
            if char in ("被", "見") and DETECT_PASSIVES:
                is_passive = True
                result.moral_features.append(MoralFeature("voice", "passive", char))
            if char in ("使", "令") and DETECT_CAUSATIVES:
                result.moral_features.append(MoralFeature("causation", "causative", char))

        # Find predicate and extract agent/patient
        predicate_idx = -1
        for i, char in enumerate(chars):
            if char in self.PREDICATES and char not in self.NOMINALS:
                predicate_idx = i
                result.bond_type = self.PREDICATES[char]
                break

        if predicate_idx >= 0 and EXTRACT_AGENT_PATIENT:
            # Agent: preverbal content
            preverbal = []
            for i in range(predicate_idx - 1, max(-1, predicate_idx - 4), -1):
                char = chars[i]
                if char in self.PARTICLES or char in self.NEGATION or char in self.MODALS:
                    continue
                preverbal.insert(0, char)
                if char in self.NOMINALS or len(preverbal) >= 2:
                    break

            if preverbal:
                agent_text = "".join(preverbal)
                if is_passive:
                    result.patient = ExtractedPatient(text=agent_text, position="preverbal")
                else:
                    result.agent = ExtractedAgent(text=agent_text, position="preverbal")

            # Patient: postverbal content
            postverbal = []
            for i in range(predicate_idx + 1, min(len(chars), predicate_idx + 4)):
                char = chars[i]
                if char in ("也", "矣", "焉", "乎", "哉"):
                    break
                if char in self.PARTICLES:
                    continue
                postverbal.append(char)
                if char in self.NOMINALS or len(postverbal) >= 2:
                    break

            if postverbal:
                patient_text = "".join(postverbal)
                if is_passive:
                    result.agent = ExtractedAgent(text=patient_text, position="postverbal")
                else:
                    result.patient = ExtractedPatient(text=patient_text, position="postverbal")

        # Calculate confidence
        result.confidence = 0.5
        if result.bond_type:
            result.confidence += 0.2
        if result.agent or result.patient:
            result.confidence += 0.1
        if result.modal:
            result.confidence += 0.1

        return result


class ArabicExtractor:
    """Arabic extraction with verb form (wazan) analysis."""

    MODALS = {
        "يجب": "OBLIGATION",
        "واجب": "OBLIGATION",
        "فرض": "OBLIGATION",
        "لازم": "OBLIGATION",
        "يجوز": "LIBERTY",
        "مباح": "LIBERTY",
        "حلال": "LIBERTY",
        "حرام": "NO_RIGHT",
        "محرم": "NO_RIGHT",
        "ممنوع": "NO_RIGHT",
    }

    NEGATION = {"لا", "ما", "ليس", "لم", "لن", "غير"}

    def extract(self, text: str) -> EnhancedBondResult:
        """Extract bond from Arabic text."""
        text = unicodedata.normalize("NFKC", text)
        text = re.sub(r"[\u064B-\u065F]", "", text)

        result = EnhancedBondResult(method="arabic_morphological")

        for modal, hohfeld in self.MODALS.items():
            if modal in text:
                result.modal = modal
                result.hohfeld_state = hohfeld
                result.context = "prescriptive"
                break

        for neg in self.NEGATION:
            if neg in text:
                result.is_negated = True
                break

        result.confidence = 0.5
        if result.modal:
            result.confidence += 0.2

        return result


class HebrewAramaicExtractor:
    """Hebrew/Aramaic extraction with modal markers."""

    MODALS = {
        "חייב": "OBLIGATION",
        "צריך": "OBLIGATION",
        "מוכרח": "OBLIGATION",
        "מצווה": "OBLIGATION",
        "מותר": "LIBERTY",
        "רשאי": "LIBERTY",
        "יכול": "LIBERTY",
        "אסור": "NO_RIGHT",
        "אי אפשר": "NO_RIGHT",
    }

    NEGATION = {"לא", "אל", "בלי", "אין", "מבלי"}

    def extract(self, text: str, language: str = "hebrew") -> EnhancedBondResult:
        """Extract bond from Hebrew/Aramaic text."""
        text = unicodedata.normalize("NFKC", text)
        text = re.sub(r"[\u0591-\u05C7]", "", text)

        result = EnhancedBondResult(method="hebrew_morphological")

        for modal, hohfeld in self.MODALS.items():
            if modal in text:
                result.modal = modal
                result.hohfeld_state = hohfeld
                result.context = "prescriptive"
                break

        words = text.split()
        for word in words:
            if word in self.NEGATION:
                result.is_negated = True
                break

        result.confidence = 0.5
        if result.modal:
            result.confidence += 0.2

        return result


class SanskritExtractor:
    """Sanskrit extraction using karaka theory."""

    MODALS = {
        "कर्तव्य": "OBLIGATION",
        "अवश्य": "OBLIGATION",
        "शक्य": "LIBERTY",
        "अनुज्ञा": "LIBERTY",
        "निषिद्ध": "NO_RIGHT",
        "वर्जित": "NO_RIGHT",
    }

    NEGATION = {"न", "मा"}

    def extract(self, text: str) -> EnhancedBondResult:
        """Extract bond from Sanskrit text."""
        text = unicodedata.normalize("NFC", text)
        result = EnhancedBondResult(method="sanskrit_karaka")

        for modal, hohfeld in self.MODALS.items():
            if modal in text:
                result.modal = modal
                result.hohfeld_state = hohfeld
                result.context = "prescriptive"
                break

        for neg in self.NEGATION:
            if neg in text:
                result.is_negated = True
                break

        # Extract agent/patient from case endings if enabled
        if EXTRACT_AGENT_PATIENT:
            words = text.split()
            for word in words:
                if word.endswith("ः") and not result.agent:
                    result.agent = ExtractedAgent(text=word, case_marking="prathamā")
                elif word.endswith("म्") and not result.patient:
                    result.patient = ExtractedPatient(text=word, case_marking="dvitīyā")

        result.confidence = 0.5
        if result.modal:
            result.confidence += 0.2
        if result.agent or result.patient:
            result.confidence += 0.1

        return result


# =============================================================================
# LEVEL 3: SPACY-BASED EXTRACTOR (English)
# =============================================================================

_spacy_nlp = None
_spacy_available = None


def _load_spacy():
    """Lazy-load spaCy model."""
    global _spacy_nlp, _spacy_available

    if _spacy_available is not None:
        return _spacy_nlp

    try:
        import spacy

        _spacy_nlp = spacy.load("en_core_web_md")
        _spacy_available = True
        print("  spaCy en_core_web_md loaded")
    except (ImportError, OSError):
        if INSTALL_SPACY_IF_NEEDED:
            print("  Installing spaCy...")
            import subprocess
            import sys

            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "spacy"])
            subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_md"])
            import spacy

            _spacy_nlp = spacy.load("en_core_web_md")
            _spacy_available = True
            print("  spaCy installed and loaded")
        else:
            _spacy_available = False
            print("  spaCy not available, using level2 for English")

    return _spacy_nlp


class SpacyEnglishExtractor:
    """English extraction using spaCy dependency parsing."""

    AGENT_DEPS = {"nsubj", "nsubjpass", "agent", "csubj"}
    PATIENT_DEPS = {"dobj", "obj", "iobj", "pobj", "dative", "nsubjpass"}

    MODAL_HOHFELD = {
        "must": "OBLIGATION",
        "shall": "OBLIGATION",
        "should": "OBLIGATION",
        "ought": "OBLIGATION",
        "may": "LIBERTY",
        "can": "LIBERTY",
        "could": "LIBERTY",
        "might": "LIBERTY",
    }

    NEGATION_WORDS = {"not", "n't", "never", "no", "none", "neither", "nor"}

    BOND_VERBS = {
        "harm": ["kill", "harm", "hurt", "injure", "damage", "destroy", "steal"],
        "care": ["help", "protect", "save", "care", "nurture", "support", "heal"],
        "authority": ["obey", "command", "order", "rule"],
        "family": ["honor", "respect"],
        "reciprocity": ["repay", "return", "owe", "borrow", "lend"],
        "contract": ["promise", "vow", "pledge"],
        "property": ["possess", "own", "steal"],
        "fairness": ["deserve", "merit"],
        "autonomy": ["choose", "consent"],
    }

    def extract(self, text: str) -> EnhancedBondResult:
        """Extract bond from English text using spaCy."""
        nlp = _load_spacy()
        if nlp is None:
            return EnhancedBondResult(method="spacy_unavailable", confidence=0.3)

        doc = nlp(text[:1000])
        result = EnhancedBondResult(method="spacy_dependency")

        # Extract agent/patient from dependencies
        if EXTRACT_AGENT_PATIENT:
            for token in doc:
                if token.pos_ == "VERB" or token.dep_ == "ROOT":
                    for child in token.children:
                        if child.dep_ in self.AGENT_DEPS and not result.agent:
                            agent_text = " ".join(t.text for t in child.subtree)
                            result.agent = ExtractedAgent(text=agent_text)
                        elif child.dep_ in self.PATIENT_DEPS and not result.patient:
                            patient_text = " ".join(t.text for t in child.subtree)
                            result.patient = ExtractedPatient(text=patient_text)
                    break

        # Extract modal and Hohfeld state
        for token in doc:
            if token.tag_ == "MD" or token.pos_ == "AUX":
                text_lower = token.text.lower()
                for modal, hohfeld in self.MODAL_HOHFELD.items():
                    if modal == text_lower:
                        result.modal = modal
                        result.hohfeld_state = hohfeld
                        result.context = "prescriptive"

                        # Check negation scope
                        for child in token.children:
                            if child.dep_ == "neg" or child.text.lower() in self.NEGATION_WORDS:
                                result.is_negated = True
                                if hohfeld == "OBLIGATION":
                                    result.hohfeld_state = "NO_RIGHT"
                        break

        # Detect bond type from verbs
        for token in doc:
            if token.pos_ == "VERB":
                lemma = token.lemma_.lower()
                for bond_type, verbs in self.BOND_VERBS.items():
                    if lemma in verbs:
                        result.bond_type = bond_type.upper()
                        if bond_type == "harm":
                            result.bond_type = "HARM_PREVENTION"
                        break

        # Confidence
        result.confidence = 0.5
        if result.bond_type:
            result.confidence += 0.2
        if result.agent:
            result.confidence += 0.1
        if result.patient:
            result.confidence += 0.1
        if result.modal:
            result.confidence += 0.1
        result.confidence = min(result.confidence, 0.95)

        return result


# =============================================================================
# UNIFIED EXTRACTOR
# =============================================================================

_chinese_extractor = ClassicalChineseExtractor()
_arabic_extractor = ArabicExtractor()
_hebrew_extractor = HebrewAramaicExtractor()
_sanskrit_extractor = SanskritExtractor()
_spacy_extractor = SpacyEnglishExtractor()


def extract_bond_level2(text: str, language: str) -> EnhancedBondResult:
    """Level 2: Language-specific grammar-aware extraction."""
    if language == "classical_chinese":
        return _chinese_extractor.extract(text)
    elif language == "arabic":
        return _arabic_extractor.extract(text)
    elif language in ("hebrew", "aramaic"):
        return _hebrew_extractor.extract(text, language)
    elif language in ("sanskrit", "pali"):
        return _sanskrit_extractor.extract(text)
    else:
        # Fallback to regex for other languages
        result = enhanced_extract_bond_regex(text, language)
        return EnhancedBondResult(
            bond_type=result.get("bond_type"),
            hohfeld_state=result.get("hohfeld_state", "OBLIGATION"),
            is_negated=result.get("negated", False),
            modal=result.get("modal"),
            confidence=result.get("confidence", 0.5),
            context=result.get("context", "unknown"),
            method="regex_fallback",
        )


def extract_bond_level3(text: str, language: str) -> EnhancedBondResult:
    """Level 3: spaCy for English, level2 for others."""
    if language == "english" and USE_SPACY_FOR_ENGLISH:
        return _spacy_extractor.extract(text)
    else:
        return extract_bond_level2(text, language)


def unified_extract_bond(text: str, language: str) -> dict:
    """
    Unified bond extraction based on EXTRACTION_LEVEL setting.
    Returns dict compatible with existing code.

    v10.16.1: Falls back to level1 regex if level2/3 doesn't find a bond_type.
    This ensures we get meaningful bond labels for training.
    """
    if EXTRACTION_LEVEL == "level1":
        return enhanced_extract_bond_regex(text, language)
    elif EXTRACTION_LEVEL == "level2":
        result = extract_bond_level2(text, language)
        d = result.to_dict()
        # Fallback: if level2 didn't find a bond_type, try level1 regex
        if d.get("bond_type") is None:
            regex_result = enhanced_extract_bond_regex(text, language)
            if regex_result.get("bond_type"):
                d["bond_type"] = regex_result["bond_type"]
                d["method"] = d.get("method", "unknown") + "+regex_fallback"
        return d
    elif EXTRACTION_LEVEL == "level3":
        result = extract_bond_level3(text, language)
        d = result.to_dict()
        # Fallback: if level3 didn't find a bond_type, try level1 regex
        if d.get("bond_type") is None:
            regex_result = enhanced_extract_bond_regex(text, language)
            if regex_result.get("bond_type"):
                d["bond_type"] = regex_result["bond_type"]
                d["method"] = d.get("method", "unknown") + "+regex_fallback"
        return d
    else:
        return enhanced_extract_bond_regex(text, language)


print(f"\nExtraction level: {EXTRACTION_LEVEL}")
if EXTRACTION_LEVEL == "level2":
    print("  Chinese: position + particles")
    print("  Arabic: verb form (wazan) analysis")
    print("  Hebrew: modal markers + binyan")
    print("  Sanskrit: karaka (case) analysis")
elif EXTRACTION_LEVEL == "level3":
    print("  English: spaCy dependency parsing")
    print("  Others: level2 grammar analysis")


def enhanced_extract_bond_regex(text: str, language: str) -> dict:
    """Enhanced bond extraction with negation + modal detection."""
    normalized = normalize_text(text, language)

    negation_cues = NEGATION_CUES.get(language, [])
    is_negated = any(cue in normalized for cue in negation_cues)

    modal_status = "unknown"
    modal_text = None
    for status, markers in MODAL_CLASSIFICATION.get(language, {}).items():
        for marker in markers:
            if marker in normalized:
                modal_status = status
                modal_text = marker
                break
        if modal_status != "unknown":
            break

    hohfeld_map = {
        "obligation": "OBLIGATION",
        "permission": "LIBERTY",
        "prohibition": "NO_RIGHT",
        "supererogation": "LIBERTY",
        "unknown": "OBLIGATION",
    }
    hohfeld = hohfeld_map[modal_status]

    bond_type = None
    confidence = 0.5
    for bt, patterns in ALL_BOND_PATTERNS.get(language, {}).items():
        for pattern in patterns:
            if re.search(pattern, normalized):
                bond_type = bt
                confidence = 0.9
                break
        if bond_type:
            break

    if is_negated:
        confidence *= 0.8

    if modal_status in ["obligation", "prohibition"]:
        context = "prescriptive"
    elif modal_status == "permission":
        context = "descriptive"
    else:
        context = "unknown"

    return {
        "bond_type": bond_type.name if bond_type else None,
        "hohfeld_state": hohfeld,
        "negated": is_negated,
        "modal": modal_text,
        "confidence": confidence,
        "context": context,
    }


print("\nContext markers defined for grammar-aware extraction")
print("  Detects: negation, obligation, prohibition, permission")

print(f"\nPatterns defined for {len(ALL_BOND_PATTERNS)} languages:")
for lang in ALL_BOND_PATTERNS:
    n = sum(len(p) for p in ALL_BOND_PATTERNS[lang].values())
    print(f"  {lang}: {n} bond patterns")

print("\nNLP improvements (Phase 1):")
print(f"  NEGATION_CUES: {len(NEGATION_CUES)} languages")
print(f"  MODAL_CLASSIFICATION: {len(MODAL_CLASSIFICATION)} languages")
print("  enhanced_extract_bond_regex() ready (level1)")

print("\n" + "=" * 60)

# ============================================================================
# EXTRACT BONDS FROM PASSAGES
# ============================================================================

print("EXTRACTING BONDS FROM PASSAGES")
print("=" * 60)

import json
from pathlib import Path

passages_file = Path("data/processed/passages.jsonl")
bonds_file = Path("data/processed/bonds.jsonl")

if bonds_file.exists():
    with open(bonds_file, encoding="utf-8") as f:
        bond_count = sum(1 for _ in f)
    print(f"  bonds.jsonl exists with {bond_count:,} bonds (cached)")
else:
    passages = []
    with open(passages_file, encoding="utf-8") as f:
        for line in f:
            passages.append(json.loads(line))

    print(f"  Processing {len(passages):,} passages...")

    bonds = []
    bond_type_counts = {}

    for i, p in enumerate(passages):
        try:
            text = normalize_text(p["text"], p["language"])
            bond = unified_extract_bond(text, p["language"])
            bond["passage_id"] = p["id"]
            bonds.append(bond)

            bt = bond.get("bond_type") or "NEUTRAL"
            bond_type_counts[bt] = bond_type_counts.get(bt, 0) + 1

        except Exception:
            bonds.append(
                {
                    "passage_id": p["id"],
                    "bond_type": "NEUTRAL",
                    "hohfeld_state": "LIBERTY",
                    "negated": False,
                    "modal": None,
                    "confidence": 0.1,
                    "context": "unknown",
                }
            )

        if (i + 1) % 20000 == 0:
            print(f"    {i + 1:,} processed...")

    bonds_file.parent.mkdir(parents=True, exist_ok=True)
    with open(bonds_file, "w", encoding="utf-8") as f:
        for b in bonds:
            f.write(json.dumps(b, ensure_ascii=False) + "\n")

    print(f"\nSaved {len(bonds):,} bonds to {bonds_file}")
    print("\nBond type distribution:")
    for bt, count in sorted(bond_type_counts.items(), key=lambda x: -x[1]):
        pct = 100 * count / len(bonds)
        print(f"    {bt:12s}: {count:6,} ({pct:.1f}%)")

print("\n" + "=" * 60)
print("BOND EXTRACTION COMPLETE")
print("=" * 60)

# =============================================================================
# MERGE ETHICS CORPUS (v10.16)
# =============================================================================
# Integrate labeled ethics datasets from Cell 3 into main corpus

ethics_corpus_file = Path("data/bond_training/ethics_corpus.jsonl")

if ethics_corpus_file.exists():
    print("\n" + "=" * 60)
    print("MERGING ETHICS CORPUS")
    print("=" * 60)

    # Load existing passages and bonds
    existing_passage_ids = set()
    with open("data/processed/passages.jsonl", encoding="utf-8") as f:
        for line in f:
            p = json.loads(line)
            existing_passage_ids.add(p["id"])

    existing_bond_ids = set()
    with open("data/processed/bonds.jsonl", encoding="utf-8") as f:
        for line in f:
            b = json.loads(line)
            existing_bond_ids.add(b["passage_id"])

    print(f"  Existing passages: {len(existing_passage_ids):,}")
    print(f"  Existing bonds: {len(existing_bond_ids):,}")

    # Load ethics corpus
    ethics_passages = []
    ethics_bonds = []

    with open(ethics_corpus_file, encoding="utf-8") as f:
        for line in f:
            p = json.loads(line)

            # Skip if already exists
            if p["id"] in existing_passage_ids:
                continue

            # Extract bond info from the passage
            bond_info = p.get("bonds", [{}])[0]

            # Map ethics bond types to BIP bond types
            ethics_to_bip = {
                "OBLIGATION": "AUTHORITY",  # Deontological duty
                "PROHIBITION": "HARM_PREVENTION",  # Don't do X
                "PERMISSION": "AUTONOMY",  # May do X
                "CLAIM": "FAIRNESS",  # Has right to X
                "VIRTUE": "CARE",  # Character-based
                "VICE": "HARM_PREVENTION",  # Negative trait
                "SUPEREROGATORY": "CARE",  # Beyond duty
                "DUTY": "AUTHORITY",  # Hohfeld duty
                "LIBERTY": "AUTONOMY",  # Hohfeld liberty
            }

            raw_bond = bond_info.get("bond_type", "OBLIGATION")
            mapped_bond = ethics_to_bip.get(raw_bond, "AUTHORITY")

            # Create passage entry
            ethics_passages.append(
                {
                    "id": p["id"],
                    "text": p["text"],
                    "language": "english",
                    "time_periods": p.get("time_periods", ["MODERN_ETHICS"]),
                    "tags": p.get("tags", ["modern", "english", "ethics"]),
                }
            )

            # Create bond entry with high confidence (labeled data!)
            ethics_bonds.append(
                {
                    "passage_id": p["id"],
                    "bond_type": mapped_bond,
                    "hohfeld_state": bond_info.get("hohfeld_state", "OBLIGATION"),
                    "negated": False,
                    "modal": None,
                    "confidence": bond_info.get("confidence", 0.8),  # High confidence - labeled!
                    "context": bond_info.get("context", "prescriptive"),
                }
            )

    # Append to passages.jsonl
    with open("data/processed/passages.jsonl", "a", encoding="utf-8") as f:
        for p in ethics_passages:
            f.write(json.dumps(p, ensure_ascii=False) + "\n")

    # Append to bonds.jsonl
    with open("data/processed/bonds.jsonl", "a", encoding="utf-8") as f:
        for b in ethics_bonds:
            f.write(json.dumps(b, ensure_ascii=False) + "\n")

    print(f"  Added {len(ethics_passages):,} ethics passages")
    print(f"  Added {len(ethics_bonds):,} ethics bonds (labeled, high confidence)")

    # Show bond type distribution of added data
    from collections import Counter

    bond_dist = Counter(b["bond_type"] for b in ethics_bonds)
    print("\n  Ethics bond distribution:")
    for bt, count in bond_dist.most_common():
        print(f"    {bt}: {count:,}")

    print("\n  Ethics corpus merged successfully!")
else:
    print("\n[Note] No ethics corpus found - run Cell 3 to generate ethics_corpus.jsonl")

print("\n" + "=" * 60)
print("CORPUS INTEGRATION COMPLETE")
print("=" * 60)


TEXT NORMALIZATION & PATTERNS

Extraction level: level2
  Chinese: position + particles
  Arabic: verb form (wazan) analysis
  Hebrew: modal markers + binyan
  Sanskrit: karaka (case) analysis

Context markers defined for grammar-aware extraction
  Detects: negation, obligation, prohibition, permission

Patterns defined for 7 languages:
  hebrew: 52 bond patterns
  aramaic: 36 bond patterns
  classical_chinese: 55 bond patterns
  arabic: 54 bond patterns
  english: 53 bond patterns
  sanskrit: 39 bond patterns
  pali: 31 bond patterns

NLP improvements (Phase 1):
  NEGATION_CUES: 7 languages
  MODAL_CLASSIFICATION: 6 languages
  enhanced_extract_bond_regex() ready (level1)

EXTRACTING BONDS FROM PASSAGES
  Processing 109,769 passages...
    20,000 processed...
    40,000 processed...
    60,000 processed...
    80,000 processed...
    100,000 processed...

Saved 109,769 bonds to data/processed/bonds.jsonl

Bond type distribution:
    NEUTRAL     : 59,902 (54.6%)
    FAMILY      : 13,98

In [ ]:
# @title 5. Generate Splits { display-mode: "form" }
# @markdown v10.13: Tag-based splits with matrix selection

# @markdown ---
# @markdown ## Split Matrix
# @markdown Select train/test tags using dropdowns. Use "none" to disable.

# @markdown ### Experiment 1
EXP1_ENABLE = True  # @param {type:"boolean"}
EXP1_NAME = "hebrew_to_others"  # @param {type:"string"}
EXP1_TRAIN = "hebrew"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP1_TEST = "all-other"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 2
EXP2_ENABLE = True  # @param {type:"boolean"}
EXP2_NAME = "semitic_to_indic"  # @param {type:"string"}
EXP2_TRAIN = "semitic"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP2_TEST = "indic"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 3
EXP3_ENABLE = True  # @param {type:"boolean"}
EXP3_NAME = "confucian_to_buddhist"  # @param {type:"string"}
EXP3_TRAIN = "confucian"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP3_TEST = "buddhist"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 4
EXP4_ENABLE = True  # @param {type:"boolean"}
EXP4_NAME = "ancient_to_modern"  # @param {type:"string"}
EXP4_TRAIN = "ancient"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP4_TEST = "modern"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 5
EXP5_ENABLE = True  # @param {type:"boolean"}
EXP5_NAME = "east_to_west"  # @param {type:"string"}
EXP5_TRAIN = "east-asia"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP5_TEST = "western"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 6
EXP6_ENABLE = True  # @param {type:"boolean"}
EXP6_NAME = "semitic_to_chinese"  # @param {type:"string"}
EXP6_TRAIN = "semitic"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP6_TEST = "chinese"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 7
EXP7_ENABLE = True  # @param {type:"boolean"}
EXP7_NAME = "jewish_to_islamic"  # @param {type:"string"}
EXP7_TRAIN = "hebrew"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP7_TEST = "arabic"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 8
EXP8_ENABLE = True  # @param {type:"boolean"}
EXP8_NAME = "stoic_to_confucian"  # @param {type:"string"}
EXP8_TRAIN = "stoic"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP8_TEST = "confucian"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 9
EXP9_ENABLE = True  # @param {type:"boolean"}
EXP9_NAME = "daoist_to_buddhist"  # @param {type:"string"}
EXP9_TRAIN = "daoist"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP9_TEST = "buddhist"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 10
EXP10_ENABLE = True  # @param {type:"boolean"}
EXP10_NAME = "hindu_to_buddhist"  # @param {type:"string"}
EXP10_TRAIN = "hindu"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP10_TEST = "buddhist"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 11
EXP11_ENABLE = False  # @param {type:"boolean"}
EXP11_NAME = "custom_11"  # @param {type:"string"}
EXP11_TRAIN = "none"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP11_TEST = "none"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ### Experiment 12
EXP12_ENABLE = False  # @param {type:"boolean"}
EXP12_NAME = "custom_12"  # @param {type:"string"}
EXP12_TRAIN = "none"  # @param ["none", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]
EXP12_TEST = "none"  # @param ["none", "all-other", "hebrew", "aramaic", "arabic", "semitic", "chinese", "confucian", "daoist", "buddhist", "hindu", "sanskrit", "pali", "indic", "greek", "latin", "stoic", "english", "western", "modern", "ancient", "classical", "east-asia", "south-asia", "middle-east"]

# @markdown ---
# @markdown ## Options
INCLUDE_MIXED_BASELINE = True  # @param {type:"boolean"}
MIN_SPLIT_SIZE = 50  # @param {type:"integer"}

import json
import random
from collections import defaultdict
from pathlib import Path

print("=" * 60)
print("GENERATING SPLITS (v10.13)")
print("=" * 60)

# =============================================================================
# TAG DEFINITIONS
# =============================================================================

# Compound tag groups
TAG_GROUPS = {
    "semitic": ["hebrew", "aramaic", "arabic"],
    "indic": ["sanskrit", "pali", "hindi"],
    "east-asia": ["chinese", "confucian", "daoist"],
    "south-asia": ["sanskrit", "pali", "hindu", "buddhist"],
    "middle-east": ["hebrew", "aramaic", "arabic", "jewish", "islamic"],
    "western": ["english", "greek", "latin", "stoic"],
    "ancient": ["ancient", "classical"],
    "modern": ["modern", "advice", "american"],
}

# Period to tags mapping
PERIOD_TO_TAGS = {
    "CONFUCIAN": ["confucian", "east-asia", "classical", "chinese"],
    "DAOIST": ["daoist", "east-asia", "classical", "chinese"],
    "BUDDHIST": ["buddhist"],
    "PALI": ["buddhist", "south-asia", "ancient", "pali"],
    "DHARMA": ["hindu", "south-asia", "ancient", "sanskrit"],
    "BIBLICAL": ["jewish", "middle-east", "ancient", "hebrew"],
    "TANNAITIC": ["jewish", "middle-east", "classical", "hebrew"],
    "AMORAIC": ["jewish", "middle-east", "classical", "aramaic"],
    "QURANIC": ["islamic", "middle-east", "medieval", "arabic"],
    "HADITH": ["islamic", "middle-east", "medieval", "arabic"],
    "CLASSICAL_GREEK": ["stoic", "mediterranean", "classical", "greek"],
    "HELLENISTIC": ["stoic", "mediterranean", "classical", "greek"],  # Epictetus, Marcus Aurelius
    "CLASSICAL_LATIN": ["stoic", "mediterranean", "classical", "latin"],  # Seneca, Cicero
    "DEAR_ABBY": ["american", "modern", "advice", "english", "western"],
    "MODERN_ETHICS": ["western", "modern", "ethics", "english"],
}

LANG_TO_TAGS = {
    "classical_chinese": ["chinese", "east-asia"],
    "hebrew": ["hebrew", "middle-east"],
    "aramaic": ["aramaic", "middle-east"],
    "arabic": ["arabic", "middle-east"],
    "sanskrit": ["sanskrit", "south-asia"],
    "pali": ["pali", "south-asia"],
    "greek": ["greek", "mediterranean"],
    "latin": ["latin", "mediterranean"],
    "english": ["english", "western"],
}


def add_tags(p: dict) -> list:
    """Generate tags for a passage."""
    tags = set()

    lang = p.get("language", "")
    if lang in LANG_TO_TAGS:
        tags.update(LANG_TO_TAGS[lang])

    for period in p.get("time_periods", []):
        if period in PERIOD_TO_TAGS:
            tags.update(PERIOD_TO_TAGS[period])

    return sorted(tags)


# =============================================================================
# LOAD PASSAGES
# =============================================================================

passages_file = Path("data/processed/passages.jsonl")
if not passages_file.exists():
    raise FileNotFoundError("Run Cell 2 first to generate passages.jsonl")

passage_meta = []
with open(passages_file, encoding="utf-8") as f:
    for line in f:
        p = json.loads(line)
        passage_meta.append(
            {
                "id": p["id"],
                "language": p.get("language", ""),
                "tags": add_tags(p),
            }
        )

print(f"Loaded {len(passage_meta):,} passages")

# Count tags
tag_counts = defaultdict(int)
for p in passage_meta:
    for tag in p["tags"]:
        tag_counts[tag] += 1

print("\nTag counts:")
for tag, count in sorted(tag_counts.items(), key=lambda x: -x[1])[:15]:
    print(f"  {tag}: {count:,}")


# =============================================================================
# SPLIT HELPERS
# =============================================================================


def expand_tag(tag: str) -> list:
    """Expand compound tags like 'semitic' to individual tags."""
    if tag in TAG_GROUPS:
        return TAG_GROUPS[tag]
    return [tag]


def ids_with_tags(tags: list) -> list:
    """Get passage IDs with ANY of the tags."""
    tag_set = set()
    for t in tags:
        tag_set.update(expand_tag(t))
    return [p["id"] for p in passage_meta if set(p["tags"]) & tag_set]


def ids_without_tags(tags: list) -> list:
    """Get passage IDs with NONE of the tags."""
    tag_set = set()
    for t in tags:
        tag_set.update(expand_tag(t))
    return [p["id"] for p in passage_meta if not (set(p["tags"]) & tag_set)]


# =============================================================================
# GENERATE SPLITS FROM MATRIX
# =============================================================================

print("\n" + "=" * 60)
print("GENERATING SPLITS")
print("=" * 60)

all_splits = {}
random.seed(42)

experiments = [
    (EXP1_ENABLE, EXP1_NAME, EXP1_TRAIN, EXP1_TEST),
    (EXP2_ENABLE, EXP2_NAME, EXP2_TRAIN, EXP2_TEST),
    (EXP3_ENABLE, EXP3_NAME, EXP3_TRAIN, EXP3_TEST),
    (EXP4_ENABLE, EXP4_NAME, EXP4_TRAIN, EXP4_TEST),
    (EXP5_ENABLE, EXP5_NAME, EXP5_TRAIN, EXP5_TEST),
    (EXP6_ENABLE, EXP6_NAME, EXP6_TRAIN, EXP6_TEST),
    (EXP7_ENABLE, EXP7_NAME, EXP7_TRAIN, EXP7_TEST),
    (EXP8_ENABLE, EXP8_NAME, EXP8_TRAIN, EXP8_TEST),
    (EXP9_ENABLE, EXP9_NAME, EXP9_TRAIN, EXP9_TEST),
    (EXP10_ENABLE, EXP10_NAME, EXP10_TRAIN, EXP10_TEST),
    (EXP11_ENABLE, EXP11_NAME, EXP11_TRAIN, EXP11_TEST),
    (EXP12_ENABLE, EXP12_NAME, EXP12_TRAIN, EXP12_TEST),
]

for enabled, name, train_tag, test_tag in experiments:
    if not enabled or train_tag == "none" or not name.strip():
        continue

    name = name.strip().replace(" ", "_")

    # Get train IDs
    train_ids = ids_with_tags([train_tag])

    # Get test IDs
    if test_tag == "all-other":
        test_ids = ids_without_tags([train_tag])
    elif test_tag == "none":
        continue
    else:
        test_ids = ids_with_tags([test_tag])
        # Remove overlap
        overlap = set(train_ids) & set(test_ids)
        train_ids = [x for x in train_ids if x not in overlap]
        test_ids = [x for x in test_ids if x not in overlap]

    if len(train_ids) < MIN_SPLIT_SIZE or len(test_ids) < MIN_SPLIT_SIZE:
        print(f"  SKIP {name}: insufficient data (train={len(train_ids)}, test={len(test_ids)})")
        continue

    random.shuffle(train_ids)
    random.shuffle(test_ids)

    all_splits[name] = {
        "train_ids": train_ids,
        "test_ids": test_ids,
        "train_size": len(train_ids),
        "test_size": len(test_ids),
        "train_tags": expand_tag(train_tag),
        "test_tags": expand_tag(test_tag) if test_tag != "all-other" else ["*"],
    }
    print(f"  {name}: {len(train_ids):,} -> {len(test_ids):,}")

# Add mixed baseline
if INCLUDE_MIXED_BASELINE:
    all_ids = [p["id"] for p in passage_meta]
    random.shuffle(all_ids)
    split_pt = int(len(all_ids) * 0.7)
    all_splits["mixed_baseline"] = {
        "train_ids": all_ids[:split_pt],
        "test_ids": all_ids[split_pt:],
        "train_size": split_pt,
        "test_size": len(all_ids) - split_pt,
        "train_tags": ["*"],
        "test_tags": ["*"],
    }
    print(f"  mixed_baseline: {split_pt:,} -> {len(all_ids) - split_pt:,}")


# =============================================================================
# SAVE
# =============================================================================

splits_file = Path("data/splits/all_splits.json")
splits_file.parent.mkdir(parents=True, exist_ok=True)

with open(splits_file, "w", encoding="utf-8") as f:
    json.dump(all_splits, f, indent=2, ensure_ascii=False)

print("\n" + "=" * 60)
print(f"SAVED {len(all_splits)} SPLITS")
print("=" * 60)

print("\n" + "-" * 50)
print(f"{'Experiment':<25} {'Train':>10} {'Test':>10}")
print("-" * 50)
for name, split in sorted(all_splits.items()):
    print(f"{name:<25} {split['train_size']:>10,} {split['test_size']:>10,}")
print("-" * 50)

GENERATING SPLITS (v10.13)
Loaded 227,859 passages

Tag counts:
  english: 168,090
  western: 168,090
  modern: 164,718
  ethics: 144,688
  ancient: 31,162
  south-asia: 25,000
  advice: 20,030
  american: 20,030
  middle-east: 16,235
  hindu: 15,000
  sanskrit: 15,000
  buddhist: 13,277
  classical: 12,722
  pali: 10,000
  jewish: 10,000

GENERATING SPLITS
  hebrew_to_others: 7,985 -> 219,874
  semitic_to_indic: 16,235 -> 25,000
  confucian_to_buddhist: 1,141 -> 13,277
  ancient_to_modern: 43,884 -> 164,718
  east_to_west: 4,924 -> 172,380
  semitic_to_chinese: 16,235 -> 4,924
  jewish_to_islamic: 7,985 -> 6,235
  stoic_to_confucian: 7,662 -> 1,141
  daoist_to_buddhist: 81 -> 13,277
  hindu_to_buddhist: 15,000 -> 13,277
  mixed_baseline: 159,501 -> 68,358

SAVED 11 SPLITS

--------------------------------------------------
Experiment                     Train       Test
--------------------------------------------------
ancient_to_modern             43,884    164,718
confucian_to_budd

In [ ]:
# @title 6. Model Architecture { display-mode: "form" }
# @markdown BIP v10.9 model with configurable backbone and adversarial heads
# @markdown - Updated: 8 languages, 26 periods

import json

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from transformers import AutoModel, AutoTokenizer

print("=" * 60)
print("MODEL ARCHITECTURE")
print("=" * 60)
print(f"Backbone: {BACKBONE} ({MODEL_NAME})")
print(f"Hidden size: {BACKBONE_HIDDEN}")

# Index mappings
BOND_TO_IDX = {bt.name: i for i, bt in enumerate(BondType)}
IDX_TO_BOND = {i: bt.name for i, bt in enumerate(BondType)}
# v10.9: 8 languages (added Sanskrit, Pali, Greek placeholder)
LANG_TO_IDX = {
    "hebrew": 0,
    "aramaic": 1,
    "classical_chinese": 2,
    "arabic": 3,
    "english": 4,
    "sanskrit": 5,  # NEW in v10.9
    "pali": 6,  # NEW in v10.9
    "greek": 7,  # FUTURE (placeholder)
}
IDX_TO_LANG = {i: l for l, i in LANG_TO_IDX.items()}

# v10.9: 26 periods (expanded Chinese, Arabic, added Sanskrit/Pali traditions)
PERIOD_TO_IDX = {
    # Semitic traditions
    "BIBLICAL": 0,
    "TANNAITIC": 1,
    "AMORAIC": 2,
    "RISHONIM": 3,
    "ACHRONIM": 4,
    # Chinese traditions (expanded)
    "CONFUCIAN": 5,
    "DAOIST": 6,
    "MOHIST": 7,  # NEW in v10.9
    "LEGALIST": 8,  # NEW in v10.9
    "BUDDHIST": 9,  # NEW in v10.9 (Chinese Buddhism)
    "NEO_CONFUCIAN": 10,  # NEW in v10.9
    # Arabic/Islamic traditions (expanded)
    "QURANIC": 11,
    "HADITH": 12,
    "FIQH": 13,  # NEW in v10.9 (Islamic jurisprudence)
    "SUFI": 14,  # NEW in v10.9
    "FALSAFA": 15,  # NEW in v10.9 (Arabic philosophy)
    # Sanskrit/Pali traditions (NEW in v10.9)
    "DHARMA": 16,  # Dharmashastra
    "UPANISHAD": 17,
    "GITA": 18,
    "ARTHA": 19,  # Arthashastra
    "PALI": 20,  # Pali Canon
    # Western traditions
    "WESTERN_CLASSICAL": 21,
    "MEDIEVAL": 22,
    # Modern
    "DEAR_ABBY": 23,
    "MODERN": 24,
    "CLASSICAL": 25,  # Generic classical (fallback)
}  # 26 periods total (0-25)
IDX_TO_PERIOD = {i: p for p, i in PERIOD_TO_IDX.items()}
HOHFELD_TO_IDX = {hs.name: i for i, hs in enumerate(HohfeldState)}
IDX_TO_HOHFELD = {i: hs.name for i, hs in enumerate(HohfeldState)}
CONTEXT_TO_IDX = {"prescriptive": 0, "descriptive": 1, "unknown": 2}
IDX_TO_CONTEXT = {i: c for c, i in CONTEXT_TO_IDX.items()}


def get_confidence_weight(conf):
    """Map confidence to sample weight. Handles both string ('high'/'medium'/'low') and numeric (0.0-1.0) values."""
    if isinstance(conf, str):
        return {"high": 2.0, "medium": 1.0, "low": 0.5}.get(conf, 1.0)
    elif isinstance(conf, (int, float)):
        return 2.0 if conf >= 0.8 else 1.0
    return 1.0


class GradientReversalLayer(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.alpha, None


class BIPModel(nn.Module):
    def __init__(self, model_name=None, hidden_size=None, z_dim=64):
        super().__init__()
        # Use global config if not specified
        model_name = model_name or MODEL_NAME
        hidden_size = hidden_size or BACKBONE_HIDDEN

        print(f"  Loading encoder: {model_name}")
        self.encoder = AutoModel.from_pretrained(model_name)

        # Freeze encoder if configured (probe-only training)
        try:
            if FREEZE_ENCODER:
                for param in self.encoder.parameters():
                    param.requires_grad = False
                print("  Encoder FROZEN (probe-only mode)")
            else:
                print("  Encoder UNFROZEN (full fine-tuning)")
        except NameError:
            print("  Encoder unfrozen (FREEZE_ENCODER not set)")

        # Get actual hidden size from model config
        actual_hidden = self.encoder.config.hidden_size
        if actual_hidden != hidden_size:
            print(f"  Note: Using actual hidden size {actual_hidden}")
            hidden_size = actual_hidden

        self.hidden_size = hidden_size
        self.model_name = model_name

        # Projection to z_bond space (scales with backbone size)
        proj_hidden = min(512, hidden_size)

        # v10.16.8: Variational Information Bottleneck (VIB)
        # Instead of deterministic z, output distribution parameters
        try:
            self.use_vib = USE_VIB
        except NameError:
            self.use_vib = False

        if self.use_vib:
            # Shared encoder layers
            self.z_encoder = nn.Sequential(
                nn.Linear(hidden_size, proj_hidden),
                nn.LayerNorm(proj_hidden),
                nn.GELU(),
                nn.Dropout(0.1),
            )
            # Output mean and log-variance
            self.z_mu = nn.Linear(proj_hidden, z_dim)
            self.z_logvar = nn.Linear(proj_hidden, z_dim)
            print(f"  VIB: Stochastic z with KL regularization")

            # Keep z_proj for backward compatibility (uses mean)
            self.z_proj = nn.Sequential(
                nn.Linear(hidden_size, proj_hidden),
                nn.LayerNorm(proj_hidden),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(proj_hidden, z_dim),
            )
        else:
            self.z_proj = nn.Sequential(
                nn.Linear(hidden_size, proj_hidden),
                nn.LayerNorm(proj_hidden),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(proj_hidden, z_dim),
            )

        # Task heads
        self.bond_head = nn.Linear(z_dim, len(BondType))
        self.hohfeld_head = nn.Linear(z_dim, len(HohfeldState))

        # Adversarial heads
        # v10.16.4: Dynamic layer count + LayerNorm for stronger disentanglement
        try:
            _adv_hidden = ADV_HIDDEN_DIM
            _adv_dropout = ADV_DROPOUT
            _adv_layers = ADV_NUM_LAYERS
        except NameError:
            _adv_hidden = 512
            _adv_dropout = 0.3
            _adv_layers = 3

        def build_adversarial_head(input_dim, output_dim, hidden_dim, num_layers, dropout):
            """Build adversarial head with configurable depth and LayerNorm."""
            layers = []
            # Input layer
            layers.extend([
                nn.Linear(input_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
            ])
            # Hidden layers
            for _ in range(num_layers - 2):
                layers.extend([
                    nn.Linear(hidden_dim, hidden_dim),
                    nn.LayerNorm(hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(dropout),
                ])
            # Output layer
            layers.append(nn.Linear(hidden_dim, output_dim))
            return nn.Sequential(*layers)

        # v10.16.7: Multiple adversarial heads with different architectures
        # This prevents "adversarial hiding" where encoder fools one head but not others
        try:
            _num_adv_heads = NUM_ADV_HEADS
        except NameError:
            _num_adv_heads = 4  # Default

        # Create diverse head architectures
        self.language_heads = nn.ModuleList()
        self.period_heads = nn.ModuleList()

        # Head configs: (hidden_dim_scale, num_layers_offset, dropout_offset)
        head_configs = [
            (1.0, 0, 0.0),      # Standard: 1024, 4 layers, 0.4 dropout
            (0.5, 1, 0.1),      # Narrow+deep: 512, 5 layers, 0.5 dropout
            (1.5, -1, -0.1),    # Wide+shallow: 1536, 3 layers, 0.3 dropout
            (0.75, 0, 0.15),    # Medium: 768, 4 layers, 0.55 dropout
            (1.25, 1, -0.05),   # Wide+deep: 1280, 5 layers, 0.35 dropout
        ]

        for i in range(_num_adv_heads):
            cfg = head_configs[i % len(head_configs)]
            h_dim = max(256, int(_adv_hidden * cfg[0]))
            n_layers = max(2, _adv_layers + cfg[1])
            drop = max(0.1, min(0.6, _adv_dropout + cfg[2]))

            self.language_heads.append(
                build_adversarial_head(z_dim, len(LANG_TO_IDX), h_dim, n_layers, drop)
            )
            self.period_heads.append(
                build_adversarial_head(z_dim, len(PERIOD_TO_IDX), h_dim, n_layers, drop)
            )

        print(f"  Adversarial heads: {_num_adv_heads} independent heads (v10.16.7 multi-head)")
        print(f"    Base config: {_adv_layers} layers, {_adv_hidden} hidden, {_adv_dropout} dropout")

        # Keep single head references for backward compatibility (use first head)
        self.language_head = self.language_heads[0]
        self.period_head = self.period_heads[0]

        # Context prediction head (auxiliary task)
        self.context_head = nn.Linear(z_dim, len(CONTEXT_TO_IDX))

        # Count parameters
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f"  Total params: {total_params:,}")
        print(f"  Trainable: {trainable_params:,}")

    def forward(self, input_ids, attention_mask, adv_lambda=1.0):
        enc = self.encoder(input_ids, attention_mask)

        # Handle different pooling strategies
        if hasattr(enc, "pooler_output") and enc.pooler_output is not None:
            pooled = enc.pooler_output
        else:
            pooled = enc.last_hidden_state[:, 0]

        # v10.16.8: VIB - sample z from learned distribution
        z_mu = None
        z_logvar = None
        if self.use_vib:
            h = self.z_encoder(pooled)
            z_mu = self.z_mu(h)
            z_logvar = self.z_logvar(h)

            # Reparameterization trick: z = mu + sigma * epsilon
            if self.training:
                std = torch.exp(0.5 * z_logvar)
                eps = torch.randn_like(std)
                z = z_mu + std * eps
            else:
                # At inference, use mean (deterministic)
                z = z_mu
        else:
            z = self.z_proj(pooled)

        # Bond prediction (main task)
        bond_pred = self.bond_head(z)
        hohfeld_pred = self.hohfeld_head(z)

        # Adversarial predictions (gradient reversal)
        z_rev = GradientReversalLayer.apply(z, adv_lambda)

        # v10.16.7: Get predictions from ALL adversarial heads
        language_preds = [head(z_rev) for head in self.language_heads]
        period_preds = [head(z_rev) for head in self.period_heads]

        # Primary prediction is average of all heads (for evaluation)
        language_pred = torch.stack(language_preds).mean(dim=0)
        period_pred = torch.stack(period_preds).mean(dim=0)

        return {
            "bond_pred": bond_pred,
            "hohfeld_pred": hohfeld_pred,
            "language_pred": language_pred,
            "period_pred": period_pred,
            "language_preds_all": language_preds,  # v10.16.7: all heads
            "period_preds_all": period_preds,      # v10.16.7: all heads
            "context_pred": self.context_head(z),
            "z": z,
            "z_mu": z_mu,        # v10.16.8: VIB mean
            "z_logvar": z_logvar,  # v10.16.8: VIB log-variance
        }

    def get_bond_embedding(self, input_ids, attention_mask):
        """Get z_bond embedding for geometric analysis."""
        enc = self.encoder(input_ids, attention_mask)
        if hasattr(enc, "pooler_output") and enc.pooler_output is not None:
            pooled = enc.pooler_output
        else:
            pooled = enc.last_hidden_state[:, 0]
        return self.z_proj(pooled)


# Initialize tokenizer for selected backbone
print(f"\nLoading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"  Vocab size: {tokenizer.vocab_size:,}")


# Dataset with Hohfeld support
class NativeDataset(Dataset):
    def __init__(self, ids_set, passages_file, bonds_file, tokenizer, max_len=128, filter_none=True):
        """
        Args:
            filter_none: If True, exclude samples with no detected bond (NONE class).
                        This improves training by focusing on labeled examples.
        """
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.data = []
        self.filter_none = filter_none
        _skipped_none = 0

        bonds_by_id = {}
        with open(bonds_file) as fb:
            for line in fb:
                b = json.loads(line)
                bonds_by_id[b["passage_id"]] = b

        with open(passages_file) as fp:
            for line in tqdm(fp, desc="Loading", unit="line"):
                p = json.loads(line)
                if p["id"] in ids_set and p["id"] in bonds_by_id:
                    b = bonds_by_id[p["id"]]
                    bond_type = b.get("bond_type") or b.get("bonds", {}).get("primary_bond")

                    # Filter out NONE/null bonds if requested
                    if filter_none and (bond_type is None or bond_type == "NONE" or bond_type == "NEUTRAL"):
                        _skipped_none += 1
                        continue

                    self.data.append(
                        {
                            "text": p["text"][:1000],
                            "language": p["language"],
                            "period": p.get("time_periods", ["UNKNOWN"])[0],
                            "bond": bond_type,
                            "hohfeld": None,
                            "context": b.get("context")
                            or b.get("bonds", {}).get("context", "unknown"),
                            "confidence": b.get("confidence")
                            or b.get("bonds", {}).get("confidence", "medium"),
                        }
                    )
        print(f"  Loaded {len(self.data):,} samples" + (f" (filtered {_skipped_none:,} NONE bonds)" if filter_none else ""))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        enc = self.tokenizer(
            item["text"],
            truncation=True,
            max_length=self.max_len,
            padding="max_length",
            return_tensors="pt",
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "bond_label": BOND_TO_IDX.get(item["bond"], 9),
            "language_label": LANG_TO_IDX.get(item["language"], 4),
            "period_label": PERIOD_TO_IDX.get(item["period"], 9),
            "hohfeld_label": HOHFELD_TO_IDX.get(item["hohfeld"], 0) if item["hohfeld"] else 0,
            "context_label": CONTEXT_TO_IDX.get(item["context"], 2),
            "sample_weight": get_confidence_weight(item["confidence"]),
            "language": item["language"],
            "context": item["context"],
            "confidence": item["confidence"],
            "text": item["text"],  # Raw text for role augmentation
        }


def collate_fn(batch):
    return {
        "input_ids": torch.stack([x["input_ids"] for x in batch]),
        "attention_mask": torch.stack([x["attention_mask"] for x in batch]),
        "bond_labels": torch.tensor([x["bond_label"] for x in batch]),
        "language_labels": torch.tensor([x["language_label"] for x in batch]),
        "period_labels": torch.tensor([x["period_label"] for x in batch]),
        "hohfeld_labels": torch.tensor([x["hohfeld_label"] for x in batch]),
        "context_labels": torch.tensor([x["context_label"] for x in batch]),
        "sample_weights": torch.tensor([x["sample_weight"] for x in batch], dtype=torch.float),
        "languages": [x["language"] for x in batch],
        "contexts": [x["context"] for x in batch],
        "confidences": [x["confidence"] for x in batch],
        "texts": [x["text"] for x in batch],  # v10.10: raw texts for role augmentation
    }


print(f"\nArchitecture ready for {BACKBONE}")
print(f"  Bond classes: {len(BondType)}")
print(f"  Languages: {len(LANG_TO_IDX)}")
print("\n" + "=" * 60)  # ===== v10.15.1: GPU Memory Probing (generalized) =====


def probe_max_batch(
    model, tokenizer, device, target_batch=4096, encoder_trainable=False, mode="train"
):
    """Binary search for max batch size that fits in GPU memory.

    Args:
        model: BIPModel instance
        tokenizer: tokenizer for the model
        device: torch device
        target_batch: starting upper bound for search
        encoder_trainable: if True, tests with backward pass (4x memory)
        mode: "train" or "eval" - eval can use 2x memory vs train

    Returns:
        Safe batch size with 20% headroom
    """
    import gc

    # Adjust target based on mode
    if mode == "eval":
        # Eval doesn't need gradients, can use ~2x train batch
        target_batch = min(target_batch * 2, 1024)
    elif encoder_trainable:
        # Much lower target if encoder is trainable (gradient memory)
        target_batch = min(target_batch, 64)

    print(f"  [v10.15.1] Probing max batch (mode={mode}, trainable={encoder_trainable})...", end="")

    low, high = 8, target_batch
    best = low
    seq_len = 128

    while low <= high:
        mid = (low + high) // 2
        try:
            test_ids = torch.zeros((mid, seq_len), dtype=torch.long, device=device)
            test_mask = torch.ones((mid, seq_len), dtype=torch.long, device=device)

            if mode == "train" and encoder_trainable:
                model.train()
                out = model(test_ids, test_mask, 0)
                loss = out["bond_pred"].mean()
                loss.backward()
                model.zero_grad()
            else:
                model.eval()
                with torch.no_grad():
                    _ = model(test_ids, test_mask, 0)

            best = mid
            low = mid + 1
            del test_ids, test_mask
            torch.cuda.empty_cache()

        except Exception as e:
            err = str(e).lower()
            if "out of memory" in err or "cuda" in err or "alloc" in err or "oom" in err:
                high = mid - 1
                try:
                    del test_ids, test_mask
                except:
                    pass
                gc.collect()
                torch.cuda.empty_cache()
                if torch.cuda.is_available():
                    torch.cuda.synchronize()
            else:
                raise

    safe_batch = int(best * 0.8)
    print(f" max={best}, using {safe_batch}")
    return max(8, safe_batch)


# Global cache for probed batch sizes
_PROBED_BATCHES = {}


def get_probed_batch(model, tokenizer, device, mode="train", encoder_trainable=False):
    """Get cached or probe batch size for given mode."""
    key = f"{mode}_{encoder_trainable}"
    if key not in _PROBED_BATCHES:
        _PROBED_BATCHES[key] = probe_max_batch(
            model, tokenizer, device, encoder_trainable=encoder_trainable, mode=mode
        )
    return _PROBED_BATCHES[key]

MODEL ARCHITECTURE
Backbone: LaBSE (sentence-transformers/LaBSE)
Hidden size: 768

Loading tokenizer: sentence-transformers/LaBSE


tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

  Vocab size: 501,153

Architecture ready for LaBSE
  Bond classes: 10
  Languages: 8



In [ ]:
# @title 7. Training Loop { display-mode: "form" }
# @markdown Training with tuned adversarial weights and hardware-optimized parameters
# @markdown v10.16.8: Encoder unfreezing + stronger adversarial training

# ===== SUPPRESS DATALOADER MULTIPROCESSING WARNINGS =====
# These occur during garbage collection and bypass normal exception handling
import io
import logging
import os
import random
import sys
import warnings

# Method 1: Filter warnings
warnings.filterwarnings("ignore", message=".*can only test a child process.*")
warnings.filterwarnings("ignore", category=UserWarning, module="torch.utils.data")

# Method 2: Suppress logging
logging.getLogger("torch.utils.data.dataloader").setLevel(logging.CRITICAL)


# Method 3: Redirect stderr during DataLoader cleanup (most effective)
class StderrFilter(io.TextIOWrapper):
    """Filters out DataLoader multiprocessing cleanup messages from stderr"""

    def __init__(self, original):
        self.original = original
        self.buffer_lines = []

    def write(self, text):
        # Filter out the specific error patterns
        skip_patterns = [
            "can only test a child process",
            "_MultiProcessingDataLoaderIter.__del__",
            "_shutdown_workers",
            "Exception ignored in:",
            "w.is_alive()",
        ]
        # Buffer multi-line error messages
        if any(p in text for p in skip_patterns):
            return len(text)  # Pretend we wrote it
        # Also skip if it looks like part of a traceback for these errors
        if text.strip().startswith("^") and len(text.strip()) < 80:
            return len(text)
        if text.strip().startswith('File "/usr') and "dataloader.py" in text:
            return len(text)
        if text.strip() == "Traceback (most recent call last):":
            self.buffer_lines = [text]
            return len(text)
        if self.buffer_lines:
            self.buffer_lines.append(text)
            # Check if this is the DataLoader error traceback
            full_msg = "".join(self.buffer_lines)
            if any(p in full_msg for p in skip_patterns):
                self.buffer_lines = []
                return len(text)
            # After 10 lines, flush if not the target error
            if len(self.buffer_lines) > 10:
                for line in self.buffer_lines:
                    self.original.write(line)
                self.buffer_lines = []
        return self.original.write(text)

    def flush(self):
        if self.buffer_lines:
            # Flush any remaining buffered content
            for line in self.buffer_lines:
                self.original.write(line)
            self.buffer_lines = []
        self.original.flush()

    def __getattr__(self, name):
        return getattr(self.original, name)


# Install the stderr filter
_original_stderr = sys.stderr
sys.stderr = StderrFilter(_original_stderr)

# Method 4: Patch the DataLoader cleanup function directly
try:
    import torch.utils.data.dataloader as dl_module

    _original_del = dl_module._MultiProcessingDataLoaderIter.__del__

    def _patched_del(self):
        try:
            _original_del(self)
        except (AssertionError, AttributeError, RuntimeError):
            pass  # Silently ignore cleanup errors

    dl_module._MultiProcessingDataLoaderIter.__del__ = _patched_del
except Exception:
    pass  # If patching fails, the stderr filter will still work

import gc

from sklearn.metrics import f1_score

# ===== INITIAL MEMORY CLEANUP =====
# Clean up any leftover GPU memory from previous runs before starting
print("Cleaning up GPU memory from previous runs...")
if torch.cuda.is_available():
    # Clear any existing models/tensors from globals
    for var_name in list(globals().keys()):
        obj = globals().get(var_name)
        if isinstance(obj, torch.nn.Module):
            try:
                obj.cpu()
                del globals()[var_name]
            except:
                pass
        elif isinstance(obj, torch.Tensor) and obj.is_cuda:
            try:
                del globals()[var_name]
            except:
                pass

    # Force garbage collection
    for _ in range(5):
        gc.collect()

    # Clear CUDA cache
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    torch.cuda.reset_peak_memory_stats()

    # Check memory status
    mem_alloc = torch.cuda.memory_allocated() / 1e9
    mem_reserved = torch.cuda.memory_reserved() / 1e9
    print(f"  GPU memory: {mem_alloc:.2f} GB allocated, {mem_reserved:.2f} GB reserved")

    if mem_alloc > 1.0:
        print(f"  WARNING: {mem_alloc:.1f} GB still allocated - consider restarting runtime")
        # Try more aggressive cleanup
        torch.cuda.ipc_collect()
        gc.collect()
        torch.cuda.empty_cache()
else:
    print("  No GPU detected")

print()

# @markdown **Splits to train:**
# @markdown v10.13: Automatically uses splits generated in Cell 4
TRAIN_ALL_SPLITS = True  # @param {type:"boolean"}
# @markdown Train all splits from Cell 4. If False, specify splits below.

SPECIFIC_SPLITS = ""  # @param {type:"string"}
# @markdown Comma-separated split names (only used if TRAIN_ALL_SPLITS=False)
# @markdown Example: "hebrew_to_others, confucian_to_buddhist, mixed_baseline"

MAX_SPLITS = 0  # @param {type:"integer"}
# @markdown Limit number of splits (0 = no limit). Useful for quick testing.

# @markdown **Reproducibility:**
USE_FIXED_SEED = True  # @param {type:"boolean"}
RANDOM_SEED = 42  # @param {type:"integer"}
# @markdown Set USE_FIXED_SEED=True for reproducible results, False for random initialization

if USE_FIXED_SEED:
    import numpy as np

    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED)
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Using fixed seed: {RANDOM_SEED}")
else:
    torch.backends.cudnn.benchmark = True  # Faster but non-deterministic
    print("Using random initialization")

# @markdown **Hyperparameters:**
LANG_WEIGHT = 1.0  # @param {type:"number"} # v10.16.4: Much stronger (was 0.3)
PERIOD_WEIGHT = 0.8  # @param {type:"number"} # v10.16.4: Much stronger (was 0.2)
# Use NUM_EPOCHS from Cell 1, or default
try:
    N_EPOCHS = NUM_EPOCHS
except NameError:
    N_EPOCHS = 10  # Default fallback

# @markdown **Context-Aware Training:**
USE_CONFIDENCE_WEIGHTING = True  # @param {type:"boolean"}
# @markdown Weight prescriptive (high confidence) examples 2x in loss

USE_CONTEXT_AUXILIARY = True  # @param {type:"boolean"}
# @markdown Add context prediction as auxiliary training target

CONTEXT_LOSS_WEIGHT = 0.33  # @param {type:"number"}
# @markdown Weight for context prediction loss

STRICT_PRESCRIPTIVE_TEST = False  # @param {type:"boolean"}
# @markdown Only evaluate on prescriptive examples (reduces test set ~97%!)

# @markdown **v10.10: Role-Aware Data Augmentation:**
USE_ROLE_AUGMENTATION = True  # @param {type:"boolean"}
# @markdown Adds contrastive loss for agent/patient role sensitivity
ROLE_AUGMENT_PROB = 0.3  # @param {type:"number"}
# @markdown Probability of augmenting each batch
ROLE_CONTRASTIVE_WEIGHT = 0.2  # @param {type:"number"}
# @markdown Weight for role contrastive loss
ROLE_CONTRASTIVE_MARGIN = 0.5  # @param {type:"number"}

# @markdown **v10.15.1.2: Gradient Penalty for Adversarial Disentanglement:**
USE_GRADIENT_PENALTY = True  # @param {type:"boolean"}
# @markdown Adds gradient penalty to adversarial heads for smoother predictions
GRADIENT_PENALTY_WEIGHT = 0.02  # @param {type:"number"}
# @markdown Weight for gradient penalty loss

USE_COSINE_LR = True  # @param {type:"boolean"}
# @markdown Use cosine annealing learning rate schedule

# @markdown Minimum embedding distance for role-swapped pairs


def swap_roles_simple(text, language):
    """Simple role swap using word order reversal for common patterns.
    v10.10: Addresses weak role_swap sensitivity (0.003) from fuzz testing."""
    patterns = {
        "english": [
            (r"(\w+) must (\w+) (\w+)", r"\3 must \2 \1"),
            (r"(\w+) should (\w+) (\w+)", r"\3 should \2 \1"),
            (r"(\w+) shall (\w+) (\w+)", r"\3 shall \2 \1"),
            (r"the (\w+) must (\w+) the (\w+)", r"the \3 must \2 the \1"),
            (r"(\w+) is obligated to (\w+) (\w+)", r"\3 is obligated to \2 \1"),
            (r"(\w+) has a duty to (\w+) (\w+)", r"\3 has a duty to \2 \1"),
        ],
        "hebrew": [
            (r"על (\S+) ל(\S+) את (\S+)", r"על \3 ל\2 את \1"),
        ],
        "classical_chinese": [
            (r"(\S)當(\S)(\S)", r"\3當\2\1"),
            (r"(\S)須(\S)(\S)", r"\3須\2\1"),
            (r"(\S)應(\S)(\S)", r"\3應\2\1"),
        ],
        "arabic": [
            (r"يجب على (\S+) أن (\S+) (\S+)", r"يجب على \3 أن \2 \1"),
            (r"(\S+) عليه أن (\S+) (\S+)", r"\3 عليه أن \2 \1"),
        ],
        "sanskrit": [
            (r"(\S+)ः (\S+)म् (\S+)ति", r"\3ः \2म् \1ति"),
        ],
        "pali": [
            (r"(\S+)o (\S+)aṃ (\S+)ti", r"\3o \2aṃ \1ti"),
        ],
    }

    lang_patterns = patterns.get(language, patterns["english"])
    for pattern, replacement in lang_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            swapped = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
            if swapped != text:
                return swapped
    return None


# =============================================================================
# v10.16.1: STRUCTURAL PERTURBATIONS FOR CONTRASTIVE TRAINING
# =============================================================================
# These patterns match the fuzz test perturbation types to ensure training
# on the same transformations that will be evaluated.

STRUCTURAL_PATTERNS = {
    "obligation_to_permission": [
        ("must protect", "may protect"),
        ("has a duty to", "is allowed to"),
        ("are required to", "are permitted to"),
        ("must pay", "may pay"),
        ("shall not", "need not"),
        ("is obligated to", "is permitted to"),
        ("swore to", "considered whether to"),
        ("must tell", "may tell"),
        ("should help", "could help"),
        ("ought to", "might"),
    ],
    "add_harm": [
        ("helped", "refused to help"),
        ("protected", "endangered"),
        ("gave", "took"),
        ("saved", "abandoned"),
        ("cared for", "neglected"),
        ("forgave", "condemned"),
        ("supported", "undermined"),
        ("guided", "misled"),
        ("healed", "harmed"),
        ("blessed", "cursed"),
    ],
    "violation_to_fulfillment": [
        ("violated", "honored"),
        ("broke", "kept"),
        ("stole", "returned"),
        ("betrayed", "supported"),
        ("abandoned", "stayed with"),
        ("deceived", "was honest with"),
        ("cheated", "dealt fairly with"),
        ("destroyed", "preserved"),
        ("corrupted", "purified"),
        ("ignored", "attended to"),
    ],
}


def create_structural_perturbation(text, language):
    """Create a structural perturbation that changes moral meaning.
    Returns: (perturbed_text, perturbation_type) or (None, None)
    """
    # First try role swap (most impactful)
    swapped = swap_roles_simple(text, language)
    if swapped:
        return swapped, "role_swap"

    # Try other structural patterns
    text_lower = text.lower()
    for perturb_type, patterns in STRUCTURAL_PATTERNS.items():
        for orig, replacement in patterns:
            if orig in text_lower:
                # Case-preserving replacement
                import re as re_mod
                pattern = re_mod.compile(re_mod.escape(orig), re_mod.IGNORECASE)
                perturbed = pattern.sub(replacement, text, count=1)
                if perturbed != text:
                    return perturbed, perturb_type

    return None, None


def triplet_loss_geometric(anchor, positive, negative, margin=0.5):
    """
    Triplet loss for BIP geometric learning.
    Enforces: d(anchor, positive) + margin < d(anchor, negative)

    This directly encodes the BIP hypothesis:
    - Surface changes should NOT move embeddings (d_positive small)
    - Structural changes SHOULD move embeddings (d_negative large)
    """
    positive = positive.to(anchor.dtype)
    negative = negative.to(anchor.dtype)

    d_positive = F.pairwise_distance(anchor, positive)
    d_negative = F.pairwise_distance(anchor, negative)

    # Standard triplet margin loss
    loss = F.relu(d_positive - d_negative + margin)
    return loss.mean()


def ratio_regularization_loss(surface_distances, structural_distances, target_ratio=2.0):
    """
    Encourage structural distances to be TARGET_RATIO times larger than surface distances.
    This is the core BIP hypothesis.
    """
    mean_surface = surface_distances.mean()
    mean_structural = structural_distances.mean()

    ratio = mean_structural / (mean_surface + 1e-8)
    loss = F.relu(target_ratio - ratio)
    return loss



# ===== v10.15.1: SURFACE AUGMENTATION & CONTRASTIVE LOSS =====

import random

# Simple synonym mappings for surface perturbation
SURFACE_SYNONYMS = {
    "good": ["virtuous", "righteous", "moral", "ethical"],
    "bad": ["evil", "wicked", "immoral", "wrong"],
    "must": ["should", "ought to", "has to", "needs to"],
    "can": ["may", "is able to", "is permitted to"],
    "right": ["correct", "proper", "appropriate"],
    "wrong": ["incorrect", "improper", "inappropriate"],
    "help": ["assist", "aid", "support"],
    "harm": ["hurt", "damage", "injure"],
    "person": ["individual", "human", "someone"],
    "people": ["individuals", "humans", "others"],
}

# Common names for swapping
COMMON_NAMES = ["Alex", "Sam", "Jordan", "Taylor", "Morgan", "Casey", "Riley", "Quinn"]

# Irrelevant details to insert
IRRELEVANT_DETAILS = [
    # v10.16.3: Aligned with fuzz test patterns for consistency
    # Original patterns (insertable mid-sentence)
    "on a Tuesday",
    "while it was raining",
    "near the old building",
    "during the afternoon",
    "in the usual manner",
    # Fuzz test patterns (appendable at end - matches Cell 9 exactly)
    "It was Tuesday.",
    "The room was blue.",
    "Last summer.",
    "The weather was pleasant.",
    "It happened at noon.",
    "The year was uncertain.",
    "Birds sang nearby.",
    "The moon was full.",
    "Rain had fallen earlier.",
    "The road was dusty.",
    "Flowers bloomed outside.",
]


def augment_surface_synonym(text: str) -> str:
    """Replace words with synonyms (surface change, same meaning)."""
    words = text.split()
    for i, word in enumerate(words):
        word_lower = word.lower().strip(".,!?")
        if word_lower in SURFACE_SYNONYMS and random.random() < 0.3:
            replacement = random.choice(SURFACE_SYNONYMS[word_lower])
            # Preserve capitalization
            if word[0].isupper():
                replacement = replacement.capitalize()
            words[i] = replacement + word[len(word_lower) :]
    return " ".join(words)


def augment_surface_name(text: str) -> str:
    """Swap names with other names (surface change, same moral content)."""
    # Find capitalized words that might be names
    for name in COMMON_NAMES:
        if name in text:
            new_name = random.choice([n for n in COMMON_NAMES if n != name])
            text = text.replace(name, new_name)
            break
    return text


def augment_surface_detail(text: str) -> str:
    """Insert irrelevant detail (surface change, same moral content).

    v10.16.3: Now matches fuzz test patterns - 50% append at end, 50% insert mid-sentence.
    This teaches model to be robust to irrelevant details in both positions.
    """
    if random.random() < 0.7 and len(text) > 20:  # Increased from 0.5 to 0.7
        detail = random.choice(IRRELEVANT_DETAILS)

        # 50% chance: append at end (matches fuzz test pattern exactly)
        # 50% chance: insert after punctuation (additional robustness)
        if random.random() < 0.5:
            # Append at end - matches fuzz test's surface_irrelevant_detail
            text = text.rstrip() + " " + detail.strip()
        else:
            # Insert after first sentence or comma
            insert_points = [m.end() for m in re.finditer(r"[,.]", text)]
            if insert_points:
                pos = random.choice(insert_points[:3])  # Early in text
                text = text[:pos] + " " + detail + text[pos:]
            else:
                # Fallback to append if no punctuation found
                text = text.rstrip() + " " + detail.strip()
    return text


def create_surface_augmented(text: str) -> str:
    """Create a surface-augmented version of text (same moral content)."""
    augmenters = [augment_surface_synonym, augment_surface_name, augment_surface_detail]
    # Apply 1-2 random augmentations
    for _ in range(random.randint(1, 2)):
        aug_fn = random.choice(augmenters)
        text = aug_fn(text)
    return text


def info_nce_loss(
    anchor: torch.Tensor, positive: torch.Tensor, temperature: float = 0.07
) -> torch.Tensor:
    """
    InfoNCE contrastive loss for surface invariance.

    anchor: embeddings of original texts [batch, z_dim]
    positive: embeddings of surface-augmented texts [batch, z_dim]
    temperature: softmax temperature (lower = harder)

    Goal: anchor should be similar to its positive (same moral content)
          and dissimilar to other positives (different moral content)
    """
    # Ensure same dtype (AMP can cause anchor=float16, positive=float32)
    positive = positive.to(anchor.dtype)

    # Normalize embeddings
    anchor = F.normalize(anchor, dim=1)
    positive = F.normalize(positive, dim=1)

    # Similarity matrix: anchor_i vs positive_j
    # Diagonal = positive pairs (same text, surface augmented)
    # Off-diagonal = negative pairs (different texts)
    similarity = torch.mm(anchor, positive.T) / temperature

    # Labels: diagonal should be highest
    labels = torch.arange(anchor.size(0), device=anchor.device)

    # Cross-entropy loss (each anchor should match its positive)
    loss = F.cross_entropy(similarity, labels)

    return loss


_skip_complete = False

# ===== SKIP TRAINING MODE =====
try:
    if SKIP_TRAINING:
        print("=" * 60)
        print("SKIP_TRAINING MODE - Loading models from Drive")
        print("=" * 60)

        # Load splits
        with open("data/splits/all_splits.json", encoding="utf-8") as f:
            all_splits = json.load(f)

        # Find available checkpoints
        available_models = []
        for split_name in all_splits.keys():
            ckpt_path = f"{SAVE_DIR}/best_{split_name}.pt"
            if os.path.exists(ckpt_path):
                available_models.append(split_name)
                print(f"  Found: {split_name}")

        if not available_models:
            print("\nWARNING: No trained models found in Drive!")
            print(f"  Looked in: {SAVE_DIR}")
            print("  Falling back to training mode...")
            SKIP_TRAINING = False
            # Continue to training below
        else:
            print(f"\nFound {len(available_models)} trained models")
            print("Skipping Cell 7 - proceed to Cell 8 for evaluation")

            # Create minimal results dict for Cell 8 compatibility
            all_results = {}
            for split_name in available_models:
                all_results[split_name] = {"status": "loaded_from_drive"}

            # Exit cell early - only when checkpoints exist
            _skip_complete = True
except NameError:
    pass  # SKIP_TRAINING not defined, continue normally


if not _skip_complete:
    print("=" * 60)
    print("TRAINING BIP MODEL")
    print("=" * 60)

    # v10.15.1.4: Check for encoder unfreezing config
    try:
        _unfreeze = UNFREEZE_ENCODER
        _unfreeze_after = UNFREEZE_AFTER_EPOCHS
    except NameError:
        _unfreeze = False
        _unfreeze_after = 2

    print(
        f"\nEncoder mode: {'UNFROZEN after epoch ' + str(_unfreeze_after) if _unfreeze else 'FROZEN (probe-only)'}"
    )

    print("\nSettings:")
    print(f"  Backbone:     {BACKBONE}")
    print(f"  GPU Tier:     {GPU_TIER}")
    print(f"  Batch size:   {BATCH_SIZE}")
    print(f"  Workers:      {NUM_WORKERS}")
    print(f"  Learning rate: {LR:.2e}")
    print(f"  Adv weights:  lang={LANG_WEIGHT}, period={PERIOD_WEIGHT}")
    print("  (v10.16.4: Increased for stronger invariance - grad clipping prevents explosion)")

    # v10.16.5: Confusion loss settings with fallbacks
    try:
        _use_confusion = USE_CONFUSION_LOSS
        _confusion_weight = CONFUSION_WEIGHT
    except NameError:
        _use_confusion = False
        _confusion_weight = 0.0
    USE_CONFUSION_LOSS = _use_confusion
    CONFUSION_WEIGHT = _confusion_weight
    print(f"  Confusion loss: {USE_CONFUSION_LOSS} (weight={CONFUSION_WEIGHT})")
    if USE_CONFUSION_LOSS:
        print("  (v10.16.5: Forces uniform predictions - prevents adversarial head evasion)")

    # v10.16.8: VIB settings with fallbacks
    try:
        _use_vib = USE_VIB
        _vib_beta = VIB_BETA
        _vib_anneal = VIB_ANNEAL_EPOCHS
    except NameError:
        _use_vib = False
        _vib_beta = 0.001
        _vib_anneal = 5
    USE_VIB = _use_vib
    VIB_BETA = _vib_beta
    VIB_ANNEAL_EPOCHS = _vib_anneal
    print(f"  VIB: {USE_VIB} (beta={VIB_BETA}, anneal={VIB_ANNEAL_EPOCHS} epochs)")
    if USE_VIB:
        print("  (v10.16.8: Information bottleneck - compresses away spurious info)")

    print(f"  Confidence weighting: {USE_CONFIDENCE_WEIGHTING}")
    print(f"  Context auxiliary: {USE_CONTEXT_AUXILIARY} (weight={CONTEXT_LOSS_WEIGHT})")
    print(f"  Strict prescriptive test: {STRICT_PRESCRIPTIVE_TEST}")
    print(
        f"  Role augmentation: {USE_ROLE_AUGMENTATION} (prob={ROLE_AUGMENT_PROB}, weight={ROLE_CONTRASTIVE_WEIGHT})"
    )

    # tokenizer loaded in Cell 6 based on BACKBONE selection

    with open("data/splits/all_splits.json") as f:
        all_splits = json.load(f)

    # Build splits_to_train from Cell 4 output
    if TRAIN_ALL_SPLITS:
        splits_to_train = list(all_splits.keys())
    else:
        # Parse comma-separated list
        splits_to_train = [s.strip() for s in SPECIFIC_SPLITS.split(",") if s.strip()]
        # Filter to only splits that exist
        splits_to_train = [s for s in splits_to_train if s in all_splits]

    # Apply max limit if set
    if MAX_SPLITS > 0:
        splits_to_train = splits_to_train[:MAX_SPLITS]

    # =============================================================================
    # DYNAMIC BATCH SIZE PROBING (like WiFi rate adaptation)
    # =============================================================================
    # v10.15.1: probe_max_batch moved to Cell 6

    PROBED_BATCH = None  # Will be set on first split

    print(f"\nTraining {len(splits_to_train)} splits: {splits_to_train}")

    all_results = {}
    MIN_TEST_SIZE = 100  # Lowered to allow smaller test sets like Chinese

    for split_idx, split_name in enumerate(splits_to_train):
        split_start = time.time()
        print("\n" + "=" * 60)
        print(f"[{split_idx + 1}/{len(splits_to_train)}] {split_name}")
        print("=" * 60)

        split = all_splits[split_name]
        print(f"Train: {split['train_size']:,} | Test: {split['test_size']:,}")

        if split["test_size"] < MIN_TEST_SIZE:
            print(f"WARNING: Test set only {split['test_size']} samples (need {MIN_TEST_SIZE})")
            print("Skipping this split - results would be unreliable")
            print("To fix: Add more data to the test languages/periods")
            continue

        # Create model with OOM recovery
        def create_model_with_retry():
            """Create model, cleaning up GPU memory if OOM occurs."""
            try:
                return BIPModel(z_dim=Z_DIM).to(device)
            except torch.cuda.OutOfMemoryError:
                print("  OOM on model creation - cleaning up and retrying...")
                # Clean up any existing model in globals
                _g = globals()
                for _var in ["model", "analyzer", "encoder"]:
                    if _var in _g and _g[_var] is not None:
                        try:
                            if hasattr(_g[_var], "cpu"):
                                _g[_var].cpu()
                            _g[_var] = None
                        except:
                            pass
                # Force cleanup
                gc.collect()
                gc.collect()
                torch.cuda.empty_cache()
                torch.cuda.synchronize()
                # Retry
                return BIPModel(z_dim=Z_DIM).to(device)

        model = create_model_with_retry()

        # v10.16.2: Enable gradient checkpointing for memory efficiency
        try:
            if USE_GRADIENT_CHECKPOINTING and hasattr(model.encoder, 'gradient_checkpointing_enable'):
                model.encoder.gradient_checkpointing_enable()
                print("  Gradient checkpointing ENABLED")
        except NameError:
            pass

        # v10.15.1.5: Class weights to handle imbalanced bond types
        # Upweight rare classes, downweight NONE (index 9)
        BOND_CLASS_WEIGHTS = torch.tensor([
            2.0,  # HARM_PREVENTION
            2.0,  # RECIPROCITY
            2.0,  # AUTONOMY
            2.0,  # PROPERTY
            2.0,  # FAMILY
            2.0,  # AUTHORITY
            2.0,  # CARE
            2.0,  # FAIRNESS
            2.0,  # CONTRACT
            0.1,  # NONE - heavily downweighted
        ], device=device)
        print(f"  Bond class weights: rare=2.0, NONE=0.1")

        # v10.15.1.4: Conditional encoder freezing
        # IMPORTANT: Do NOT enable gradient checkpointing yet - it causes NaN when unfreezing
        if _unfreeze:
            print(f"  Encoder will be UNFROZEN after epoch {_unfreeze_after}")
            # Start frozen, unfreeze later (warmup)
            for param in model.encoder.parameters():
                param.requires_grad = False
            _encoder_frozen = True
        else:
            print("  Encoder FROZEN (probe-only mode)")
            for param in model.encoder.parameters():
                param.requires_grad = False
            _encoder_frozen = True

        # Count trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in model.parameters())
        print(
            f"  Trainable: {trainable_params:,} / {total_params:,} ({100 * trainable_params / total_params:.1f}%)"
        )

        train_dataset = NativeDataset(
            set(split["train_ids"]),
            "data/processed/passages.jsonl",
            "data/processed/bonds.jsonl",
            tokenizer,
        )

        test_ids_to_use = split["test_ids"][:MAX_TEST_SAMPLES]

        # Optional: strict prescriptive-only test
        if STRICT_PRESCRIPTIVE_TEST:
            print("Filtering to prescriptive examples only...")
            # Load bonds to filter
            prescriptive_ids = set()
            with open("data/processed/bonds.jsonl") as f:
                for line in f:
                    b = json.loads(line)
                    if b.get("context") == "prescriptive":
                        prescriptive_ids.add(b["passage_id"])
            test_ids_to_use = [tid for tid in test_ids_to_use if tid in prescriptive_ids]
            print(f"  Filtered to {len(test_ids_to_use):,} prescriptive samples")

        test_dataset = NativeDataset(
            set(test_ids_to_use),
            "data/processed/passages.jsonl",
            "data/processed/bonds.jsonl",
            tokenizer,
        )

        if len(train_dataset) == 0:
            print("ERROR: No training data!")
            continue

        # Use hardware-optimized batch size (WiFi-style probing)
        # v10.15.1.4: Probe with encoder_trainable=False since we start frozen
        if "_probed_batch" not in globals():
            _PROBED_BATCHES["train"] = get_probed_batch(
                model, tokenizer, device, BATCH_SIZE, encoder_trainable=False
            )
        # v10.16.6: Hard cap on batch size for large datasets to prevent OOM
        # For datasets > 50k samples, cap at 512 even if probe says more is OK
        # This prevents OOM on mixed_baseline (125k samples)
        _probed = _PROBED_BATCHES.get("train", BATCH_SIZE)
        _size_based = max(32, len(train_dataset) // 20)
        _hard_cap = 512 if len(train_dataset) > 50000 else 1024 if len(train_dataset) > 20000 else 2048
        actual_batch = min(_probed, _size_based, _hard_cap)
        print(f"  Batch cap: dataset={len(train_dataset):,} -> hard_cap={_hard_cap}")
        print(f"Actual batch size: {actual_batch}")

        train_loader = DataLoader(
            train_dataset,
            batch_size=actual_batch,
            shuffle=True,
            collate_fn=collate_fn,
            drop_last=True,
            num_workers=0,
            pin_memory=True,
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=actual_batch,
            shuffle=False,
            collate_fn=collate_fn,
            num_workers=0,
            pin_memory=True,
        )

        # v10.15.1.4: Differential learning rates
        try:
            _encoder_lr = ENCODER_LR
            _head_lr = HEAD_LR
        except NameError:
            _encoder_lr = 5e-7
            _head_lr = 1e-3

        encoder_params = list(model.encoder.parameters())
        head_params = [p for n, p in model.named_parameters() if "encoder" not in n]

        optimizer = torch.optim.AdamW(
            [
                {"params": encoder_params, "lr": _encoder_lr},
                {"params": head_params, "lr": _head_lr},
            ],
            weight_decay=0.01,
        )
        print(f"  Optimizer: AdamW (encoder_lr={_encoder_lr:.0e}, head_lr={_head_lr:.0e})")

        # v10.15.1.2: Cosine annealing LR schedule
        if USE_COSINE_LR:
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer, T_max=N_EPOCHS, eta_min=LR / 10
            )
            print(f"  Using cosine LR schedule: {LR:.2e} -> {LR / 10:.2e}")
        else:
            scheduler = None

        # Gradient clipping setup
        try:
            grad_clip = GRADIENT_CLIP if GRADIENT_CLIP > 0 else None
        except NameError:
            grad_clip = 1.0  # Default

        # Early stopping setup
        try:
            early_stop_patience = EARLY_STOPPING_PATIENCE if EARLY_STOPPING_PATIENCE > 0 else None
        except NameError:
            early_stop_patience = 3  # Default
        epochs_without_improvement = 0

        def get_adv_lambda(
            epoch, warmup=ADV_WARMUP_EPOCHS, max_lambda=ADV_MAX_LAMBDA, split_name=None
        ):
            """Ramp adversarial strength with per-split support (v10.15.1.3)"""
            effective_max = max_lambda
            try:
                if PER_SPLIT_TUNING and split_name and split_name in SPLIT_ADV_LAMBDA:
                    effective_max = SPLIT_ADV_LAMBDA[split_name]
            except NameError:
                pass
            if epoch <= warmup:
                return (epoch / warmup) * effective_max
            return effective_max

        best_loss = float("inf")
        no_improve_count = 0
        start_epoch = 1
        _unfreeze_warmup = 0  # Track warmup epochs after unfreeze

        # Check for existing checkpoint to resume from
        checkpoint_path = f"models/checkpoints/latest_{split_name}.pt"
        if os.path.exists(checkpoint_path):
            print("  Found checkpoint, checking validity...")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            ckpt_loss = checkpoint.get("best_loss", float("inf"))

            # Skip corrupted checkpoints (inf/nan loss indicates bad weights)
            if ckpt_loss == float("inf") or ckpt_loss != ckpt_loss:  # NaN check
                print(f"  Checkpoint corrupted (best_loss={ckpt_loss}) - deleting and starting fresh")
                os.remove(checkpoint_path)
                # Also remove best checkpoint if it exists (might be corrupted too)
                best_ckpt = f"models/checkpoints/best_{split_name}.pt"
                if os.path.exists(best_ckpt):
                    os.remove(best_ckpt)
            else:
                model.load_state_dict(checkpoint["model_state"])
                optimizer.load_state_dict(checkpoint["optimizer_state"])
                start_epoch = checkpoint["epoch"] + 1
                best_loss = ckpt_loss
                print(f"  Resuming from epoch {start_epoch}, best_loss={best_loss:.4f}")

        # v10.15.1.4: Gradient accumulation
        try:
            _grad_accum = GRADIENT_ACCUMULATION_STEPS
        except NameError:
            _grad_accum = 1

        _nan_batch_count = 0  # Track consecutive NaN batches
        _max_nan_before_reset = 5  # Reset model after this many consecutive NaN batches

        for epoch in range(start_epoch, N_EPOCHS + 1):
            # v10.15.1.4: Check if we should unfreeze encoder
            if _unfreeze and _encoder_frozen and epoch >= _unfreeze_after:
                print(f"\n  >>> UNFREEZING ENCODER at epoch {epoch} <<<")

                # Step 1: Unfreeze encoder parameters (v10.15.1: layer-wise support)
                try:
                    _n_layers = UNFREEZE_LAYERS
                except NameError:
                    _n_layers = 0  # 0 = unfreeze all

                if _n_layers > 0 and hasattr(model.encoder, "encoder"):
                    # Only unfreeze top N transformer layers
                    try:
                        all_layers = list(model.encoder.encoder.layer)
                        layers_to_unfreeze = all_layers[-_n_layers:]
                        for layer in layers_to_unfreeze:
                            for param in layer.parameters():
                                param.requires_grad = True
                        # Also unfreeze pooler if exists
                        if hasattr(model.encoder, "pooler"):
                            for param in model.encoder.pooler.parameters():
                                param.requires_grad = True
                        print(f"  Unfroze top {len(layers_to_unfreeze)} encoder layers")
                    except Exception as e:
                        print(f"  Layer-wise unfreeze failed ({e}), unfreezing all")
                        for param in model.encoder.parameters():
                            param.requires_grad = True
                else:
                    # Unfreeze all encoder parameters
                    for param in model.encoder.parameters():
                        param.requires_grad = True
                    print("  Unfroze ALL encoder parameters")
                _encoder_frozen = False

                trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
                print(f"  Trainable params now: {trainable:,}")

                # Step 2: Create fresh optimizer (old one has no state for encoder params)
                encoder_params = list(model.encoder.parameters())
                head_params = [p for n, p in model.named_parameters() if "encoder" not in n]

                # Start encoder at scaled LR (v10.15.1: configurable)
                try:
                    _lr_scale = ENCODER_LR_SCALE
                except NameError:
                    _lr_scale = 0.1  # Default 10x smaller
                _current_encoder_lr = _encoder_lr * _lr_scale
                optimizer = torch.optim.AdamW(
                    [
                        {"params": encoder_params, "lr": _current_encoder_lr},
                        {"params": head_params, "lr": _head_lr},
                    ],
                    weight_decay=0.01,
                )
                print(f"  New optimizer (encoder_lr={_current_encoder_lr:.0e}, warming up)")

                # Step 3: Reset AMP scaler if using AMP
                if USE_AMP:
                    scaler = torch.amp.GradScaler("cuda")
                    print("  Reset AMP scaler")

                # Step 4: Reduce batch size for unfrozen training
                # Re-probe with trainable encoder
                new_batch = probe_max_batch(
                    model, tokenizer, device, actual_batch, encoder_trainable=True
                )
                if new_batch < actual_batch:
                    actual_batch = new_batch
                    train_loader = DataLoader(
                        train_dataset,
                        batch_size=actual_batch,
                        shuffle=True,
                        collate_fn=collate_fn,
                        drop_last=True,
                        num_workers=0,
                        pin_memory=True,
                    )
                    print(f"  Reduced batch size to {actual_batch}")

                _unfreeze_warmup = 0

            # v10.15.1.4: Warm up encoder LR after unfreeze (over 5 epochs)
            if not _encoder_frozen and _unfreeze_warmup < 5:
                warmup_factor = (_unfreeze_warmup + 1) / 5
                _current_encoder_lr = _encoder_lr * warmup_factor
                for pg in optimizer.param_groups:
                    # Encoder param group has more params
                    if sum(p.numel() for p in pg["params"]) > 1000000:
                        pg["lr"] = _current_encoder_lr
                _unfreeze_warmup += 1
                print(f"  Encoder LR warmup: {_current_encoder_lr:.1e} ({_unfreeze_warmup}/5)")

            model.train()
            total_loss = 0
            n_batches = 0
            batch_count = 0

            for batch in tqdm(train_loader, desc=f"Epoch {epoch}", leave=False):
                optimizer.zero_grad()

                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                bond_labels = batch["bond_labels"].to(device)
                language_labels = batch["language_labels"].to(device)
                period_labels = batch["period_labels"].to(device)

                adv_lambda = get_adv_lambda(epoch, split_name=split_name)

                # Use new autocast API
                with torch.amp.autocast("cuda", enabled=USE_AMP):
                    out = model(input_ids, attention_mask, adv_lambda=adv_lambda)

                    # Check for NaN in model output (indicates corrupted weights)
                    if torch.isnan(out["z"]).any() or torch.isinf(out["z"]).any():
                        _nan_batch_count += 1
                        if _nan_batch_count >= _max_nan_before_reset:
                            print(f"    {_nan_batch_count} consecutive NaN outputs - resetting from best checkpoint")
                            best_ckpt = f"models/checkpoints/best_{split_name}.pt"
                            if os.path.exists(best_ckpt):
                                model.load_state_dict(torch.load(best_ckpt, map_location=device))
                                # Reset optimizer
                                optimizer = torch.optim.AdamW(
                                    [
                                        {"params": list(model.encoder.parameters()), "lr": _encoder_lr},
                                        {"params": [p for n, p in model.named_parameters() if "encoder" not in n], "lr": _head_lr},
                                    ],
                                    weight_decay=0.01,
                                )
                                if USE_AMP:
                                    scaler = torch.amp.GradScaler("cuda")
                                _nan_batch_count = 0
                                print("    Model reset successful")
                            else:
                                print("    No checkpoint to reset from - skipping batch")
                        continue

                    # Weighted bond loss with class weights
                    if USE_CONFIDENCE_WEIGHTING:
                        sample_weights = batch["sample_weights"].to(device)
                        loss_bond = F.cross_entropy(out["bond_pred"], bond_labels, weight=BOND_CLASS_WEIGHTS, reduction="none")
                        loss_bond = (loss_bond * sample_weights).mean()
                    else:
                        loss_bond = F.cross_entropy(out["bond_pred"], bond_labels, weight=BOND_CLASS_WEIGHTS)

                    # Context auxiliary loss
                    if USE_CONTEXT_AUXILIARY:
                        context_labels = batch["context_labels"].to(device)
                        loss_context = F.cross_entropy(out["context_pred"], context_labels)
                    else:
                        loss_context = 0

                    # v10.16.7: Compute adversarial loss from ALL heads
                    # Each head contributes to the loss - encoder must fool ALL of them
                    if "language_preds_all" in out and len(out["language_preds_all"]) > 1:
                        loss_lang = sum(F.cross_entropy(pred, language_labels)
                                       for pred in out["language_preds_all"]) / len(out["language_preds_all"])
                        loss_period = sum(F.cross_entropy(pred, period_labels)
                                         for pred in out["period_preds_all"]) / len(out["period_preds_all"])
                    else:
                        loss_lang = F.cross_entropy(out["language_pred"], language_labels)
                        loss_period = F.cross_entropy(out["period_pred"], period_labels)

                    # v10.16.7: Confusion loss applied to ALL adversarial heads
                    # This prevents ANY head from successfully predicting language/period
                    loss_confusion = torch.tensor(0.0, device=device)
                    try:
                        if USE_CONFUSION_LOSS:
                            max_lang_entropy = torch.log(torch.tensor(float(len(LANG_TO_IDX)), device=device))
                            max_period_entropy = torch.log(torch.tensor(float(len(PERIOD_TO_IDX)), device=device))

                            # Apply confusion loss to ALL heads
                            if "language_preds_all" in out and len(out["language_preds_all"]) > 1:
                                lang_confusions = []
                                period_confusions = []
                                for lang_pred, period_pred in zip(out["language_preds_all"], out["period_preds_all"]):
                                    # Language entropy for this head
                                    lang_probs = F.softmax(lang_pred, dim=-1)
                                    lang_entropy = -(lang_probs * torch.log(lang_probs + 1e-8)).sum(dim=-1).mean()
                                    lang_confusions.append(1.0 - (lang_entropy / max_lang_entropy))

                                    # Period entropy for this head
                                    period_probs = F.softmax(period_pred, dim=-1)
                                    period_entropy = -(period_probs * torch.log(period_probs + 1e-8)).sum(dim=-1).mean()
                                    period_confusions.append(1.0 - (period_entropy / max_period_entropy))

                                # Average confusion across all heads
                                lang_confusion = sum(lang_confusions) / len(lang_confusions)
                                period_confusion = sum(period_confusions) / len(period_confusions)
                            else:
                                # Fallback to single head
                                lang_probs = F.softmax(out["language_pred"], dim=-1)
                                lang_entropy = -(lang_probs * torch.log(lang_probs + 1e-8)).sum(dim=-1).mean()
                                period_probs = F.softmax(out["period_pred"], dim=-1)
                                period_entropy = -(period_probs * torch.log(period_probs + 1e-8)).sum(dim=-1).mean()
                                lang_confusion = 1.0 - (lang_entropy / max_lang_entropy)
                                period_confusion = 1.0 - (period_entropy / max_period_entropy)

                            loss_confusion = (lang_confusion + period_confusion) / 2
                    except NameError:
                        pass  # USE_CONFUSION_LOSS not defined

                # v10.16.8: VIB KL divergence loss
                # KL(q(z|x) || p(z)) where p(z) = N(0,1)
                # = -0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
                loss_kl = torch.tensor(0.0, device=device)
                try:
                    if USE_VIB and out.get("z_mu") is not None and out.get("z_logvar") is not None:
                        z_mu = out["z_mu"]
                        z_logvar = out["z_logvar"]

                        # KL divergence per sample, then mean
                        kl_per_sample = -0.5 * torch.sum(1 + z_logvar - z_mu.pow(2) - z_logvar.exp(), dim=1)
                        loss_kl = kl_per_sample.mean()

                        # Anneal beta from 0 to VIB_BETA over VIB_ANNEAL_EPOCHS
                        if epoch <= VIB_ANNEAL_EPOCHS:
                            effective_beta = VIB_BETA * (epoch / VIB_ANNEAL_EPOCHS)
                        else:
                            effective_beta = VIB_BETA

                        loss_kl = effective_beta * loss_kl
                except NameError:
                    pass
                except Exception as e:
                    pass  # Skip if VIB not configured

                loss = (
                    loss_bond
                    + LANG_WEIGHT * loss_lang
                    + PERIOD_WEIGHT * loss_period
                    + CONTEXT_LOSS_WEIGHT * loss_context
                    + CONFUSION_WEIGHT * loss_confusion  # v10.16.5
                    + loss_kl  # v10.16.8: VIB KL divergence (already scaled by beta)
                )

                # v10.10: Role contrastive loss for agent/patient sensitivity
                loss_role = torch.tensor(0.0, device=device)
                if USE_ROLE_AUGMENTATION and random.random() < ROLE_AUGMENT_PROB:
                    batch_texts = batch.get("texts", [])
                    batch_languages = batch.get("languages", [])

                    swapped_texts = []
                    original_indices = []

                    for i, (text, lang) in enumerate(zip(batch_texts, batch_languages)):
                        swapped = swap_roles_simple(text, lang)
                        if swapped:
                            swapped_texts.append(swapped)
                            original_indices.append(i)

                    if swapped_texts and len(swapped_texts) >= 2:
                        # Tokenize swapped texts
                        swapped_encoded = tokenizer(
                            swapped_texts,
                            padding=True,
                            truncation=True,
                            max_length=128,
                            return_tensors="pt",
                        )
                        swapped_ids = swapped_encoded["input_ids"].to(device)
                        swapped_mask = swapped_encoded["attention_mask"].to(device)

                        # Get embeddings for swapped texts (no gradients needed - saves memory!)
                        # We only need gradients through z_original, not z_swapped
                        with torch.no_grad():
                            swapped_out = model(swapped_ids, swapped_mask, adv_lambda=0)
                            z_swapped = swapped_out["z"].detach()

                        # Get original embeddings for corresponding indices (keeps gradients)
                        z_original = out["z"][original_indices]

                        # Ensure same dtype (AMP: z_original=float16, z_swapped=float32)
                        z_swapped = z_swapped.to(z_original.dtype)

                        # Contrastive loss: push role-swapped embeddings apart
                        # Hinge loss: max(0, margin - distance)
                        # Gradients flow through z_original only
                        distances = F.pairwise_distance(z_original, z_swapped)
                        loss_role = F.relu(ROLE_CONTRASTIVE_MARGIN - distances).mean()

                        # Clean up to prevent memory accumulation
                        del swapped_ids, swapped_mask, swapped_out, swapped_encoded
                        del z_original, z_swapped, distances

                loss = loss + ROLE_CONTRASTIVE_WEIGHT * loss_role

                # =============================================================
                # v10.16.1: STRUCTURAL CONTRASTIVE LOSS
                # =============================================================
                loss_structural = torch.tensor(0.0, device=device)
                _structural_distances = torch.tensor([0.0], device=device)
                try:
                    if USE_STRUCTURAL_CONTRASTIVE:
                        batch_texts = batch.get("texts", [])
                        batch_languages = batch.get("languages", [])

                        structural_texts = []
                        struct_original_indices = []

                        for si, (stext, slang) in enumerate(zip(batch_texts, batch_languages)):
                            sperturbed, _ = create_structural_perturbation(stext, slang)
                            if sperturbed:
                                structural_texts.append(sperturbed)
                                struct_original_indices.append(si)

                        if structural_texts and len(structural_texts) >= 2:
                            struct_encoded = tokenizer(
                                structural_texts,
                                padding=True,
                                truncation=True,
                                max_length=128,
                                return_tensors="pt",
                            )
                            struct_ids = struct_encoded["input_ids"].to(device)
                            struct_mask = struct_encoded["attention_mask"].to(device)

                            with torch.no_grad():
                                struct_out = model(struct_ids, struct_mask, adv_lambda=0)
                                z_structural = struct_out["z"].detach()

                            z_orig_struct = out["z"][struct_original_indices]
                            z_structural = z_structural.to(z_orig_struct.dtype)

                            # Push apart: penalize if distance < margin
                            _structural_distances = F.pairwise_distance(z_orig_struct, z_structural)
                            loss_structural = F.relu(STRUCTURAL_CONTRASTIVE_MARGIN - _structural_distances).mean()

                            del struct_ids, struct_mask, struct_out
                except NameError:
                    pass
                except Exception as e:
                    pass

                if not (torch.isnan(loss_structural) or torch.isinf(loss_structural)):
                    try:
                        loss = loss + STRUCTURAL_CONTRASTIVE_WEIGHT * loss_structural
                    except NameError:
                        loss = loss + 0.4 * loss_structural

                # =============================================================
                # v10.16.1: STRUCTURAL CONTRASTIVE LOSS
                # =============================================================
                # Push structural perturbations APART (they change moral meaning)
                loss_structural = torch.tensor(0.0, device=device)
                structural_distances_batch = []
                try:
                    if USE_STRUCTURAL_CONTRASTIVE:
                        batch_texts = batch.get("texts", [])
                        batch_languages = batch.get("languages", [])

                        structural_texts = []
                        original_indices = []

                        for i, (text, lang) in enumerate(zip(batch_texts, batch_languages)):
                            perturbed, _ = create_structural_perturbation(text, lang)
                            if perturbed:
                                structural_texts.append(perturbed)
                                original_indices.append(i)

                        if structural_texts and len(structural_texts) >= 2:
                            struct_encoded = tokenizer(
                                structural_texts,
                                padding=True,
                                truncation=True,
                                max_length=128,
                                return_tensors="pt",
                            )
                            struct_ids = struct_encoded["input_ids"].to(device)
                            struct_mask = struct_encoded["attention_mask"].to(device)

                            with torch.no_grad():
                                struct_out = model(struct_ids, struct_mask, adv_lambda=0)
                                z_structural = struct_out["z"].detach()

                            z_original_struct = out["z"][original_indices]
                            z_structural = z_structural.to(z_original_struct.dtype)

                            # Push apart: penalize if distance < margin
                            distances = F.pairwise_distance(z_original_struct, z_structural)
                            structural_distances_batch = distances.detach()
                            loss_structural = F.relu(STRUCTURAL_CONTRASTIVE_MARGIN - distances).mean()

                            del struct_ids, struct_mask, struct_out, z_structural, z_original_struct
                except NameError:
                    pass

                if not (torch.isnan(loss_structural) or torch.isinf(loss_structural)):
                    try:
                        loss = loss + STRUCTURAL_CONTRASTIVE_WEIGHT * loss_structural
                    except NameError:
                        loss = loss + 0.4 * loss_structural

                # v10.15.1: Surface contrastive loss for invariance
                loss_surface = torch.tensor(0.0, device=device)
                try:
                    if SURFACE_AUGMENT and CONTRASTIVE_WEIGHT > 0:
                        batch_texts = batch.get("texts", [])
                        if batch_texts and len(batch_texts) >= 4:
                            # Create surface-augmented versions
                            augmented_texts = [create_surface_augmented(t) for t in batch_texts]

                            # Tokenize augmented texts
                            aug_encoded = tokenizer(
                                augmented_texts,
                                padding=True,
                                truncation=True,
                                max_length=128,
                                return_tensors="pt",
                            )
                            aug_ids = aug_encoded["input_ids"].to(device)
                            aug_mask = aug_encoded["attention_mask"].to(device)

                            # Get embeddings for augmented texts
                            with torch.no_grad():
                                aug_out = model(aug_ids, aug_mask, adv_lambda=0)
                                z_augmented = aug_out["z"]

                            # Original embeddings (anchor)
                            z_original = out["z"]

                            # Ensure same dtype (AMP: z_original=float16, z_augmented=float32)
                            z_augmented = z_augmented.to(z_original.dtype)

                            # InfoNCE contrastive loss
                            loss_surface = info_nce_loss(
                                z_original, z_augmented, temperature=CONTRASTIVE_TEMPERATURE
                            )

                            # Augment similarity loss (direct MSE)
                            if AUGMENT_SIMILARITY_WEIGHT > 0:
                                sim_loss = F.mse_loss(z_original, z_augmented)
                                loss_surface = loss_surface + AUGMENT_SIMILARITY_WEIGHT * sim_loss

                            # Guard against NaN in contrastive loss
                            if torch.isnan(loss_surface) or torch.isinf(loss_surface):
                                loss_surface = torch.tensor(0.0, device=device)

                            # Cleanup
                            del aug_ids, aug_mask, aug_out, z_augmented
                except NameError:
                    pass  # Config params not defined

                # Only add contrastive loss if valid
                if not (torch.isnan(loss_surface) or torch.isinf(loss_surface)):
                    loss = loss + CONTRASTIVE_WEIGHT * loss_surface

                # =============================================================
                # v10.16.1: TRIPLET LOSS (anchor, surface+, structural-)
                # =============================================================
                loss_triplet = torch.tensor(0.0, device=device)
                try:
                    if USE_TRIPLET_LOSS:
                        # Need both surface and structural perturbations
                        if 'z_augmented' in dir() and z_augmented is not None and len(z_augmented) >= 2:
                            if 'z_structural' in dir() and z_structural is not None and len(z_structural) >= 2:
                                min_len = min(len(out["z"]), len(z_augmented), len(z_structural))
                                if min_len >= 2:
                                    t_anchor = out["z"][:min_len]
                                    t_positive = z_augmented[:min_len].to(t_anchor.dtype)
                                    t_negative = z_structural[:min_len].to(t_anchor.dtype)
                                    loss_triplet = triplet_loss_geometric(t_anchor, t_positive, t_negative, margin=TRIPLET_MARGIN)
                except NameError:
                    pass
                except Exception:
                    pass

                if not (torch.isnan(loss_triplet) or torch.isinf(loss_triplet)):
                    try:
                        loss = loss + TRIPLET_WEIGHT * loss_triplet
                    except NameError:
                        loss = loss + 0.3 * loss_triplet

                # =============================================================
                # v10.16.1: RATIO REGULARIZATION LOSS
                # =============================================================
                loss_ratio = torch.tensor(0.0, device=device)
                try:
                    if USE_RATIO_LOSS:
                        # Get surface distances
                        if 'z_augmented' in dir() and z_augmented is not None and len(z_augmented) >= 2:
                            _surface_distances = F.pairwise_distance(
                                out["z"][:len(z_augmented)],
                                z_augmented.to(out["z"].dtype)
                            )
                        else:
                            _surface_distances = None

                        # Use structural distances computed earlier
                        if '_structural_distances' in dir() and len(_structural_distances) >= 2:
                            _struct_dists = _structural_distances
                        else:
                            _struct_dists = None

                        if _surface_distances is not None and _struct_dists is not None:
                            loss_ratio = ratio_regularization_loss(
                                _surface_distances, _struct_dists, target_ratio=TARGET_RATIO
                            )
                except NameError:
                    pass
                except Exception:
                    pass

                if not (torch.isnan(loss_ratio) or torch.isinf(loss_ratio)):
                    try:
                        loss = loss + RATIO_LOSS_WEIGHT * loss_ratio
                    except NameError:
                        loss = loss + 0.2 * loss_ratio

                # =============================================================
                # v10.16.1: TRIPLET LOSS (anchor, surface+, structural-)
                # =============================================================
                loss_triplet = torch.tensor(0.0, device=device)
                try:
                    if USE_TRIPLET_LOSS and 'z_augmented' in dir() and 'z_structural' in dir():
                        # We have both surface and structural perturbations
                        # Find common indices where we have both
                        if len(original_indices) >= 2 and len(z_augmented) >= 2:
                            # Use minimum overlap
                            min_len = min(len(z_augmented), len(z_structural), len(out["z"]))
                            if min_len >= 2:
                                anchor = out["z"][:min_len]
                                positive = z_augmented[:min_len].to(anchor.dtype)
                                negative = z_structural[:min_len].to(anchor.dtype)

                                loss_triplet = triplet_loss_geometric(
                                    anchor, positive, negative, margin=TRIPLET_MARGIN
                                )
                except NameError:
                    pass
                except Exception:
                    pass  # Skip if dimensions don't match

                if not (torch.isnan(loss_triplet) or torch.isinf(loss_triplet)):
                    try:
                        loss = loss + TRIPLET_WEIGHT * loss_triplet
                    except NameError:
                        loss = loss + 0.3 * loss_triplet

                # =============================================================
                # v10.16.1: RATIO REGULARIZATION LOSS
                # =============================================================
                loss_ratio = torch.tensor(0.0, device=device)
                try:
                    if USE_RATIO_LOSS:
                        # Get surface distances from this batch
                        if 'z_augmented' in dir() and len(z_augmented) >= 2:
                            surface_distances = F.pairwise_distance(
                                out["z"][:len(z_augmented)],
                                z_augmented.to(out["z"].dtype)
                            )
                        else:
                            surface_distances = None

                        # Get structural distances
                        if len(structural_distances_batch) >= 2:
                            structural_distances = structural_distances_batch
                        else:
                            structural_distances = None

                        if surface_distances is not None and structural_distances is not None:
                            loss_ratio = ratio_regularization_loss(
                                surface_distances,
                                structural_distances,
                                target_ratio=TARGET_RATIO
                            )
                except NameError:
                    pass
                except Exception:
                    pass

                if not (torch.isnan(loss_ratio) or torch.isinf(loss_ratio)):
                    try:
                        loss = loss + RATIO_LOSS_WEIGHT * loss_ratio
                    except NameError:
                        loss = loss + 0.2 * loss_ratio

                # v10.15.1.2: Gradient penalty for adversarial heads
                loss_gp = torch.tensor(0.0, device=device)
                if USE_GRADIENT_PENALTY and adv_lambda > 0.1:
                    # Penalize large gradients in adversarial predictions
                    # This encourages smoother, more invariant representations
                    lang_probs = F.softmax(out["language_pred"], dim=-1)
                    period_probs = F.softmax(out["period_pred"], dim=-1)
                    # Entropy penalty: encourage uniform (confused) predictions
                    lang_entropy = -(lang_probs * torch.log(lang_probs + 1e-8)).sum(dim=-1).mean()
                    period_entropy = (
                        -(period_probs * torch.log(period_probs + 1e-8)).sum(dim=-1).mean()
                    )
                    # Maximize entropy = minimize negative entropy
                    loss_gp = -GRADIENT_PENALTY_WEIGHT * (lang_entropy + period_entropy)

                loss = loss + loss_gp

                # v10.15.1.4: NaN detection
                if torch.isnan(loss) or torch.isinf(loss):
                    print("    NaN/Inf loss detected - skipping batch")
                    optimizer.zero_grad()
                    continue

                # v10.15.1.4: Gradient accumulation
                loss_scaled = loss / _grad_accum

                if USE_AMP and scaler:
                    scaler.scale(loss_scaled).backward()
                    if (batch_count + 1) % _grad_accum == 0:
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                        scaler.step(optimizer)
                        scaler.update()
                        optimizer.zero_grad()
                else:
                    loss_scaled.backward()
                    if (batch_count + 1) % _grad_accum == 0:
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                        optimizer.step()
                        optimizer.zero_grad()

                batch_count += 1
                total_loss += loss.item()
                n_batches += 1
                _nan_batch_count = 0  # Reset NaN counter on successful batch

                # Delete intermediate tensors to prevent memory accumulation
                del input_ids, attention_mask, bond_labels, language_labels, period_labels
                del out, loss, loss_bond, loss_lang, loss_period
                if USE_CONFIDENCE_WEIGHTING:
                    del sample_weights
                if USE_CONTEXT_AUXILIARY:
                    del context_labels, loss_context
                if USE_ROLE_AUGMENTATION:
                    del loss_role

                # Periodic memory cleanup every 50 batches
                if n_batches % 50 == 0:
                    gc.collect()
                    torch.cuda.empty_cache()

            if n_batches == 0:
                print(f"Epoch {epoch}: No valid batches! All had NaN loss.")
                continue

            avg_loss = total_loss / n_batches

            # Aggressive memory cleanup after each epoch
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

            if torch.cuda.is_available():
                mem_alloc = torch.cuda.memory_allocated() / 1e9
                mem_reserved = torch.cuda.memory_reserved() / 1e9
                print(
                    f"Epoch {epoch}: Loss={avg_loss:.4f} (adv_lambda={adv_lambda:.2f}) [GPU: {mem_alloc:.1f}GB alloc, {mem_reserved:.1f}GB reserved]"
                )
            else:
                print(f"Epoch {epoch}: Loss={avg_loss:.4f} (adv_lambda={adv_lambda:.2f})")

            # v10.16.2: Quick language accuracy check every few epochs
            if epoch % 3 == 0 or epoch == N_EPOCHS:
                model.eval()
                _sample_preds = []
                _sample_labels = []
                with torch.no_grad():
                    for _sb in list(test_loader)[:5]:  # Sample 5 batches
                        _sout = model(_sb["input_ids"].to(device), _sb["attention_mask"].to(device), 0)
                        _sample_preds.extend(_sout["language_pred"].argmax(-1).cpu().tolist())
                        _sample_labels.extend(_sb["language_labels"].tolist())
                _lang_acc_sample = sum(p == l for p, l in zip(_sample_preds, _sample_labels)) / len(_sample_preds) if _sample_preds else 0
                model.train()
                print(f"  -> lang_acc={_lang_acc_sample:.1%} (target: <20%)")

            # v10.15.1.2: Step LR scheduler
            if USE_COSINE_LR and scheduler:
                scheduler.step()

            # Save checkpoint every epoch (for crash recovery)
            checkpoint = {
                "epoch": epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "loss": avg_loss,
                "best_loss": best_loss,
            }
            torch.save(checkpoint, f"models/checkpoints/latest_{split_name}.pt")

            if avg_loss < best_loss:
                best_loss = avg_loss
                torch.save(model.state_dict(), f"models/checkpoints/best_{split_name}.pt")
                torch.save(model.state_dict(), f"{SAVE_DIR}/best_{split_name}.pt")
                no_improve_count = 0
            else:
                no_improve_count += 1
                if early_stop_patience and no_improve_count >= early_stop_patience:
                    print(f"Early stopping: no improvement for {no_improve_count} epochs")
                    break

        # Evaluate
        print("\nEvaluating...")
        model.load_state_dict(torch.load(f"models/checkpoints/best_{split_name}.pt"))
        model.eval()

        # v10.15.1.4: Clear memory and use smaller batch for testing
        torch.cuda.empty_cache()
        import gc

        gc.collect()

        # Recreate test loader with smaller batch to avoid OOM
        test_batch = min(32, actual_batch)
        test_loader = DataLoader(
            test_dataset,
            batch_size=test_batch,
            shuffle=False,
            collate_fn=collate_fn,
            num_workers=0,
            pin_memory=True,
        )
        print(f"  Testing with batch size {test_batch}")

        all_preds = {"bond": [], "lang": []}
        all_labels = {"bond": [], "lang": []}
        all_languages = []

        with torch.no_grad():
            for batch in tqdm(test_loader, desc="Testing"):
                out = model(batch["input_ids"].to(device), batch["attention_mask"].to(device), 0)
                all_preds["bond"].extend(out["bond_pred"].argmax(-1).cpu().tolist())
                all_preds["lang"].extend(out["language_pred"].argmax(-1).cpu().tolist())
                all_labels["bond"].extend(batch["bond_labels"].tolist())
                all_labels["lang"].extend(batch["language_labels"].tolist())
                all_languages.extend(batch["languages"])

        bond_f1 = f1_score(all_labels["bond"], all_preds["bond"], average="macro", zero_division=0)
        bond_acc = sum(p == l for p, l in zip(all_preds["bond"], all_labels["bond"])) / len(
            all_preds["bond"]
        )
        lang_acc = sum(p == l for p, l in zip(all_preds["lang"], all_labels["lang"])) / len(
            all_preds["lang"]
        )

        # Per-language F1
        lang_f1 = {}
        for lang in set(all_languages):
            mask = [l == lang for l in all_languages]
            if sum(mask) > 10:
                preds = [p for p, m in zip(all_preds["bond"], mask) if m]
                labels = [l for l, m in zip(all_labels["bond"], mask) if m]
                lang_f1[lang] = {
                    "f1": f1_score(labels, preds, average="macro", zero_division=0),
                    "n": sum(mask),
                }

        all_results[split_name] = {
            "bond_f1_macro": bond_f1,
            "bond_acc": bond_acc,
            "language_acc": lang_acc,
            "per_language_f1": lang_f1,
            "training_time": time.time() - split_start,
        }

        print(f"\n{split_name} RESULTS:")
        print(f"  Bond F1 (macro): {bond_f1:.3f} ({bond_f1 / 0.1:.1f}x chance)")
        print(f"  Bond accuracy:   {bond_acc:.1%}")
        print(f"  Language acc:    {lang_acc:.1%} (want ~20% = invariant)")
        print("  Per-language:")
        for lang, m in sorted(lang_f1.items(), key=lambda x: -x[1]["n"]):
            print(f"    {lang:20s}: F1={m['f1']:.3f} (n={m['n']:,})")

        # Context analysis
        high_conf = sum(1 for c in test_dataset.data if c["confidence"] == "high")
        prescriptive = sum(1 for c in test_dataset.data if c["context"] == "prescriptive")
        print(
            f"  Context: {prescriptive:,}/{len(test_dataset):,} prescriptive ({prescriptive / len(test_dataset) * 100:.1f}%)"
        )
        print(
            f"  High confidence: {high_conf:,}/{len(test_dataset):,} ({high_conf / len(test_dataset) * 100:.1f}%)"
        )

        # GPU memory usage before cleanup
        if torch.cuda.is_available():
            mem = torch.cuda.memory_allocated() / 1e9
            print(
                f"\n  GPU memory (before cleanup): {mem:.1f} GB / {VRAM_GB:.1f} GB ({mem / VRAM_GB * 100:.0f}%)"
            )

        # Aggressive memory cleanup between splits
        # Step 1: Zero out gradients to release gradient memory
        model.zero_grad(set_to_none=True)
        for param in model.parameters():
            param.grad = None

        # Step 2: Clear optimizer state (can hold significant memory)
        optimizer.zero_grad(set_to_none=True)
        optimizer_state = optimizer.state
        for state in optimizer_state.values():
            for k, v in list(state.items()):
                if isinstance(v, torch.Tensor):
                    state[k] = None

        # Step 3: Move model to CPU to release GPU memory
        model.cpu()

        # Step 4: Delete all references
        del model, train_dataset, test_dataset, train_loader, test_loader, optimizer
        if USE_AMP and scaler:
            del scaler

        # Step 5: Force garbage collection (multiple passes)
        for _ in range(5):
            gc.collect()

        # Step 6: Clear CUDA cache and reset memory stats
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            torch.cuda.reset_peak_memory_stats()

            # If memory is still high, try more aggressive cleanup
            mem_check = torch.cuda.memory_allocated() / 1e9
            if mem_check > 2.0:
                print(f"  Memory still high ({mem_check:.1f}GB), attempting deeper cleanup...")
                # Clear all cached allocations
                torch.cuda.memory._dump_snapshot = lambda: None  # Disable snapshot if enabled
                gc.collect()
                torch.cuda.empty_cache()
                torch.cuda.ipc_collect()

        # Step 7: Re-create scaler for next split
        if USE_AMP:
            scaler = torch.amp.GradScaler("cuda")

        # GPU memory after cleanup
        if torch.cuda.is_available():
            mem_after = torch.cuda.memory_allocated() / 1e9
            print(
                f"  GPU memory (after cleanup): {mem_after:.1f} GB (freed {mem - mem_after:.1f} GB)"
            )
            if mem_after > 1.0:
                print(
                    f"  WARNING: {mem_after:.1f} GB still allocated - may cause OOM on next split"
                )
                print("  Consider running with BACKBONE='MiniLM' for lower memory usage")

    print("\n" + "=" * 60)
    print("TRAINING COMPLETE")
    print("=" * 60)

Cleaning up GPU memory from previous runs...
  GPU memory: 4.02 GB allocated, 7.72 GB reserved

Using fixed seed: 42
TRAINING BIP MODEL

Encoder mode: UNFROZEN after epoch 2

Settings:
  Backbone:     LaBSE
  GPU Tier:     L4/A100
  Batch size:   4096
  Workers:      4
  Learning rate: 3.20e-04
  Adv weights:  lang=1.0, period=0.8
  (v10.16.4: Increased for stronger invariance - grad clipping prevents explosion)
  Confusion loss: True (weight=2.0)
  (v10.16.5: Forces uniform predictions - prevents adversarial head evasion)
  VIB: True (beta=0.001, anneal=5 epochs)
  (v10.16.8: Information bottleneck - compresses away spurious info)
  Confidence weighting: True
  Context auxiliary: True (weight=0.33)
  Strict prescriptive test: False
  Role augmentation: True (prob=0.3, weight=0.2)

Training 11 splits: ['hebrew_to_others', 'semitic_to_indic', 'confucian_to_buddhist', 'ancient_to_modern', 'east_to_west', 'semitic_to_chinese', 'jewish_to_islamic', 'stoic_to_confucian', 'daoist_to_buddhist

Loading: 0line [00:00, ?line/s]

  Loaded 4,325 samples (filtered 3,660 NONE bonds)


Loading: 0line [00:00, ?line/s]

  Loaded 33,298 samples (filtered 4,875 NONE bonds)
  Batch cap: dataset=4,325 -> hard_cap=2048
Actual batch size: 216
  Optimizer: AdamW (encoder_lr=1e-06, head_lr=1e-03)
  Using cosine LR schedule: 3.20e-04 -> 3.20e-05
  Found checkpoint, checking validity...
  Resuming from epoch 16, best_loss=8.0881

Evaluating...
  Testing with batch size 32


...

Epoch 11: Loss=4.6454 (adv_lambda=1.50) [GPU: 4.2GB alloc, 7.7GB reserved]
  Encoder LR warmup: 1.0e-06 (5/5)


Epoch 12:   0%|          | 0/2469 [00:00<?, ?it/s]

Epoch 12: Loss=4.5939 (adv_lambda=1.50) [GPU: 4.2GB alloc, 7.7GB reserved]
  -> lang_acc=26.9% (target: <20%)
Early stopping: no improvement for 5 epochs

Evaluating...
  Testing with batch size 32


Testing:   0%|          | 0/1036 [00:00<?, ?it/s]


mixed_baseline RESULTS:
  Bond F1 (macro): 0.272 (2.7x chance)
  Bond accuracy:   62.6%
  Language acc:    98.3% (want ~20% = invariant)
  Per-language:
    english             : F1=0.240 (n=31,272)
    sanskrit            : F1=0.140 (n=655)
    hebrew              : F1=0.091 (n=385)
    pali                : F1=0.087 (n=307)
    classical_chinese   : F1=0.068 (n=262)
    arabic              : F1=0.079 (n=234)
    aramaic             : F1=0.167 (n=29)
  Context: 461/33,144 prescriptive (1.4%)
  High confidence: 0/33,144 (0.0%)

  GPU memory (before cleanup): 4.2 GB / 23.8 GB (18%)
  Memory still high (4.0GB), attempting deeper cleanup...
  GPU memory (after cleanup): 4.0 GB (freed 0.2 GB)
  Consider running with BACKBONE='MiniLM' for lower memory usage

TRAINING COMPLETE


In [ ]:
# @title 8. Geometric Analysis & Linear Probe { display-mode: "form" }
# @markdown v10.9: New geometric analysis module + linear probe test
# @markdown Tests latent space structure (axis discovery, role swap analysis)
# @markdown Tests if z_bond encodes language/period (should be low = invariant)


import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


# ===== v10.9: GEOMETRIC ANALYZER CLASS =====
class GeometricAnalyzer:
    """
    Probe the latent space geometry to discover moral structure.
    """

    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device

    @torch.no_grad()
    def get_embedding(self, text: str) -> np.ndarray:
        inputs = self.tokenizer(
            text, return_tensors="pt", truncation=True, max_length=128, padding="max_length"
        )
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        z = self.model.get_bond_embedding(inputs["input_ids"], inputs["attention_mask"])
        return z.cpu().numpy().flatten()

    def find_direction(self, positive_texts: list[str], negative_texts: list[str]) -> np.ndarray:
        """
        Find the direction in z-space that separates two concepts.
        E.g., obligation vs permission, harm vs care.
        """
        pos_embs = np.array([self.get_embedding(t) for t in positive_texts])
        neg_embs = np.array([self.get_embedding(t) for t in negative_texts])

        pos_mean = pos_embs.mean(axis=0)
        neg_mean = neg_embs.mean(axis=0)

        direction = pos_mean - neg_mean
        direction = direction / (np.linalg.norm(direction) + 1e-9)
        return direction

    def test_direction_transfer(
        self, direction: np.ndarray, test_pairs: list[tuple[str, str]]
    ) -> float:
        """
        Test if a direction generalizes to new examples.
        Returns accuracy of direction-based classification.
        """
        scores = []
        for pos_text, neg_text in test_pairs:
            pos_proj = np.dot(self.get_embedding(pos_text), direction)
            neg_proj = np.dot(self.get_embedding(neg_text), direction)
            scores.append(1.0 if pos_proj > neg_proj else 0.0)
        return np.mean(scores)

    def pca_on_pairs(self, concept_pairs: dict[str, list[tuple[str, str]]]) -> dict:
        """
        Run PCA on difference vectors to find dominant axes.

        concept_pairs: {"obligation_permission": [(obl1, perm1), ...], ...}
        """
        all_diffs = []
        labels = []

        for concept, pairs in concept_pairs.items():
            for pos, neg in pairs:
                diff = self.get_embedding(pos) - self.get_embedding(neg)
                all_diffs.append(diff)
                labels.append(concept)

        X = np.array(all_diffs)

        pca = PCA(n_components=min(10, len(X)))
        pca.fit(X)

        return {
            "components": pca.components_,
            "explained_variance_ratio": pca.explained_variance_ratio_,
            "labels": labels,
            "transformed": pca.transform(X),
        }

    def role_swap_analysis(self, agent_patient_pairs: list[tuple[str, str]]) -> dict:
        """
        Test if swapping agent/patient produces consistent transformation.

        agent_patient_pairs: [("A harmed B", "B harmed A"), ...]
        """
        transformations = []

        for original, swapped in agent_patient_pairs:
            orig_emb = self.get_embedding(original)
            swap_emb = self.get_embedding(swapped)
            transformations.append(swap_emb - orig_emb)

        T = np.array(transformations)

        # Check consistency: are all transformations similar?
        mean_transform = T.mean(axis=0)
        cosines = [
            np.dot(t, mean_transform) / (np.linalg.norm(t) * np.linalg.norm(mean_transform) + 1e-9)
            for t in T
        ]

        return {
            "mean_transform": mean_transform,
            "consistency": np.mean(cosines),
            "consistency_std": np.std(cosines),
        }


print("=" * 60)
print("LINEAR PROBE TEST")
print("=" * 60)
print("\nIf probe accuracy is NEAR CHANCE, representation is INVARIANT")
print("(This is what we want for BIP)")

probe_results = {}

for split_name in ["hebrew_to_others", "semitic_to_non_semitic"]:
    model_path = f"{SAVE_DIR}/best_{split_name}.pt"
    if not os.path.exists(model_path):
        print(f"\nSkipping {split_name} - no saved model")
        continue

    print(f"\n{'=' * 50}")
    print(f"PROBE: {split_name}")
    print("=" * 50)

    model = BIPModel(z_dim=Z_DIM).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    test_ids = set(all_splits[split_name]["test_ids"][:5000])
    test_dataset = NativeDataset(
        test_ids, "data/processed/passages.jsonl", "data/processed/bonds.jsonl", tokenizer
    )

    if len(test_dataset) < 50:
        print(f"  Skip - only {len(test_dataset)} samples")
        continue

    test_loader = DataLoader(
        test_dataset,
        batch_size=get_probed_batch(model, tokenizer, device, mode="eval"),
        collate_fn=collate_fn,
        num_workers=0,
    )

    all_z, all_lang, all_period = [], [], []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Extract"):
            out = model(batch["input_ids"].to(device), batch["attention_mask"].to(device), 0)
            all_z.append(out["z"].cpu().numpy())
            all_lang.extend(batch["language_labels"].tolist())
            all_period.extend(batch["period_labels"].tolist())

    X = np.vstack(all_z)
    y_lang = np.array(all_lang)
    y_period = np.array(all_period)

    scaler_probe = StandardScaler()
    X_scaled = scaler_probe.fit_transform(X)

    # Train/test split for probes
    n = len(X)
    idx = np.random.permutation(n)
    train_idx, test_idx = idx[: int(0.7 * n)], idx[int(0.7 * n) :]

    # Language probe - check for multiple classes
    unique_langs = np.unique(y_lang[test_idx])
    if len(unique_langs) < 2:
        print(f"  SKIP language probe - only {len(unique_langs)} class")
        lang_acc = 1.0 / max(1, len(np.unique(y_lang)))
        lang_chance = lang_acc
    else:
        lang_probe = LogisticRegression(max_iter=1000, n_jobs=-1)
        lang_probe.fit(X_scaled[train_idx], y_lang[train_idx])
        lang_acc = (lang_probe.predict(X_scaled[test_idx]) == y_lang[test_idx]).mean()
        lang_chance = 1.0 / len(unique_langs)

    # Period probe - same check
    unique_periods = np.unique(y_period[test_idx])
    if len(unique_periods) < 2:
        print(f"  SKIP period probe - only {len(unique_periods)} class")
        period_acc = 1.0 / max(1, len(np.unique(y_period)))
        period_chance = period_acc
    else:
        period_probe = LogisticRegression(max_iter=1000, n_jobs=-1)
        period_probe.fit(X_scaled[train_idx], y_period[train_idx])
        period_acc = (period_probe.predict(X_scaled[test_idx]) == y_period[test_idx]).mean()
        period_chance = 1.0 / len(unique_periods)

    lang_status = "INVARIANT" if lang_acc < lang_chance + 0.15 else "NOT invariant"
    period_status = "INVARIANT" if period_acc < period_chance + 0.15 else "NOT invariant"

    probe_results[split_name] = {
        "language_acc": lang_acc,
        "language_chance": lang_chance,
        "language_status": lang_status,
        "period_acc": period_acc,
        "period_chance": period_chance,
        "period_status": period_status,
    }

    print("\nRESULTS:")
    print(f"  Language: {lang_acc:.1%} (chance: {lang_chance:.1%}) -> {lang_status}")
    print(f"  Period:   {period_acc:.1%} (chance: {period_chance:.1%}) -> {period_status}")

    del model
    torch.cuda.empty_cache()

print("\n" + "=" * 60)
print("Probe tests complete")
print("=" * 60)

# ===== v10.9: GEOMETRIC ANALYSIS =====
print("\n" + "=" * 60)
print("GEOMETRIC ANALYSIS (v10.9)")
print("=" * 60)
print("\nDiscovering interpretable axes in latent space...")

# Test pairs for axis discovery (cross-lingual)
OBLIGATION_PERMISSION_TRAIN = [
    # English - training set
    ("You must help the elderly", "You may help the elderly"),
    ("He is required to pay", "He is allowed to pay"),
    ("Parents must protect children", "Parents may protect children"),
]

OBLIGATION_PERMISSION_TEST = [
    # Chinese
    ("君子必孝", "君子可孝"),  # Gentleman must/may be filial
    ("民必從法", "民可從法"),  # People must/may follow law
    # Arabic
    ("يجب عليك أن تساعد", "يجوز لك أن تساعد"),  # You must/may help
    # Hebrew
    ("חייב לכבד", "מותר לכבד"),  # Obligated/permitted to honor
    # English - held out
    ("She must attend", "She may attend"),
]

HARM_CARE_PAIRS = [
    ("He injured the child", "He protected the child"),
    ("殺人者", "救人者"),  # One who kills / one who saves
    ("ظلم الضعيف", "رحم الضعيف"),  # Oppressed / showed mercy to the weak
    ("She hurt the patient", "She healed the patient"),
]

ROLE_SWAP_PAIRS = [
    ("The master commands the servant", "The servant commands the master"),
    ("君命臣", "臣命君"),  # Lord commands minister / minister commands lord
    ("الأب يأمر الابن", "الابن يأمر الأب"),  # Father commands son / son commands father
    ("The parent guides the child", "The child guides the parent"),
]

geometry_results = {}

# Use the best model from mixed_baseline split for geometric analysis
model_path = f"{SAVE_DIR}/best_mixed_baseline.pt"
if os.path.exists(model_path):
    print("\nLoading model for geometric analysis...")
    model = BIPModel(z_dim=Z_DIM).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    analyzer = GeometricAnalyzer(model, tokenizer, device)

    # 1. Find obligation/permission axis
    print("\n--- Obligation/Permission Axis ---")
    obl_texts = [p[0] for p in OBLIGATION_PERMISSION_TRAIN]
    perm_texts = [p[1] for p in OBLIGATION_PERMISSION_TRAIN]
    obl_perm_axis = analyzer.find_direction(obl_texts, perm_texts)

    # Test transfer to other languages
    transfer_acc = analyzer.test_direction_transfer(obl_perm_axis, OBLIGATION_PERMISSION_TEST)
    print("  Direction found from English training pairs")
    print(f"  Transfer accuracy to other languages: {transfer_acc:.1%}")
    axis_status = "STRONG" if transfer_acc > 0.8 else "WEAK" if transfer_acc > 0.5 else "FAILED"
    print(f"  Status: {axis_status} deontic axis")

    geometry_results["obligation_permission"] = {
        "transfer_accuracy": transfer_acc,
        "status": axis_status,
    }

    # 2. Find harm/care axis
    print("\n--- Harm/Care Axis ---")
    harm_texts = [p[0] for p in HARM_CARE_PAIRS]
    care_texts = [p[1] for p in HARM_CARE_PAIRS]
    harm_care_axis = analyzer.find_direction(harm_texts, care_texts)

    # Check axis orthogonality
    axis_correlation = abs(np.dot(obl_perm_axis, harm_care_axis))
    print("  Axis found")
    print(f"  Correlation with obl/perm axis: {axis_correlation:.3f}")
    orthogonal = "ORTHOGONAL" if axis_correlation < 0.3 else "CORRELATED"
    print(f"  Status: {orthogonal}")

    geometry_results["harm_care"] = {
        "axis_correlation": axis_correlation,
        "orthogonal": axis_correlation < 0.3,
    }

    # 3. Role swap analysis
    print("\n--- Role Swap Analysis ---")
    role_analysis = analyzer.role_swap_analysis(ROLE_SWAP_PAIRS)
    print(
        f"  Mean consistency: {role_analysis['consistency']:.3f} +/- {role_analysis['consistency_std']:.3f}"
    )
    role_status = "CONSISTENT" if role_analysis["consistency"] > 0.9 else "VARIABLE"
    print(f"  Status: {role_status} agent/patient transformation")

    geometry_results["role_swap"] = {
        "consistency": role_analysis["consistency"],
        "consistency_std": role_analysis["consistency_std"],
        "status": role_status,
    }

    # 4. PCA on all structural pairs
    print("\n--- PCA Analysis ---")
    all_concept_pairs = {
        "obligation_permission": OBLIGATION_PERMISSION_TRAIN + OBLIGATION_PERMISSION_TEST,
        "harm_care": HARM_CARE_PAIRS,
    }
    pca_results = analyzer.pca_on_pairs(all_concept_pairs)

    cumsum = np.cumsum(pca_results["explained_variance_ratio"])
    n_components_90 = np.argmax(cumsum > 0.9) + 1 if any(cumsum > 0.9) else len(cumsum)

    print(f"  Explained variance ratio: {pca_results['explained_variance_ratio'][:5]}")
    print(f"  Components for 90% variance: {n_components_90}")
    pca_status = "LOW-DIM" if n_components_90 <= 3 else "HIGH-DIM"
    print(f"  Status: {pca_status} moral structure")

    geometry_results["pca"] = {
        "explained_variance": pca_results["explained_variance_ratio"].tolist(),
        "n_components_90pct": n_components_90,
        "status": pca_status,
    }

    del model
    torch.cuda.empty_cache()
else:
    print(f"\nSkipping geometric analysis - no model at {model_path}")
    geometry_results = {"error": "No model available"}

print("\n" + "=" * 60)
print("Geometric analysis complete")
print("=" * 60)

LINEAR PROBE TEST

If probe accuracy is NEAR CHANCE, representation is INVARIANT
(This is what we want for BIP)

PROBE: hebrew_to_others
  Loading encoder: sentence-transformers/LaBSE
  Encoder UNFROZEN (full fine-tuning)
  VIB: Stochastic z with KL regularization
  Adversarial heads: 4 independent heads (v10.16.7 multi-head)
    Base config: 4 layers, 1024 hidden, 0.4 dropout
  Total params: 485,346,201
  Trainable: 485,346,201


Loading: 0line [00:00, ?line/s]

  Loaded 23,216 samples (filtered 1,337 NONE bonds)
  [v10.15.1] Probing max batch (mode=eval, trainable=False)... max=1024, using 819


Extract:   0%|          | 0/29 [00:00<?, ?it/s]


RESULTS:
  Language: 99.8% (chance: 16.7%) -> NOT invariant
  Period:   95.3% (chance: 14.3%) -> NOT invariant

Skipping semitic_to_non_semitic - no saved model

Probe tests complete

GEOMETRIC ANALYSIS (v10.9)

Discovering interpretable axes in latent space...

Loading model for geometric analysis...
  Loading encoder: sentence-transformers/LaBSE
  Encoder UNFROZEN (full fine-tuning)
  VIB: Stochastic z with KL regularization
  Adversarial heads: 4 independent heads (v10.16.7 multi-head)
    Base config: 4 layers, 1024 hidden, 0.4 dropout
  Total params: 485,346,201
  Trainable: 485,346,201

--- Obligation/Permission Axis ---
  Direction found from English training pairs
  Transfer accuracy to other languages: 100.0%
  Status: STRONG deontic axis

--- Harm/Care Axis ---
  Axis found
  Correlation with obl/perm axis: 0.005
  Status: ORTHOGONAL

--- Role Swap Analysis ---
  Mean consistency: 0.733 +/- 0.077
  Status: VARIABLE agent/patient transformation

--- PCA Analysis ---
  Explain

In [1]:
# @title 9. Fuzz Testing v10.12: Structural vs Surface Perturbations { display-mode: "form" }
# @markdown Tests whether structural perturbations move embeddings more than surface perturbations.
# @markdown **Run immediately after Cell 6/7 training completes - uses model in memory.**
# @markdown
# @markdown v10.12 enhancements:
# @markdown - **30+ samples per category** for 6-sigma statistical confidence
# @markdown - **Runtime-adaptive thresholds** based on GPU type (L4/A100/T4)
# @markdown - **Extended bond type coverage** including cross-cultural scenarios
# @markdown - **Bootstrap confidence intervals** for robust statistics

# @markdown ---
# @markdown ## Enable Fuzz Testing
RUN_FUZZ_TEST = True  # @param {type:"boolean"}

import random
import warnings

import numpy as np
import torch
import torch.nn.functional as F

# ============================================================================
# RUNTIME DETECTION AND ADAPTIVE THRESHOLDS
# ============================================================================


def detect_runtime() -> dict:
    """Detect GPU type and set appropriate thresholds."""
    runtime_config = {
        "gpu_type": "unknown",
        "vram_gb": 0,
        "batch_size": 16,
        "max_scenarios": 50,
        "bootstrap_samples": 1000,
    }

    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0).lower()
        vram = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        runtime_config["vram_gb"] = vram

        if "a100" in gpu_name:
            runtime_config["gpu_type"] = "A100"
            runtime_config["batch_size"] = 64
            runtime_config["max_scenarios"] = 100
            runtime_config["bootstrap_samples"] = 5000
        elif "l4" in gpu_name:
            runtime_config["gpu_type"] = "L4"
            runtime_config["batch_size"] = 32
            runtime_config["max_scenarios"] = 75
            runtime_config["bootstrap_samples"] = 2000
        elif "t4" in gpu_name:
            runtime_config["gpu_type"] = "T4"
            runtime_config["batch_size"] = 16
            runtime_config["max_scenarios"] = 50
            runtime_config["bootstrap_samples"] = 1000
        elif "v100" in gpu_name:
            runtime_config["gpu_type"] = "V100"
            runtime_config["batch_size"] = 32
            runtime_config["max_scenarios"] = 60
            runtime_config["bootstrap_samples"] = 2000
        else:
            # Default based on VRAM
            if vram >= 40:
                runtime_config["gpu_type"] = "high_vram"
                runtime_config["batch_size"] = 64
                runtime_config["max_scenarios"] = 100
            elif vram >= 20:
                runtime_config["gpu_type"] = "medium_vram"
                runtime_config["batch_size"] = 32
                runtime_config["max_scenarios"] = 75
            else:
                runtime_config["gpu_type"] = "low_vram"
                runtime_config["batch_size"] = 16
                runtime_config["max_scenarios"] = 40

    return runtime_config


# Ensure directories exist
import os

os.makedirs("models/checkpoints", exist_ok=True)
os.makedirs("models", exist_ok=True)

if not RUN_FUZZ_TEST:
    print("Fuzz testing disabled. Check RUN_FUZZ_TEST to enable.")
else:
    print("=" * 70)
    print("FUZZ TESTING v10.12: STRUCTURAL VS SURFACE PERTURBATIONS")
    print("=" * 70)
    print()

    # Detect runtime and set thresholds
    RUNTIME = detect_runtime()
    print(f"Runtime detected: {RUNTIME['gpu_type']} ({RUNTIME['vram_gb']:.1f} GB VRAM)")
    print(f"Batch size: {RUNTIME['batch_size']}, Max scenarios: {RUNTIME['max_scenarios']}")
    print(f"Bootstrap samples: {RUNTIME['bootstrap_samples']}")
    print()

    # ========================================================================
    # USE EXISTING MODEL FROM TRAINING SESSION
    # ========================================================================

    # ========================================================================
    # MODEL LOADING WITH CHECKPOINT FALLBACK
    # ========================================================================

    _fuzz_model = None
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # First try: Use model from memory
    try:
        if "model" in dir() and model is not None:
            if hasattr(model, "module"):
                _fuzz_model = model.module
                print("Using unwrapped model from Accelerator")
            else:
                _fuzz_model = model
                print("Using model from training session")
    except NameError:
        pass

    # Second try: Load from checkpoint
    if _fuzz_model is None:
        import glob
        import os

        # Look for checkpoint files
        checkpoint_patterns = [
            f"{SAVE_DIR}/best_*.pt",  # v10.15.1: Check Drive first
            "models/checkpoints/best_*.pt",
            "models/best_*.pt",
            "*.pt",
        ]

        checkpoint_path = None
        for pattern in checkpoint_patterns:
            matches = glob.glob(pattern)
            if matches:
                # Use most recent
                checkpoint_path = max(matches, key=os.path.getmtime)
                break

        if checkpoint_path and os.path.exists(checkpoint_path):
            print(f"Loading model from checkpoint: {checkpoint_path}")

            # Need to create model first
            try:
                # Try to use existing tokenizer
                if "tokenizer" not in dir():
                    from transformers import AutoTokenizer

                    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

                # Create model architecture
                _fuzz_model = BIPModel(z_dim=Z_DIM)
                _fuzz_model.load_state_dict(torch.load(checkpoint_path, map_location=device))
                _fuzz_model.to(device)
                print(f"Model loaded successfully from {checkpoint_path}")

            except Exception as e:
                print(f"Error loading checkpoint: {e}")
                _fuzz_model = None
        else:
            print("No checkpoint files found in:")
            for pattern in checkpoint_patterns:
                print(f"  - {pattern}")

    # Final check
    if _fuzz_model is None:
        print()
        print("ERROR: No model found in memory and no checkpoint available!")
        print("Please run training (Cell 7) first.")
        print()
        print("Expected checkpoint locations:")
        print("  models/checkpoints/best_mixed_baseline.pt")
        print("  models/checkpoints/best_ancient_to_modern.pt")
        print("  etc.")
        RUN_FUZZ_TEST = False
    else:
        _fuzz_model.eval()
        try:
            device = next(_fuzz_model.parameters()).device
        except StopIteration:
            pass
        print(f"Device: {device}")

    if RUN_FUZZ_TEST:
        print()

        # ====================================================================
        # EMBEDDING FUNCTIONS
        # ====================================================================

        @torch.no_grad()
        def get_embedding(text: str) -> np.ndarray:
            inputs = tokenizer(
                text, return_tensors="pt", truncation=True, max_length=128, padding="max_length"
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}
            z = _fuzz_model.get_bond_embedding(inputs["input_ids"], inputs["attention_mask"])
            return z.cpu().numpy().flatten()

        @torch.no_grad()
        def get_embeddings_batch(texts: list[str]) -> np.ndarray:
            """Batch embedding for efficiency."""
            all_embeddings = []
            batch_size = RUNTIME["batch_size"]

            for i in range(0, len(texts), batch_size):
                batch = texts[i : i + batch_size]
                inputs = tokenizer(
                    batch,
                    return_tensors="pt",
                    truncation=True,
                    max_length=128,
                    padding="max_length",
                )
                inputs = {k: v.to(device) for k, v in inputs.items()}
                z = _fuzz_model.get_bond_embedding(inputs["input_ids"], inputs["attention_mask"])
                all_embeddings.append(z.cpu().numpy())

            return np.vstack(all_embeddings)

        def cosine_distance(v1: np.ndarray, v2: np.ndarray) -> float:
            sim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-9)
            return 1 - sim

        # ====================================================================
        # EXPANDED BASE SCENARIOS (30+ for statistical power)
        # ====================================================================

        BASE_SCENARIOS = [
            # OBLIGATION / DUTY (8 scenarios)
            {
                "text": "John borrowed money from Mary and promised to repay it by Friday.",
                "bond_type": "OBLIGATION",
                "category": "promise",
            },
            {
                "text": "The doctor has a duty to keep patient information confidential.",
                "bond_type": "DUTY",
                "category": "professional",
            },
            {
                "text": "Parents must protect their children from harm.",
                "bond_type": "DUTY",
                "category": "familial",
            },
            {
                "text": "The teacher promised to grade all exams by Monday.",
                "bond_type": "OBLIGATION",
                "category": "promise",
            },
            {
                "text": "Soldiers are required to follow orders from superior officers.",
                "bond_type": "DUTY",
                "category": "institutional",
            },
            {
                "text": "The witness swore to tell the truth in court.",
                "bond_type": "OBLIGATION",
                "category": "oath",
            },
            {
                "text": "Citizens must pay their taxes to the government.",
                "bond_type": "DUTY",
                "category": "civic",
            },
            {
                "text": "The contractor agreed to complete the building within six months.",
                "bond_type": "OBLIGATION",
                "category": "contract",
            },
            # CARE / HELP (8 scenarios)
            {
                "text": "Sarah helped her neighbor carry groceries, expecting nothing in return.",
                "bond_type": "CARE",
                "category": "altruism",
            },
            {
                "text": "The nurse stayed late to comfort the dying patient.",
                "bond_type": "CARE",
                "category": "compassion",
            },
            {
                "text": "She donated her savings to help earthquake victims.",
                "bond_type": "CARE",
                "category": "charity",
            },
            {
                "text": "The mentor guided the young artist without asking for payment.",
                "bond_type": "CARE",
                "category": "guidance",
            },
            {
                "text": "He gave his coat to the homeless man shivering in the cold.",
                "bond_type": "CARE",
                "category": "generosity",
            },
            {
                "text": "The stranger stopped to help change the flat tire.",
                "bond_type": "CARE",
                "category": "assistance",
            },
            {
                "text": "She listened patiently as he shared his troubles.",
                "bond_type": "CARE",
                "category": "empathy",
            },
            {
                "text": "The community gathered to rebuild the family's burned house.",
                "bond_type": "CARE",
                "category": "solidarity",
            },
            # HARM / VIOLATION (8 scenarios)
            {
                "text": "He stole the wallet from the elderly woman.",
                "bond_type": "HARM",
                "category": "theft",
            },
            {
                "text": "The company violated the contract by delivering late.",
                "bond_type": "VIOLATION",
                "category": "breach",
            },
            {
                "text": "She spread false rumors to destroy his reputation.",
                "bond_type": "HARM",
                "category": "slander",
            },
            {
                "text": "The politician broke his campaign promises after election.",
                "bond_type": "VIOLATION",
                "category": "betrayal",
            },
            {
                "text": "He poisoned the well that the village depended on.",
                "bond_type": "HARM",
                "category": "sabotage",
            },
            {
                "text": "The trustee embezzled funds from the charity.",
                "bond_type": "VIOLATION",
                "category": "fraud",
            },
            {
                "text": "She abandoned her children to pursue her own interests.",
                "bond_type": "VIOLATION",
                "category": "abandonment",
            },
            {
                "text": "The invaders destroyed the sacred temple.",
                "bond_type": "HARM",
                "category": "desecration",
            },
            # FAIRNESS / JUSTICE (8 scenarios)
            {
                "text": "The judge ruled fairly, giving each side equal consideration.",
                "bond_type": "FAIRNESS",
                "category": "impartiality",
            },
            {
                "text": "She forgave him for breaking his promise.",
                "bond_type": "FORGIVENESS",
                "category": "mercy",
            },
            {
                "text": "The council distributed resources equally among all villages.",
                "bond_type": "FAIRNESS",
                "category": "equity",
            },
            {
                "text": "He returned the extra change the shopkeeper gave by mistake.",
                "bond_type": "FAIRNESS",
                "category": "honesty",
            },
            {
                "text": "The elder mediated the dispute without favoring either party.",
                "bond_type": "FAIRNESS",
                "category": "mediation",
            },
            {
                "text": "She gave credit to her assistant for the discovery.",
                "bond_type": "FAIRNESS",
                "category": "attribution",
            },
            {
                "text": "The king pardoned the rebels who surrendered peacefully.",
                "bond_type": "FORGIVENESS",
                "category": "clemency",
            },
            {
                "text": "They compensated the wrongly accused man for his suffering.",
                "bond_type": "FAIRNESS",
                "category": "restitution",
            },
            # CROSS-CULTURAL BOND TYPES (8 scenarios)
            {
                "text": "The student honored his teacher by caring for him in old age.",
                "bond_type": "PIETY",
                "category": "filial",
            },
            {
                "text": "She upheld the family honor by keeping her grandfather's promise.",
                "bond_type": "LOYALTY",
                "category": "ancestral",
            },
            {
                "text": "The warrior spared his defeated enemy as custom demanded.",
                "bond_type": "HONOR",
                "category": "chivalry",
            },
            {
                "text": "He returned the sacred artifact to the temple it was taken from.",
                "bond_type": "REVERENCE",
                "category": "restoration",
            },
            {
                "text": "The host provided shelter to the stranger as hospitality required.",
                "bond_type": "HOSPITALITY",
                "category": "xenia",
            },
            {
                "text": "She maintained ritual purity to preserve cosmic order.",
                "bond_type": "PURITY",
                "category": "ritual",
            },
            {
                "text": "The merchant kept his word even when it meant financial loss.",
                "bond_type": "INTEGRITY",
                "category": "commercial",
            },
            {
                "text": "The community shunned him for violating the ancestral taboo.",
                "bond_type": "TABOO",
                "category": "prohibition",
            },
        ]

        # ====================================================================
        # PERTURBATION GENERATORS
        # ====================================================================

        # Name substitution pools for variety
        NAME_POOLS = {
            "male": ["John", "Michael", "David", "James", "Robert", "William", "Thomas", "Daniel"],
            "female": ["Mary", "Sarah", "Emma", "Lisa", "Anna", "Rachel", "Rebecca", "Hannah"],
        }

        IRRELEVANT_DETAILS = [
            " It was Tuesday.",
            " The room was blue.",
            " Last summer.",
            " The weather was pleasant.",
            " It happened at noon.",
            " The year was uncertain.",
            " Birds sang nearby.",
            " The moon was full.",
            " Rain had fallen earlier.",
            " The road was dusty.",
            " Flowers bloomed outside.",
        ]

        SYNONYMS = {
            "money": ["cash", "funds", "currency"],
            "groceries": ["bags", "supplies", "provisions"],
            "house": ["home", "dwelling", "residence"],
            "promise": ["vow", "pledge", "commitment"],
            "help": ["assist", "aid", "support"],
            "truth": ["facts", "reality", "honesty"],
        }

        def surface_perturbations(scenario: dict) -> list[dict]:
            """Generate surface perturbations that shouldn't change moral meaning."""
            text = scenario["text"]
            perturbs = []

            # Name changes (multiple variations)
            for old_name in NAME_POOLS["male"] + NAME_POOLS["female"]:
                if old_name in text:
                    for new_name in (
                        NAME_POOLS["male"]
                        if old_name in NAME_POOLS["male"]
                        else NAME_POOLS["female"]
                    ):
                        if new_name != old_name:
                            new_text = text.replace(old_name, new_name)
                            if new_text != text:
                                perturbs.append(
                                    {
                                        "text": new_text,
                                        "type": "name_change",
                                        "original": old_name,
                                        "new": new_name,
                                    }
                                )
                                if len(perturbs) >= 3:  # Limit per scenario
                                    break

            # Irrelevant detail additions
            # v10.16.3: Limit to 1 random detail per scenario for balanced comparison
            # (was 4, causing 160 samples vs 36 structural - skewing results)
            detail = random.choice(IRRELEVANT_DETAILS[:4])
            perturbs.append(
                {"text": text + detail, "type": "irrelevant_detail", "detail": detail}
            )

            # Synonym substitutions
            new_text = text
            for word, synonyms in SYNONYMS.items():
                if word in new_text.lower():
                    for syn in synonyms[:2]:
                        test_text = new_text.replace(word, syn)
                        if test_text != new_text:
                            perturbs.append(
                                {"text": test_text, "type": "synonym", "original": word, "new": syn}
                            )
                            break

            return perturbs

        def structural_perturbations(scenario: dict) -> list[dict]:
            """Generate structural perturbations that SHOULD change moral meaning."""
            text = scenario["text"]
            perturbs = []

            # Role swaps (agent/patient reversal)
            role_swaps = [
                ("John borrowed money from Mary", "Mary borrowed money from John"),
                (
                    "He stole the wallet from the elderly woman",
                    "The elderly woman stole the wallet from him",
                ),
                (
                    "She spread false rumors to destroy his reputation",
                    "He spread false rumors to destroy her reputation",
                ),
                ("Sarah helped her neighbor", "Her neighbor helped Sarah"),
                ("The teacher promised to grade", "The students promised to grade"),
                ("He gave his coat to the homeless man", "The homeless man gave his coat to him"),
                ("She donated her savings to help", "They donated their savings to help her"),
                (
                    "The host provided shelter to the stranger",
                    "The stranger provided shelter to the host",
                ),
            ]
            for orig, swap in role_swaps:
                if orig in text:
                    perturbs.append(
                        {
                            "text": text.replace(orig, swap),
                            "type": "role_swap",
                            "swap": (orig, swap),
                        }
                    )

            # Obligation to permission
            obl_to_perm = [
                ("must protect", "may protect"),
                ("has a duty to", "is allowed to"),
                ("are required to", "are permitted to"),
                ("swore to", "considered whether to"),
                ("must pay", "may pay"),
                ("agreed to", "considered whether to"),
            ]
            for obl, perm in obl_to_perm:
                if obl in text:
                    perturbs.append(
                        {
                            "text": text.replace(obl, perm),
                            "type": "obligation_to_permission",
                            "change": (obl, perm),
                        }
                    )

            # Positive to negative (harm introduction)
            pos_to_neg = [
                ("helped", "refused to help"),
                ("ruled fairly", "ruled unfairly"),
                ("forgave", "refused to forgive"),
                ("stayed late to comfort", "left early despite"),
                ("donated", "hoarded"),
                ("guided", "misled"),
                ("gave", "took"),
                ("stopped to help", "drove past without helping"),
                ("listened patiently", "ignored"),
                ("gathered to rebuild", "refused to rebuild"),
            ]
            for pos, neg in pos_to_neg:
                if pos in text:
                    perturbs.append(
                        {"text": text.replace(pos, neg), "type": "add_harm", "change": (pos, neg)}
                    )

            # Violation to fulfillment
            viol_to_fulf = [
                ("violated", "honored"),
                ("stole", "returned"),
                ("breaking", "keeping"),
                ("spread false rumors", "defended his reputation"),
                ("broke his campaign promises", "kept his campaign promises"),
                ("poisoned", "purified"),
                ("embezzled", "safeguarded"),
                ("abandoned", "cared for"),
                ("destroyed", "preserved"),
            ]
            for viol, fulf in viol_to_fulf:
                if viol in text:
                    perturbs.append(
                        {
                            "text": text.replace(viol, fulf),
                            "type": "violation_to_fulfillment",
                            "change": (viol, fulf),
                        }
                    )

            return perturbs

        # ====================================================================
        # STATISTICAL ANALYSIS FUNCTIONS
        # ====================================================================

        def bootstrap_ci(
            data: np.ndarray, n_bootstrap: int = 1000, confidence: float = 0.95
        ) -> tuple[float, float, float]:
            """Calculate bootstrap confidence interval."""
            n = len(data)
            if n < 2:
                return data.mean(), data.mean(), data.mean()

            boot_means = []
            for _ in range(n_bootstrap):
                sample = np.random.choice(data, size=n, replace=True)
                boot_means.append(sample.mean())

            boot_means = np.array(boot_means)
            alpha = (1 - confidence) / 2
            lower = np.percentile(boot_means, alpha * 100)
            upper = np.percentile(boot_means, (1 - alpha) * 100)

            return lower, data.mean(), upper

        def effect_size_cohens_d(group1: np.ndarray, group2: np.ndarray) -> float:
            """Calculate Cohen's d effect size."""
            n1, n2 = len(group1), len(group2)
            var1, var2 = group1.var(), group2.var()
            pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
            return (group1.mean() - group2.mean()) / (pooled_std + 1e-9)

        # ====================================================================
        # RUN TESTS
        # ====================================================================

        print("=" * 70)
        print("RUNNING FUZZ TESTS")
        print("=" * 70)
        print()

        # Organize results by perturbation type
        results_by_type = {
            "structural_obligation_to_permission": [],
            "structural_add_harm": [],
            "structural_role_swap": [],
            "structural_violation_to_fulfillment": [],
            "surface_name_change": [],
            "surface_irrelevant_detail": [],
            "surface_synonym": [],
        }

        all_surface_distances = []
        all_structural_distances = []

        scenarios_to_run = BASE_SCENARIOS[: RUNTIME["max_scenarios"]]
        print(f"Processing {len(scenarios_to_run)} scenarios...")
        print()

        for i, scenario in enumerate(scenarios_to_run):
            base_emb = get_embedding(scenario["text"])

            # Process surface perturbations
            surface_perturbs = surface_perturbations(scenario)
            for p in surface_perturbs:
                dist = cosine_distance(base_emb, get_embedding(p["text"]))
                all_surface_distances.append(dist)
                key = f"surface_{p['type']}"
                if key in results_by_type:
                    results_by_type[key].append(dist)

            # Process structural perturbations
            structural_perturbs = structural_perturbations(scenario)
            for p in structural_perturbs:
                dist = cosine_distance(base_emb, get_embedding(p["text"]))
                all_structural_distances.append(dist)
                key = f"structural_{p['type']}"
                if key in results_by_type:
                    results_by_type[key].append(dist)

            if (i + 1) % 10 == 0:
                print(f"  Processed {i + 1}/{len(scenarios_to_run)} scenarios...")

        print()
        print(f"Total surface perturbations: {len(all_surface_distances)}")
        print(f"Total structural perturbations: {len(all_structural_distances)}")
        print()

        # ====================================================================
        # DETAILED RESULTS
        # ====================================================================

        print("=" * 70)
        print("RESULTS BY PERTURBATION TYPE")
        print("=" * 70)
        print()

        fuzz_results = {}

        for ptype, distances in results_by_type.items():
            if len(distances) > 0:
                distances = np.array(distances)
                lower, mean, upper = bootstrap_ci(distances, RUNTIME["bootstrap_samples"])
                fuzz_results[ptype] = {
                    "mean_distance": str(mean),
                    "std": str(distances.std()),
                    "ci_lower": str(lower),
                    "ci_upper": str(upper),
                    "n": len(distances),
                }
                category = "STRUCTURAL" if "structural" in ptype else "SURFACE"
                print(f"{ptype}:")
                print(f"  n={len(distances)}, mean={mean:.4f}, std={distances.std():.4f}")
                print(f"  95% CI: [{lower:.4f}, {upper:.4f}]")
                print()

        # ====================================================================
        # AGGREGATE COMPARISON
        # ====================================================================

        print("=" * 70)
        print("AGGREGATE COMPARISON")
        print("=" * 70)
        print()

        surface_arr = np.array(all_surface_distances)
        structural_arr = np.array(all_structural_distances)

        surf_lower, surf_mean, surf_upper = bootstrap_ci(surface_arr, RUNTIME["bootstrap_samples"])
        struct_lower, struct_mean, struct_upper = bootstrap_ci(
            structural_arr, RUNTIME["bootstrap_samples"]
        )

        print("Surface (should be SMALL):")
        print(f"  mean={surf_mean:.4f}, std={surface_arr.std():.4f}")
        print(f"  95% CI: [{surf_lower:.4f}, {surf_upper:.4f}]")
        print()
        print("Structural (should be LARGE):")
        print(f"  mean={struct_mean:.4f}, std={structural_arr.std():.4f}")
        print(f"  95% CI: [{struct_lower:.4f}, {struct_upper:.4f}]")
        print()

        # Statistical tests
        from scipy import stats

        t_stat, p_value = stats.ttest_ind(structural_arr, surface_arr)

        # Mann-Whitney U for non-parametric comparison
        u_stat, u_pvalue = stats.mannwhitneyu(structural_arr, surface_arr, alternative="greater")

        ratio = struct_mean / (surf_mean + 1e-9)
        cohens_d = effect_size_cohens_d(structural_arr, surface_arr)

        print(f"Ratio (structural/surface): {ratio:.2f}x")
        print(f"Cohen's d effect size: {cohens_d:.3f}")
        print(f"t-statistic: {t_stat:.4f}, p-value: {p_value:.6f}")
        print(f"Mann-Whitney U: {u_stat:.0f}, p-value: {u_pvalue:.6f}")
        print()

        # Store comparison results
        fuzz_results["comparison"] = {
            "structural_mean": str(struct_mean),
            "structural_ci": [str(struct_lower), str(struct_upper)],
            "surface_mean": str(surf_mean),
            "surface_ci": [str(surf_lower), str(surf_upper)],
            "ratio": str(ratio),
            "cohens_d": str(cohens_d),
            "t_statistic": t_stat,
            "p_value": p_value,
            "mann_whitney_u": float(u_stat),
            "mann_whitney_p": float(u_pvalue),
            "n_structural": len(structural_arr),
            "n_surface": len(surface_arr),
        }

        # ====================================================================
        # VERDICT (RUNTIME-ADAPTIVE THRESHOLDS)
        # ====================================================================

        print("=" * 70)
        print("VERDICT")
        print("=" * 70)
        print()

        # Adaptive thresholds based on sample size and runtime
        if len(structural_arr) >= 30 and len(surface_arr) >= 30:
            # Strong statistical power - use stricter thresholds
            strong_ratio = 3.0
            moderate_ratio = 2.0
            weak_ratio = 1.5
            p_threshold = 0.01
        elif len(structural_arr) >= 15:
            # Medium statistical power
            strong_ratio = 2.5
            moderate_ratio = 1.8
            weak_ratio = 1.3
            p_threshold = 0.05
        else:
            # Low statistical power - use looser thresholds but note uncertainty
            strong_ratio = 2.0
            moderate_ratio = 1.5
            weak_ratio = 1.2
            p_threshold = 0.10

        verdict = "NOT_SUPPORTED"
        verdict_detail = ""

        if ratio >= strong_ratio and p_value < p_threshold and cohens_d > 0.8:
            verdict = "STRONG_SUPPORT"
            verdict_detail = f"Model learned moral structure (ratio={ratio:.1f}x, d={cohens_d:.2f}, p={p_value:.4f})"
        elif ratio >= moderate_ratio and p_value < 0.05 and cohens_d > 0.5:
            verdict = "MODERATE_SUPPORT"
            verdict_detail = f"Evidence for moral structure (ratio={ratio:.1f}x, d={cohens_d:.2f})"
        elif ratio >= weak_ratio and p_value < 0.10:
            verdict = "WEAK_SUPPORT"
            verdict_detail = f"Weak evidence (ratio={ratio:.1f}x, needs more data)"
        else:
            verdict = "NOT_SUPPORTED"
            verdict_detail = "May be encoding surface features rather than moral structure"

        print(f"Verdict: {verdict}")
        print(f"Detail: {verdict_detail}")
        print()
        print(f"Runtime: {RUNTIME['gpu_type']}")
        print(
            f"Thresholds used: strong>{strong_ratio}x, moderate>{moderate_ratio}x, p<{p_threshold}"
        )
        print("=" * 70)

        fuzz_results["verdict"] = verdict
        fuzz_results["verdict_detail"] = verdict_detail
        fuzz_results["runtime"] = RUNTIME

        # Make results available for integration
        FUZZ_RESULTS_V1011 = fuzz_results


# ========================================================================
# v10.14.3: CROSS-LINGUAL FUZZ TESTING
# ========================================================================
CROSS_LINGUAL_TEST_PAIRS = [
    (
        "English",
        "Hebrew",
        "Promise keeping",
        "A person promised to return a borrowed book and must fulfill that promise.",
        "אדם הבטיח להחזיר ספר שאול וחייב לקיים את ההבטחה.",
    ),
    (
        "English",
        "Arabic",
        "Duty to help",
        "One has a duty to help those in distress when able to do so.",
        "على المرء واجب مساعدة المحتاجين عندما يكون قادراً على ذلك.",
    ),
    (
        "English",
        "Chinese",
        "Filial obligation",
        "Children have an obligation to care for their elderly parents.",
        "子女有義務照顧年邁的父母。",
    ),
]


def run_cross_lingual_fuzz_test(model, tokenizer, device):
    """Test cross-lingual invariance of z_bond embeddings."""
    print("\n" + "=" * 70)
    print("CROSS-LINGUAL FUZZ TEST (v10.14.3)")
    print("=" * 70)

    model.eval()
    results = []

    with torch.no_grad():
        for lang1, lang2, desc, text1, text2 in CROSS_LINGUAL_TEST_PAIRS:
            enc1 = tokenizer(
                text1, return_tensors="pt", padding=True, truncation=True, max_length=128
            )
            enc2 = tokenizer(
                text2, return_tensors="pt", padding=True, truncation=True, max_length=128
            )
            enc1 = {k: v.to(device) for k, v in enc1.items()}
            enc2 = {k: v.to(device) for k, v in enc2.items()}

            out1 = model(enc1["input_ids"], enc1.get("attention_mask"))
            out2 = model(enc2["input_ids"], enc2.get("attention_mask"))

            cos_sim = F.cosine_similarity(out1["z"], out2["z"], dim=-1).item()
            results.append({"langs": f"{lang1}-{lang2}", "cos_sim": cos_sim, "desc": desc})
            print(f"  {lang1:8s} <-> {lang2:8s} | cos_sim={cos_sim:+.4f} | {desc}")

    avg_sim = sum(r["cos_sim"] for r in results) / len(results)
    print(f"\nAverage cross-lingual similarity: {avg_sim:+.4f}")
    if avg_sim > 0.7:
        print("  ✓ Good cross-lingual invariance")
    else:
        print("  ✗ Poor cross-lingual invariance")
    return {"results": results, "avg_similarity": avg_sim}


# Run if model available
if RUN_FUZZ_TEST and _fuzz_model is not None:
    cross_lingual_results = run_cross_lingual_fuzz_test(_fuzz_model, tokenizer, device)

FUZZ TESTING v10.12: STRUCTURAL VS SURFACE PERTURBATIONS

Runtime detected: L4 (22.2 GB VRAM)
Batch size: 32, Max scenarios: 75
Bootstrap samples: 2000



NameError: name 'SAVE_DIR' is not defined

In [ ]:
# @title 10. Save & Download Results { display-mode: "form" }
# @markdown Persist results to Google Drive and optionally download as zip

import shutil

print("=" * 60)
print("SAVING RESULTS")
print("=" * 60)

# Always persist results to Drive
if SAVE_DIR and os.path.exists(SAVE_DIR):
    print(f"\nPersisting to: {SAVE_DIR}")

    # Save final results JSON
    if os.path.exists("results/final_results.json"):
        dest = f"{SAVE_DIR}/final_results.json"
        shutil.copy("results/final_results.json", dest)
        print("  Saved: final_results.json")

    # Save splits config
    if os.path.exists("data/splits/all_splits.json"):
        dest = f"{SAVE_DIR}/all_splits.json"
        shutil.copy("data/splits/all_splits.json", dest)
        print("  Saved: all_splits.json")

    # Models are already saved to SAVE_DIR during training
    model_files = [f for f in os.listdir(SAVE_DIR) if f.endswith(".pt")]
    if model_files:
        print(f"  Models already in Drive: {len(model_files)} files")
        for mf in model_files[:5]:
            print(f"    - {mf}")
        if len(model_files) > 5:
            print(f"    ... and {len(model_files) - 5} more")

    print(f"\nResults persisted to Google Drive: {SAVE_DIR}")
else:
    print("WARNING: SAVE_DIR not available, results only in local directories")

# Optional: Create download zip
if CREATE_DOWNLOAD_ZIP:
    import zipfile

    zip_path = f"BIP_v{BIP_VERSION}_results.zip"
    print("\n" + "-" * 60)
    print("Creating download package...")

    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        # Results
        if os.path.exists("results/final_results.json"):
            zf.write("results/final_results.json")

        # Models (from Drive)
        if SAVE_DIR and os.path.exists(SAVE_DIR):
            for f in os.listdir(SAVE_DIR):
                if f.endswith(".pt"):
                    zf.write(f"{SAVE_DIR}/{f}", f"models/{f}")

        # Config
        if os.path.exists("data/splits/all_splits.json"):
            zf.write("data/splits/all_splits.json")

    print(f"Download package ready: {zip_path}")

    # Download in Colab, or show path otherwise
    try:
        from google.colab import files

        files.download(zip_path)
    except ImportError:
        print(f"Not running in Colab. Zip saved to: {os.path.abspath(zip_path)}")
else:
    print("\n(Zip download disabled - set CREATE_DOWNLOAD_ZIP=True in cell 1 to enable)")

print("\n" + "=" * 60)
print("COMPLETE")
print("=" * 60)

SAVING RESULTS

Persisting to: /content/drive/MyDrive/BIP_v10.16.5
  Saved: all_splits.json
  Models already in Drive: 11 files
    - best_confucian_to_buddhist.pt
    - best_daoist_to_buddhist.pt
    - best_hebrew_to_others.pt
    - best_semitic_to_indic.pt
    - best_ancient_to_modern.pt
    ... and 6 more

Results persisted to Google Drive: /content/drive/MyDrive/BIP_v10.16.5

(Zip download disabled - set CREATE_DOWNLOAD_ZIP=True in cell 1 to enable)

COMPLETE
